# 자동차 환경 빅데이터 3차
- 과제2 : 건설기계 배출가스 관리 체계 구축
- 과제4 : 배출가스 전문정비제도 관리체계 구축
- 과제5 : 자동차 환경 빅데이터 제작사 공동 활용체게 구축

# Library

In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime
import pyexasol

In [2]:
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

In [3]:
# 경고문 없애는 코드
# UserWarning: Workbook contains no default style, apply openpyxl's default warn("Workbook contains no default style, apply openpyxl's default")
import warnings
warnings.simplefilter("ignore")

# Fold

In [4]:
pub_fold = 'D:/data/processing/big3/raw/public'
raw_fold = 'D:/data/processing/big3/raw'
raw2_fold = 'D:/data/processing/big3/raw/BD2'
raw4_fold = 'D:/data/processing/big3/raw/BD4'
raw5_fold = 'D:/data/processing/big3/raw/BD5'
an_fold = 'D:/data/processing/big3/analysis'
an2_fold = 'D:/data/processing/big3/analysis/BD2'
an4_fold = 'D:/data/processing/big3/analysis/BD4'
an5_fold = 'D:/data/processing/big3/analysis/BD5'
df_fold = 'D:/data/processing/big3/df'
df2_fold = 'D:/data/processing/big3/df/BD2'
df4_fold = 'D:/data/processing/big3/df/BD4'
df5_fold = 'D:/data/processing/big3/df/BD5'
folds = {
    'pub_fold':pub_fold, 
    'raw_fold':raw_fold, 
    'raw2_fold':raw2_fold, 
    'raw4_fold':raw4_fold, 
    'raw5_fold':raw5_fold, 
    'an_fold':an_fold, 
    'an2_fold:':an2_fold, 
    'an4_fold:':an4_fold, 
    'an5_fold:':an5_fold, 
    'df_fold':df_fold, 
    'df2_fold':df2_fold, 
    'df4_fold':df4_fold, 
    'df5_fold':df5_fold, 
    }
folds

{'pub_fold': 'D:/data/processing/big3/raw/public',
 'raw_fold': 'D:/data/processing/big3/raw',
 'raw2_fold': 'D:/data/processing/big3/raw/BD2',
 'raw4_fold': 'D:/data/processing/big3/raw/BD4',
 'raw5_fold': 'D:/data/processing/big3/raw/BD5',
 'an_fold': 'D:/data/processing/big3/analysis',
 'an2_fold:': 'D:/data/processing/big3/analysis/BD2',
 'an4_fold:': 'D:/data/processing/big3/analysis/BD4',
 'an5_fold:': 'D:/data/processing/big3/analysis/BD5',
 'df_fold': 'D:/data/processing/big3/df',
 'df2_fold': 'D:/data/processing/big3/df/BD2',
 'df4_fold': 'D:/data/processing/big3/df/BD4',
 'df5_fold': 'D:/data/processing/big3/df/BD5'}

# server

In [5]:
# exasol db
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSE')

# EDA
+ 과제2
    - 건설기계 인증자료
    - 건설기계 인증대장
+ 과제4
    - 자동차전문정비 : STD_TB_ERR_RES
    - 자동차전문정비상세 : STD_TB_ERR_RES_DET
+ 과제5
    - 구조변경검사 : STD_DLM_TB_ERP_GUBUN_BAECHUL

# 과제2

## 로드

In [ ]:
# test kbj

### 등록정보(STD_CEG_CAR_MIG)

In [299]:
# 5m 31.4s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD FROM STD_CEG_CAR_MIG;")
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자', 
    'VHCL_MNG_NO':'차량관리번호', 
    'VHRNO':'자동차등록번호',
    'EXTGAS_INSP_VLD_YMD':'검사유효일',
    'VHCL_OWNR_CL_CD':'소유자구분',  
}
carr = car.rename(columns=car_ch_col)

print('data load : STD_CEG_CAR_MIG')

data load : STD_CEG_CAR_MIG


#### 데이터 확인

In [300]:
carr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30625663 entries, 0 to 30625662
Data columns (total 15 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   차대번호      object
 1   법정동코드     int64 
 2   배출가스등급    object
 3   배출가스인증번호  object
 4   차량말소YN    object
 5   제원관리번호    object
 6   차량연식      int64 
 7   차종        object
 8   용도        object
 9   최초등록일자    object
 10  제작일자      object
 11  차량관리번호    object
 12  자동차등록번호   object
 13  검사유효일     int64 
 14  소유자구분     object
dtypes: int64(3), object(12)
memory usage: 3.4+ GB


In [302]:
cols_df = pd.DataFrame({'컬럼명':car_ch_col.keys(), '컬럼설명':car_ch_col.values()})
cols_df

,컬럼명,컬럼설명
0,VIN,차대번호
1,BSPL_STDG_CD,법정동코드
2,EXHST_GAS_GRD_CD,배출가스등급
3,EXHST_GAS_CERT_NO,배출가스인증번호
4,VHCL_ERSR_YN,차량말소YN
5,MANP_MNG_NO,제원관리번호
6,YRIDNW,차량연식
7,VHCTY_CD,차종
8,PURPS_CD2,용도
9,FRST_REG_YMD,최초등록일자


In [304]:
carr_stat = carr.isnull().sum().reset_index().rename(columns={'index':'컬럼설명', 0:'빈 값 개수'})
carr_stat['총 행수'] = carr.shape[0]
carr_stat.insert(0, '번호', [x for x in range(1, carr_stat.shape[0] + 1)])
carr_stat_final = carr_stat.merge(cols_df, on='컬럼설명', how='left')
carr_stat_final = carr_stat_final[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
carr_stat_final

,번호,컬럼명,컬럼설명,빈 값 개수,총 행수
0,1,VIN,차대번호,0,30625663
1,2,BSPL_STDG_CD,법정동코드,0,30625663
2,3,EXHST_GAS_GRD_CD,배출가스등급,1060024,30625663
3,4,EXHST_GAS_CERT_NO,배출가스인증번호,3860506,30625663
4,5,VHCL_ERSR_YN,차량말소YN,0,30625663
5,6,MANP_MNG_NO,제원관리번호,0,30625663
6,7,YRIDNW,차량연식,0,30625663
7,8,VHCTY_CD,차종,0,30625663
8,9,PURPS_CD2,용도,0,30625663
9,10,FRST_REG_YMD,최초등록일자,18,30625663


##### [출력] 데이터 정보

In [320]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.11.14'

In [316]:
carr_stat_final.to_excel(os.path.join(an2_fold, f'데이터정보_등록정보({today_date}).xlsx'), index=False)

### 제원정보(STD_CEG_CAR_SRC_MIG)

In [305]:
# 4.1s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY, VHCTY_CL_CD, TOTL_WGHT, CRYNG_WGHT, DSPLVL, EGIN_OTPT FROM STD_CEG_CAR_SRC_MIG;")
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'VHCTY_CL_CD':'차종분류',
    'TOTL_WGHT':'총중량',
    'CRYNG_WGHT':'적재중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',
}
srcr = src.rename(columns=src_ch_col)

print('data load : STD_CEG_CAR_SRC_MIG')

data load : STD_CEG_CAR_SRC_MIG


#### 데이터 확인

In [306]:
srcr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566615 entries, 0 to 566614
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   제원관리번호  566615 non-null  object 
 1   연료      545937 non-null  object 
 2   차종유형    553038 non-null  object 
 3   제작사명    559794 non-null  object 
 4   차명      566544 non-null  object 
 5   자동차형식   536460 non-null  object 
 6   엔진형식    517013 non-null  object 
 7   차종분류    551140 non-null  object 
 8   총중량     563088 non-null  float64
 9   적재중량    461878 non-null  float64
 10  배기량     530949 non-null  float64
 11  엔진출력    491335 non-null  float64
dtypes: float64(4), object(8)
memory usage: 51.9+ MB


In [307]:
cols_df = pd.DataFrame({'컬럼명':src_ch_col.keys(), '컬럼설명':src_ch_col.values()})
cols_df

,컬럼명,컬럼설명
0,MANP_MNG_NO,제원관리번호
1,FUEL_CD,연료
2,VHCTY_TY_CD2,차종유형
3,MNFCTR_NM,제작사명
4,VHCNM,차명
5,VHCL_FRM,자동차형식
6,EGIN_TY,엔진형식
7,VHCTY_CL_CD,차종분류
8,TOTL_WGHT,총중량
9,CRYNG_WGHT,적재중량


In [308]:
srcr_stat = srcr.isnull().sum().reset_index().rename(columns={'index':'컬럼설명', 0:'빈 값 개수'})
srcr_stat['총 행수'] = srcr.shape[0]
srcr_stat.insert(0, '번호', [x for x in range(1, srcr_stat.shape[0] + 1)])
srcr_stat_final = srcr_stat.merge(cols_df, on='컬럼설명', how='left')
srcr_stat_final = srcr_stat_final[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
srcr_stat_final

,번호,컬럼명,컬럼설명,빈 값 개수,총 행수
0,1,MANP_MNG_NO,제원관리번호,0,566615
1,2,FUEL_CD,연료,20678,566615
2,3,VHCTY_TY_CD2,차종유형,13577,566615
3,4,MNFCTR_NM,제작사명,6821,566615
4,5,VHCNM,차명,71,566615
5,6,VHCL_FRM,자동차형식,30155,566615
6,7,EGIN_TY,엔진형식,49602,566615
7,8,VHCTY_CL_CD,차종분류,15475,566615
8,9,TOTL_WGHT,총중량,3527,566615
9,10,CRYNG_WGHT,적재중량,104737,566615


##### [출력] 데이터 정보

In [314]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.11.14'

In [317]:
srcr_stat_final.to_excel(os.path.join(an2_fold, f'데이터정보_제원정보({today_date}).xlsx'), index=False)

### 정기검사(STD_TB_JGT_HIS)

In [309]:
# 3m 34.9s
# jgt = wb.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_JT_YN1, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1 FROM STD_TB_JGT_HIS WHERE ROWNUM <= 10000;") # 테스트용
jgt = wd.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_MSTVL4, FDRM_NLOD_SMO_MSTVL5, FDRM_NLOD_SMO_MSTVL6, FDRM_NLOD_SMO_JT_YN1, FDRM_NLOD_SMO_JT_YN2, FDRM_NLOD_SMO_JT_YN3, FDRM_NLOD_SMO_JT_YN4, FDRM_NLOD_SMO_JT_YN5, FDRM_NLOD_SMO_JT_YN6, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1, FDRM_NLOD_SMO_PRMVL2, FDRM_NLOD_SMO_PRMVL3, FDRM_NLOD_SMO_PRMVL4, FDRM_NLOD_SMO_PRMVL5, FDRM_NLOD_SMO_PRMVL6 FROM STD_TB_JGT_HIS;")
jgt_ch_col = {
    'VIN':'차대번호', 
    'FDRM_INSP_INSP_MTHD_CD':'검사방법', 
    'FDRM_INSP_KND_CD':'검사종류', 
    'FDRM_INSP_JGMT':'검사판정', 
    'FDRM_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'FDRM_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'FDRM_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'FDRM_NLOD_SMO_MSTVL4':'무부하매연측정치4', 
    'FDRM_NLOD_SMO_MSTVL5':'무부하매연측정치5', 
    'FDRM_NLOD_SMO_MSTVL6':'무부하매연측정치6', 
    'FDRM_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'FDRM_NLOD_SMO_JT_YN2':'무부하매연판정2', 
    'FDRM_NLOD_SMO_JT_YN3':'무부하매연판정3', 
    'FDRM_NLOD_SMO_JT_YN4':'무부하매연판정4', 
    'FDRM_NLOD_SMO_JT_YN5':'무부하매연판정5', 
    'FDRM_NLOD_SMO_JT_YN6':'무부하매연판정6', 
    'FDRM_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
    'FDRM_NLOD_SMO_PRMVL2':'무부하매연허용치2', 
    'FDRM_NLOD_SMO_PRMVL3':'무부하매연허용치3', 
    'FDRM_NLOD_SMO_PRMVL4':'무부하매연허용치4', 
    'FDRM_NLOD_SMO_PRMVL5':'무부하매연허용치5', 
    'FDRM_NLOD_SMO_PRMVL6':'무부하매연허용치6', 
    'FDRM_INSP_YMD':'검사일자',
    'FDRM_DRVNG_DSTNC':'주행거리',
}
jgtr = jgt.rename(columns=jgt_ch_col)

print('data load : STD_CEG_CAR_SRC_MIG')

data load : STD_CEG_CAR_SRC_MIG


#### 데이터 확인

In [310]:
jgtr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65903838 entries, 0 to 65903837
Data columns (total 24 columns):
 #   Column     Dtype  
---  ------     -----  
 0   차대번호       object 
 1   검사방법       object 
 2   검사종류       object 
 3   검사판정       object 
 4   무부하매연측정치1  float64
 5   무부하매연측정치2  float64
 6   무부하매연측정치3  float64
 7   무부하매연측정치4  float64
 8   무부하매연측정치5  float64
 9   무부하매연측정치6  float64
 10  무부하매연판정1   object 
 11  무부하매연판정2   object 
 12  무부하매연판정3   object 
 13  무부하매연판정4   object 
 14  무부하매연판정5   object 
 15  무부하매연판정6   object 
 16  검사일자       int64  
 17  주행거리       float64
 18  무부하매연허용치1  float64
 19  무부하매연허용치2  float64
 20  무부하매연허용치3  float64
 21  무부하매연허용치4  float64
 22  무부하매연허용치5  float64
 23  무부하매연허용치6  float64
dtypes: float64(13), int64(1), object(10)
memory usage: 11.8+ GB


In [311]:
cols_df = pd.DataFrame({'컬럼명':jgt_ch_col.keys(), '컬럼설명':jgt_ch_col.values()})
cols_df

,컬럼명,컬럼설명
0,VIN,차대번호
1,FDRM_INSP_INSP_MTHD_CD,검사방법
2,FDRM_INSP_KND_CD,검사종류
3,FDRM_INSP_JGMT,검사판정
4,FDRM_NLOD_SMO_MSTVL1,무부하매연측정치1
5,FDRM_NLOD_SMO_MSTVL2,무부하매연측정치2
6,FDRM_NLOD_SMO_MSTVL3,무부하매연측정치3
7,FDRM_NLOD_SMO_MSTVL4,무부하매연측정치4
8,FDRM_NLOD_SMO_MSTVL5,무부하매연측정치5
9,FDRM_NLOD_SMO_MSTVL6,무부하매연측정치6


In [312]:
jgtr_stat = jgtr.isnull().sum().reset_index().rename(columns={'index':'컬럼설명', 0:'빈 값 개수'})
jgtr_stat['총 행수'] = jgtr.shape[0]
jgtr_stat.insert(0, '번호', [x for x in range(1, jgtr_stat.shape[0] + 1)])
jgtr_stat_final = jgtr_stat.merge(cols_df, on='컬럼설명', how='left')
jgtr_stat_final = jgtr_stat_final[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
jgtr_stat_final

,번호,컬럼명,컬럼설명,빈 값 개수,총 행수
0,1,VIN,차대번호,0,65903838
1,2,FDRM_INSP_INSP_MTHD_CD,검사방법,0,65903838
2,3,FDRM_INSP_KND_CD,검사종류,0,65903838
3,4,FDRM_INSP_JGMT,검사판정,0,65903838
4,5,FDRM_NLOD_SMO_MSTVL1,무부하매연측정치1,3377038,65903838
5,6,FDRM_NLOD_SMO_MSTVL2,무부하매연측정치2,40994887,65903838
6,7,FDRM_NLOD_SMO_MSTVL3,무부하매연측정치3,40994887,65903838
7,8,FDRM_NLOD_SMO_MSTVL4,무부하매연측정치4,45948127,65903838
8,9,FDRM_NLOD_SMO_MSTVL5,무부하매연측정치5,45948127,65903838
9,10,FDRM_NLOD_SMO_MSTVL6,무부하매연측정치6,45948127,65903838


##### [출력] 데이터 정보

In [318]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.11.14'

In [319]:
jgtr_stat_final.to_excel(os.path.join(an2_fold, f'데이터정보_정기정보({today_date}).xlsx'), index=False)

### 정밀검사(STD_TB_EET_HIS_ME)

In [6]:
# 10m 30.8s
eet = wd.export_to_pandas("SELECT VIN, PRCINSP_MSRMT_MTHD_CD, PRCINSP_KND_CD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PREC_NLOD_SMO_MSTVL4, PREC_NLOD_SMO_MSTVL5, PREC_NLOD_SMO_MSTVL6, PREC_NLOD_SMO_JT_YN1, PREC_NLOD_SMO_JT_YN2, PREC_NLOD_SMO_JT_YN3, PREC_NLOD_SMO_JT_YN4, PREC_NLOD_SMO_JT_YN5, PREC_NLOD_SMO_JT_YN6, PRCINSP_YMD, PRCINSP_DRVNG_DSTNC, PREC_NLOD_SMO_PRMVL1, PREC_NLOD_SMO_PRMVL2, PREC_NLOD_SMO_PRMVL3, PREC_NLOD_SMO_PRMVL4, PREC_NLOD_SMO_PRMVL5, PREC_NLOD_SMO_PRMVL6 FROM STD_TB_EET_HIS_ME;")
eet_ch_col = {
    'VIN':'차대번호', 
    'PRCINSP_MSRMT_MTHD_CD':'검사방법', 
    'PRCINSP_KND_CD':'검사종류', 
    'PRCINSP_JGMT':'검사판정', 
    'PREC_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'PREC_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'PREC_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'PREC_NLOD_SMO_MSTVL4':'무부하매연측정치4', 
    'PREC_NLOD_SMO_MSTVL5':'무부하매연측정치5', 
    'PREC_NLOD_SMO_MSTVL6':'무부하매연측정치6', 
    'PREC_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'PREC_NLOD_SMO_JT_YN2':'무부하매연판정2', 
    'PREC_NLOD_SMO_JT_YN3':'무부하매연판정3', 
    'PREC_NLOD_SMO_JT_YN4':'무부하매연판정4', 
    'PREC_NLOD_SMO_JT_YN5':'무부하매연판정5', 
    'PREC_NLOD_SMO_JT_YN6':'무부하매연판정6', 
    'PREC_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
    'PREC_NLOD_SMO_PRMVL2':'무부하매연허용치2', 
    'PREC_NLOD_SMO_PRMVL3':'무부하매연허용치3', 
    'PREC_NLOD_SMO_PRMVL4':'무부하매연허용치4', 
    'PREC_NLOD_SMO_PRMVL5':'무부하매연허용치5', 
    'PREC_NLOD_SMO_PRMVL6':'무부하매연허용치6', 
    'PRCINSP_YMD':'검사일자',
    'PRCINSP_DRVNG_DSTNC':'주행거리', 
}
eetr = eet.rename(columns=eet_ch_col)

print('data load : STD_TB_EET_HIS_ME')

data load : STD_TB_EET_HIS_ME


#### 데이터 확인

In [9]:
eetr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102191938 entries, 0 to 102191937
Data columns (total 24 columns):
 #   Column     Dtype  
---  ------     -----  
 0   차대번호       object 
 1   검사방법       object 
 2   검사종류       object 
 3   검사판정       object 
 4   무부하매연측정치1  float64
 5   무부하매연측정치2  float64
 6   무부하매연측정치3  float64
 7   무부하매연측정치4  float64
 8   무부하매연측정치5  float64
 9   무부하매연측정치6  float64
 10  무부하매연판정1   object 
 11  무부하매연판정2   object 
 12  무부하매연판정3   object 
 13  무부하매연판정4   object 
 14  무부하매연판정5   object 
 15  무부하매연판정6   object 
 16  검사일자       int64  
 17  주행거리       float64
 18  무부하매연허용치1  float64
 19  무부하매연허용치2  float64
 20  무부하매연허용치3  float64
 21  무부하매연허용치4  float64
 22  무부하매연허용치5  float64
 23  무부하매연허용치6  float64
dtypes: float64(13), int64(1), object(10)
memory usage: 18.3+ GB


In [10]:
cols_df = pd.DataFrame({'컬럼명':eet_ch_col.keys(), '컬럼설명':eet_ch_col.values()})
cols_df

,컬럼명,컬럼설명
0,VIN,차대번호
1,PRCINSP_MSRMT_MTHD_CD,검사방법
2,PRCINSP_KND_CD,검사종류
3,PRCINSP_JGMT,검사판정
4,PREC_NLOD_SMO_MSTVL1,무부하매연측정치1
5,PREC_NLOD_SMO_MSTVL2,무부하매연측정치2
6,PREC_NLOD_SMO_MSTVL3,무부하매연측정치3
7,PREC_NLOD_SMO_MSTVL4,무부하매연측정치4
8,PREC_NLOD_SMO_MSTVL5,무부하매연측정치5
9,PREC_NLOD_SMO_MSTVL6,무부하매연측정치6


In [11]:
# 
eetr_stat = eetr.isnull().sum().reset_index().rename(columns={'index':'컬럼설명', 0:'빈 값 개수'})
eetr_stat['총 행수'] = eetr.shape[0]
eetr_stat.insert(0, '번호', [x for x in range(1, eetr_stat.shape[0] + 1)])
eetr_stat_final = eetr_stat.merge(cols_df, on='컬럼설명', how='left')
eetr_stat_final = eetr_stat_final[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
eetr_stat_final

,번호,컬럼명,컬럼설명,빈 값 개수,총 행수
0,1,VIN,차대번호,0,102191938
1,2,PRCINSP_MSRMT_MTHD_CD,검사방법,7,102191938
2,3,PRCINSP_KND_CD,검사종류,19420,102191938
3,4,PRCINSP_JGMT,검사판정,123941,102191938
4,5,PREC_NLOD_SMO_MSTVL1,무부하매연측정치1,2407076,102191938
5,6,PREC_NLOD_SMO_MSTVL2,무부하매연측정치2,28362934,102191938
6,7,PREC_NLOD_SMO_MSTVL3,무부하매연측정치3,30952683,102191938
7,8,PREC_NLOD_SMO_MSTVL4,무부하매연측정치4,43941015,102191938
8,9,PREC_NLOD_SMO_MSTVL5,무부하매연측정치5,43941075,102191938
9,10,PREC_NLOD_SMO_MSTVL6,무부하매연측정치6,57716513,102191938


##### [출력] 데이터 정보

In [90]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.12.13'

In [91]:
# # 0.8s
# eetr_stat_final.to_excel(os.path.join(an2_fold, f'데이터정보_정밀정보({today_date}).xlsx'), index=False)

### 건설기계 인증자료(12개)

In [92]:
def construct_machine_certification_data_load(name, sheet_name=0):
    fold = os.path.join(raw2_fold, 'KENCIS 건설기계 인증자료_231018')
    try:
        file_name = f'{name}.xlsx'
        cert_dt = pd.read_excel(os.path.join(fold, file_name), sheet_name=sheet_name)
        return cert_dt
    except:
        file_name = f'{name}.xlsx'
        cert_dt = pd.read_excel(os.path.join(fold, file_name))
        return cert_dt

#### 정보 모음

In [93]:
cols = construct_machine_certification_data_load('KENCIS_건설기계인증정보_테이블명세서', '테이블정의서')
cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   테이블명    12 non-null     object
 1   테이블설명   12 non-null     object
 2   컬럼명     649 non-null    object
 3   컬럼설명    644 non-null    object
 4   데이터형식   649 non-null    object
 5   길이      499 non-null    object
 6   Null    649 non-null    object
dtypes: object(7)
memory usage: 35.6+ KB


In [94]:
cols.head()

,테이블명,테이블설명,컬럼명,컬럼설명,데이터형식,길이,Null
0,UNI_FC_TCMASTER,통합 정식/기본인증 마스터,ADM_NO,관리번호,VARCHAR2,10,N
1,NaN,NaN,AGREE_SUBMIS,교정정보 및 사후 확정정보 제출 협약,VARCHAR2,100,Y
2,NaN,NaN,AGREE_SUBMIS_CT,교정정보 및 사후 확정정보 제출 협약 첨부파일,VARCHAR2,100,Y
3,NaN,NaN,APP_WAR,보증에 관한 설명,VARCHAR2,100,Y
4,NaN,NaN,APP_WAR_CT,보증에 관한 설명첨부파일,VARCHAR2,100,Y


In [95]:
cols[['테이블명', '테이블설명']] = cols[['테이블명', '테이블설명']].ffill()

In [96]:
name_list = [
    'UNI_FC_TCMASTER', 
    'UNI_FC_TCERTIMODI', 
    'UNI_FC_ENG_TEFSPEC', 
    'UNI_FC_TEEMISSN', 
    'UNI_FC_TEEMISSN_ENG_RES', 
    'UNI_FC_TESTINFO', 
    'UNI_FC_ENG_LIST', 
    'UNI_FC_ENDUENG_EMI', 
    'UNI_FC_ENDUENGTEST', 
    'UNI_FC_ENDUENGTEST_RES', 
    'UNI_FC_KIENGTEST', 
    'UNI_FC_KIENGTEST_RES', 
    ]
len(name_list)

12

In [97]:
cols_dict = {}
for t in name_list:
    cols_dict[t] = {a:b for a, b in cols.loc[cols['테이블명'] == t, ['컬럼명', '컬럼설명']].values}

In [98]:
cols_dict

{'UNI_FC_TCMASTER': {'ADM_NO': '관리번호',
  'AGREE_SUBMIS': '교정정보 및 사후 확정정보 제출 협약',
  'AGREE_SUBMIS_CT': '교정정보 및 사후 확정정보 제출 협약 첨부파일',
  'APP_WAR': '보증에 관한 설명',
  'APP_WAR_CT': '보증에 관한 설명첨부파일',
  'APPLI_ID': '신청아이디',
  'APPREGI_DATE': '결재일',
  'APPROVAL_YN': '결재여부',
  'CERTI_DIV': '인증구분',
  'CERTI_INTRO': '인증소개',
  'CERTI_INTRO_CT': '인증소개 첨부파일',
  'CERTI_STATE': '처분내용',
  'CERTI_TYPE': '인증종류',
  'CO_DEVT': 'CO 개발목표치',
  'CO_OP': '산술기호(1:덧셈 2:곱셈)',
  'COLD_CO_DEVT': 'Cold CO 개발목표치',
  'COLD_CO_OP': 'Cold CO 연산자',
  'CONF_INFO': '기밀사항',
  'CONF_INFO_CT': '기밀사항 첨부파일',
  'CONF_ISSUE_NO': '발급문서번호',
  'DF_CO': 'CO 열화계수',
  'DF_COLD_CO': 'HC 열화계수',
  'DF_EVAP_EMIS': 'COLD_CO 열화계수',
  'DF_HC': 'HC 열화계수',
  'DF_HCNOX': 'HCNOX 열화계수',
  'DF_MILEAGE': '열화계수 사용량',
  'DF_NOX': 'NOX 열화계수',
  'DF_PM': 'PM 열화계수',
  'DF_PN': 'PN 열화계수',
  'DF_SMOKE': 'SMOKE 열화계수',
  'DURA_TEST_EF_SNO': '내구성시험 동일차종 일련번호',
  'DURA_TEST_REMARK': '내구성테스트 비고',
  'DURA_TEST_YN': '내구성시험 실시 여부',
  'E_CONFIRM': '결제상태',
  'EF_CD': '동일

In [99]:
name = 'UNI_FC_TCMASTER'
tcmaster = construct_machine_certification_data_load(name)
tcmaster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Columns: 159 entries, ADM_NO to NOISE_STD_NOTE
dtypes: datetime64[ns](1), float64(127), int64(2), object(29)
memory usage: 775.3+ KB


In [100]:
cols_dict[name]

{'ADM_NO': '관리번호',
 'AGREE_SUBMIS': '교정정보 및 사후 확정정보 제출 협약',
 'AGREE_SUBMIS_CT': '교정정보 및 사후 확정정보 제출 협약 첨부파일',
 'APP_WAR': '보증에 관한 설명',
 'APP_WAR_CT': '보증에 관한 설명첨부파일',
 'APPLI_ID': '신청아이디',
 'APPREGI_DATE': '결재일',
 'APPROVAL_YN': '결재여부',
 'CERTI_DIV': '인증구분',
 'CERTI_INTRO': '인증소개',
 'CERTI_INTRO_CT': '인증소개 첨부파일',
 'CERTI_STATE': '처분내용',
 'CERTI_TYPE': '인증종류',
 'CO_DEVT': 'CO 개발목표치',
 'CO_OP': '산술기호(1:덧셈 2:곱셈)',
 'COLD_CO_DEVT': 'Cold CO 개발목표치',
 'COLD_CO_OP': 'Cold CO 연산자',
 'CONF_INFO': '기밀사항',
 'CONF_INFO_CT': '기밀사항 첨부파일',
 'CONF_ISSUE_NO': '발급문서번호',
 'DF_CO': 'CO 열화계수',
 'DF_COLD_CO': 'HC 열화계수',
 'DF_EVAP_EMIS': 'COLD_CO 열화계수',
 'DF_HC': 'HC 열화계수',
 'DF_HCNOX': 'HCNOX 열화계수',
 'DF_MILEAGE': '열화계수 사용량',
 'DF_NOX': 'NOX 열화계수',
 'DF_PM': 'PM 열화계수',
 'DF_PN': 'PN 열화계수',
 'DF_SMOKE': 'SMOKE 열화계수',
 'DURA_TEST_EF_SNO': '내구성시험 동일차종 일련번호',
 'DURA_TEST_REMARK': '내구성테스트 비고',
 'DURA_TEST_YN': '내구성시험 실시 여부',
 'E_CONFIRM': '결제상태',
 'EF_CD': '동일차종기호',
 'EF_CD_1': '인증년도',
 'EF_CD_2': '차종기호',
 'EF_CD

In [101]:
cols_df = pd.DataFrame({'컬럼명':cols_dict[name].keys(), '컬럼설명':cols_dict[name].values()})
cols_df

,컬럼명,컬럼설명
0,ADM_NO,관리번호
1,AGREE_SUBMIS,교정정보 및 사후 확정정보 제출 협약
2,AGREE_SUBMIS_CT,교정정보 및 사후 확정정보 제출 협약 첨부파일
3,APP_WAR,보증에 관한 설명
4,APP_WAR_CT,보증에 관한 설명첨부파일
...,...,...
154,TONG_NUM,통관대수
155,TRECOGREP,비정기정비
156,TREGREP,정기정비
157,USER_ID,사용자아이디


In [102]:
temp = tcmaster.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
temp['총 행수'] = tcmaster.shape[0]
temp.insert(0, '번호', [x for x in range(1, temp.shape[0] + 1)])
ttemp = temp.merge(cols_df, on='컬럼명', how='left')
ttemp = ttemp[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
ttemp

,번호,컬럼명,컬럼설명,빈 값 개수,총 행수
0,1,ADM_NO,관리번호,0,624
1,2,FACT_SNO,사업장,464,624
2,3,CERTI_DIV,인증구분,0,624
3,4,REG_DATETIME,등록일시,0,624
4,5,RECT_DATE,접수일,175,624
...,...,...,...,...,...
154,155,MOD_DATETIME,수정일시,333,624
155,156,ENG_CAR_GB,건설기계/농업기계구분,131,624
156,157,EMIS_NOTE2,변경인증관련비고내용,363,624
157,158,EMIS_STD_NOTE,배출기준정보,619,624


In [103]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.12.13'

#### [출력] 컬럼, 데이터 정보들

In [104]:
# # 41.2s
# with pd.ExcelWriter(os.path.join(an2_fold, f'건설기계_인증자료_데이터정보({today_date}).xlsx')) as writer:
#     for i, name in enumerate(name_list):
#         cols_df = pd.DataFrame({'컬럼명':cols_dict[name].keys(), '컬럼설명':cols_dict[name].values()})
#         ct = construct_machine_certification_data_load(name)
#         temp = ct.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
#         temp['총 행수'] = ct.shape[0]
#         temp.insert(0, '번호', [x for x in range(1, temp.shape[0] + 1)])
#         ttemp = temp.merge(cols_df, on='컬럼명', how='left')
#         ttemp = ttemp[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
#         ttemp.to_excel(writer, sheet_name=f'{i+1}. {name}', index=False)

In [105]:
name_list

['UNI_FC_TCMASTER',
 'UNI_FC_TCERTIMODI',
 'UNI_FC_ENG_TEFSPEC',
 'UNI_FC_TEEMISSN',
 'UNI_FC_TEEMISSN_ENG_RES',
 'UNI_FC_TESTINFO',
 'UNI_FC_ENG_LIST',
 'UNI_FC_ENDUENG_EMI',
 'UNI_FC_ENDUENGTEST',
 'UNI_FC_ENDUENGTEST_RES',
 'UNI_FC_KIENGTEST',
 'UNI_FC_KIENGTEST_RES']

#### 1\. UNI_FC_TCMASTER

In [106]:
name01 = name_list[0]
name01

'UNI_FC_TCMASTER'

In [107]:
# 1.3s
tcmaster = construct_machine_certification_data_load(name01)
tcmaster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Columns: 159 entries, ADM_NO to NOISE_STD_NOTE
dtypes: datetime64[ns](1), float64(127), int64(2), object(29)
memory usage: 775.3+ KB


In [108]:
# CERTI_DIV(인증구분) : 1, 2
# EMIS_CERTI_NO(배출가스인증번호) : 00EN*AA*00 형식(숫자, 영문, 특수문자 혼합 10자리)
# EMIS_VEH_CLASS(배출가스 차종) : CE
# CERTI_STATE 처분내용 : nan, '90', 'C0', '10', '42', '46', '41'
# RECEIVE_STATE 반영 전 데이타 : nan,  0.
# NATION 제작국가 : nan, 'SW', 'US', 'KR', 'JP', 'GM', 'IT', 'FN', 'UK', 'CN', 'FR', 'SP', 'SU', 'ID'
# SELF_TEST 시설확인유무 : nan, 'Y', 'N'
# EMIS_CERTI_SNO 배출가스인증번호 : nan,  1.,  2.,  7.,  3.,  5., 12., 13.,  4.,  6., 10.,  8., 11., 9.
# OFFICE_CD 업체코드 : 'DDDC', 'VVIC', 'HIIC', 'KJDC', 'SSIC', 'DSDC', 'HDDC', 'WYIC', 'SJIC', 'ASIC', 'DPIC', 'LSIC', 'KBIC', 'CMIC', 'WTIC', 'MEIC', 'WJIC', 'HCDC', 'TMIC', 'LHIC', 'MXIC', 'HWDC', 'SCIC', 'EXAN', 'RADC', 'TYDC'
# OLDCERTI 과거자료여부 : nan, 'Y'
# REG2_YN 재신청여부 : nan, 'N'
# SNOTE 취소반납사유 : nan, '권리 및 의무 승계', '권리 및 의무 승계 신고', '권리 및 의무 신고에 따른 인증서 재교부'
# EF_CNT 동일차량 대수 : nan,  5.,  2.,  1.,  6.,  4.,  3., 22.,  9.,  0., 31.,  8., 14., 23.,  7., 15., 16., 11., 51., 13., 17., 10., 18.
# ELEC_CAR_GB 전기차 구분 ( N:일반,E:전기차(사용안함),H:하이브리드(사용안함), C:친환경차 ) : nan, 'N'
# CONF_ISSUE_NO 발급문서번호 : nan,  821.,  941.,  901., 1141., 1022., 1122.,  822.
# APPROVAL_YN 결재여부 : 'N', 'Y'
# ENG_CAR_GB 건설기계/농업기계구분 : nan, 'C', 'A', 'CA'

In [109]:
# ADM_NO 관리번호
col01 = 'ADM_NO'
print(cols_dict[name01][col01])
set(tcmaster[col01].unique())

관리번호


{52346,
 52347,
 52348,
 52349,
 52350,
 52351,
 52352,
 52353,
 52354,
 52355,
 52356,
 52359,
 52360,
 52361,
 52362,
 52366,
 52367,
 52370,
 52372,
 52376,
 52379,
 52381,
 52382,
 52385,
 52386,
 52388,
 52390,
 52391,
 52393,
 52395,
 52397,
 52398,
 52399,
 52401,
 52405,
 52406,
 52407,
 52408,
 52409,
 52410,
 52411,
 52412,
 52413,
 52414,
 52415,
 52416,
 52417,
 52418,
 52419,
 52420,
 52421,
 52422,
 52423,
 52424,
 52425,
 52426,
 52427,
 52428,
 52429,
 52431,
 52432,
 52433,
 52434,
 52435,
 52436,
 52437,
 52439,
 52452,
 52454,
 52456,
 52457,
 52458,
 52462,
 52466,
 52473,
 52474,
 52475,
 52476,
 52479,
 52480,
 52481,
 52483,
 52484,
 52485,
 52486,
 52487,
 52492,
 52493,
 52494,
 52495,
 52497,
 52505,
 52510,
 52511,
 52513,
 52514,
 52516,
 52517,
 52518,
 52520,
 52523,
 52534,
 52535,
 52536,
 52540,
 52549,
 52551,
 52552,
 52554,
 52560,
 52561,
 52562,
 52567,
 52573,
 52578,
 52583,
 52587,
 52589,
 52593,
 52597,
 52598,
 52599,
 52600,
 52601,
 52602,


In [110]:
tcmaster[col01].notnull().sum(), len(set(tcmaster[col01].unique()))

(624, 624)

#### 2\. UNI_FC_TCERTIMODI

In [111]:
name02 = name_list[1]
name02

'UNI_FC_TCERTIMODI'

In [112]:
# 5.0s
tcertimodi = construct_machine_certification_data_load(name02)
tcertimodi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830 entries, 0 to 2829
Data columns (total 91 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CERTI_MODI_SNO        2830 non-null   int64         
 1   REG_DATETIME          2830 non-null   object        
 2   CERTI_MODI_DIV        2375 non-null   float64       
 3   EMIS_NOISE_DIV        2670 non-null   float64       
 4   EMIS_MODI_ITEM        0 non-null      float64       
 5   NOISE_MODI_ITEM       0 non-null      float64       
 6   EMIS_MODI_REASON      2346 non-null   object        
 7   NOISE_MODI_REASON     0 non-null      float64       
 8   MODI_ITEM_ETC         0 non-null      float64       
 9   RECT_DATE             2828 non-null   object        
 10  APPLI_ID              2670 non-null   object        
 11  OFFICER_ID            0 non-null      float64       
 12  MODI_EXPLAIN          0 non-null      float64       
 13  EMIS_CHG_EXAM     

In [113]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 77
# CERTI_MODI_DIV 변경구분(1:변경인증, 2:변경보고) : 1.,  2., nan
# EMIS_NOISE_DIV 배출소음구분(1:배출 2:소음 3:배출+소음) : 1., nan
# MODIF_PROC_STATE 변경인증 진행상태 : nan, 0.0
# CERTI_STATE 인증진행상태 : nan, '10', '20', '30', '42', '46', '90', 'C', 'C0', 'C2'
# OLDCERTI 과거자료여부 : nan, 'N', 'Y'
# CERTI_SNO TCERTIMODI의IDX : nan, 1.0 ~ 48.0
# EMIS_DOC 배출가스인증서교부 공문 내용 : 1.0, nan
# CERTI_DIV 인증구분(1:국내,2:수입) : 1.0, 2.0, nan
# APPROVAL_YN 결재여부 : 'N', 'Y'
# MAKE_MTRL_NO 인증신청 작성자료번호 : 1.0, 3.0, 4.0, 5.0, 6.0, nan
# STANDARD_SLT 배출허용기준 : nan, 1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0, 10.0
# STANDARD_SLT2 배출허용기준2 : nan, 1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0, 10.0
# EMIS_ROWS_CODE 배출가스저감기술 : nan, '1', '1/2', '1/2/3', '1/2/3/4', '1/2/3/4/5', '1/2/4', '1/2/4/5', '1/3', '1/3/4', '1/3/4/5', '1/4', '1/4/5', '1/5', '2', '2/3/4', '2/4', '3/4', '5'
# EMIS_LOWS 배출가스저감기술코드 : nan, 'In-cylinder 방식', 'LNC', 'Turbocharger, Common rail', 'VGT, common rail', '연료장치, 터보챠저', '전자제어 연료분사 시스템', '전자제어 연료분사시스템', '전자제어장치,커먼레일', '커먼레일', '터보', '터보, 기계식', '터보차저', '터보차져'

In [114]:
# ADM_NO 관리번호
col02 = 'TC_NO'
print(cols_dict[name02][col02])
set(tcertimodi[col02].unique())

고유번호


{nan}

In [115]:
tcertimodi[col02].notnull().sum(), len(set(tcertimodi[col02].unique()))

(0, 1)

In [116]:
tcertimodi.loc[tcertimodi['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
1068,52346,3
1066,52346,1
1067,52346,2
1065,52346,0
1070,52346,5
...,...,...
2829,59491,1
2786,59492,2
2799,59492,0
2824,59492,1


In [117]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : X
tcertimodi.loc[tcertimodi[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO


#### 3\. UNI_FC_ENG_TEFSPEC

In [118]:
name03 = name_list[2]
name03

'UNI_FC_ENG_TEFSPEC'

In [119]:
# 16.1s
eng_tefspec = construct_machine_certification_data_load(name03)
eng_tefspec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8646 entries, 0 to 8645
Columns: 108 entries, ADM_NO to MAX_OUTT_POWER_UNIT
dtypes: float64(23), int64(3), object(82)
memory usage: 7.1+ MB


In [120]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 77
# EF_SNO 동일원동기일련번호 : 0 ~ 256
# CYCLE 싸이클(행정) : nan, '4', '4cycle', '4사이클', '4행정', '5'
# COMB_TYPE 연소방식 : nan, 1.0, 2.0
# COOL_TYPE 냉각매체 : nan, 1.0
# CAT_TYPE 종류 : nan, 1.0, 2.0, 8.0, 9.0, 12.0, 15.0, 19.0, 20.0, 21.0, 22.0, 24.0, 99.0
# ENG_CAR_GB 건설기계/농업기계구분 : nan, 'A', 'C', 'CA'

In [121]:
# ADM_NO 관리번호
# EF_SNO 동일원동기일련번호
col03 = 'ADM_NO'
print(cols_dict[name03][col03])
set(eng_tefspec[col03].unique())

관리번호


{52346,
 52347,
 52348,
 52349,
 52350,
 52351,
 52352,
 52353,
 52354,
 52355,
 52356,
 52359,
 52360,
 52361,
 52362,
 52366,
 52367,
 52370,
 52372,
 52376,
 52379,
 52381,
 52382,
 52385,
 52386,
 52388,
 52390,
 52391,
 52393,
 52398,
 52399,
 52401,
 52405,
 52406,
 52407,
 52408,
 52409,
 52410,
 52411,
 52412,
 52413,
 52414,
 52418,
 52419,
 52420,
 52421,
 52422,
 52423,
 52424,
 52425,
 52426,
 52429,
 52431,
 52432,
 52433,
 52437,
 52439,
 52452,
 52454,
 52456,
 52457,
 52458,
 52466,
 52474,
 52476,
 52479,
 52486,
 52492,
 52495,
 52497,
 52505,
 52510,
 52511,
 52513,
 52517,
 52520,
 52534,
 52535,
 52540,
 52549,
 52552,
 52554,
 52567,
 52573,
 52578,
 52587,
 52589,
 52602,
 52606,
 52607,
 52612,
 52613,
 52614,
 52622,
 52640,
 52643,
 52644,
 52645,
 52646,
 52647,
 52648,
 52655,
 52656,
 52657,
 52664,
 52665,
 52666,
 52672,
 52689,
 52690,
 52702,
 52704,
 52707,
 52708,
 52718,
 52719,
 52721,
 52722,
 52723,
 52727,
 52739,
 52761,
 52763,
 52764,
 52765,


In [122]:
eng_tefspec[col03].notnull().sum(), len(set(eng_tefspec[col03].unique()))

(8646, 472)

In [123]:
eng_tefspec.loc[eng_tefspec['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'EF_SNO']].sort_values('ADM_NO')

,ADM_NO,EF_SNO
5028,52346,16
5018,52346,1
5020,52346,3
5021,52346,0
5022,52346,8
...,...,...
7868,59500,0
8314,59571,0
8315,59571,1
7904,59573,1


In [124]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
eng_tefspec.loc[eng_tefspec[['ADM_NO', 'EF_SNO']].duplicated(keep=False), ['ADM_NO', 'EF_SNO']].sort_values('ADM_NO')

,ADM_NO,EF_SNO
5022,52346,8
5024,52346,8
4803,52589,8
4805,52589,8


In [125]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
eng_tefspec.loc[eng_tefspec[['ADM_NO', 'EF_SNO']].duplicated(keep=False), ['ADM_NO', 'EF_SNO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,EF_SNO,CERTI_MODI_SNO
5022,52346,8,1
5024,52346,8,2
4803,52589,8,3
4805,52589,8,4


In [126]:
# 3컬럼('ADM_NO', 'EF_SNO', 'CERTI_MODI_SNO') 중복 존재 : X
eng_tefspec.loc[eng_tefspec[['ADM_NO', 'EF_SNO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'EF_SNO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,EF_SNO,CERTI_MODI_SNO


#### 4\. UNI_FC_TEEMISSN

In [127]:
name04 = name_list[3]
name04

'UNI_FC_TEEMISSN'

In [128]:
# 0.3s
teemissn = construct_machine_certification_data_load(name04)
teemissn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058 entries, 0 to 1057
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ADM_NO           1058 non-null   int64  
 1   CERTI_MODI_SNO   1058 non-null   int64  
 2   FUEL             882 non-null    object 
 3   SEL_STAND        592 non-null    float64
 4   F_STAND          609 non-null    float64
 5   GU_TERM          577 non-null    float64
 6   TEST_DIV         0 non-null      float64
 7   TEST_METHOD_CHK  1053 non-null   object 
 8   COL              0 non-null      float64
 9   REG_ID           1058 non-null   object 
 10  REG_DATETIME     1058 non-null   object 
 11  MOD_ID           538 non-null    object 
 12  MOD_DATETIME     538 non-null    object 
 13  TBL_PK_SEQ_ID    1058 non-null   int64  
 14  LEV_CRTR_CD      0 non-null      float64
dtypes: float64(6), int64(3), object(6)
memory usage: 124.1+ KB


In [129]:
# CERTI_MODI_SNO 인증일련번호 : 0 ~ 70(빈 번호 존재)
# FUEL 사용연료코드 : nan, ‘D’
# SEL_STAND 적용기준 : nan, 3.0, 5.0, 6.0, 7.0, 8.0
# F_STAND 외국기준 : nan, 5.0, 6.0, 9.0, 10.0, 11.0

In [130]:
# ADM_NO 관리번호
col04 = 'ADM_NO'
print(cols_dict[name04][col04])
set(teemissn[col04].unique())

관리번호


{52346,
 52347,
 52348,
 52349,
 52350,
 52351,
 52352,
 52353,
 52354,
 52355,
 52356,
 52359,
 52360,
 52361,
 52362,
 52366,
 52367,
 52370,
 52372,
 52376,
 52379,
 52381,
 52382,
 52385,
 52386,
 52388,
 52390,
 52391,
 52393,
 52398,
 52399,
 52401,
 52405,
 52406,
 52407,
 52408,
 52409,
 52410,
 52411,
 52412,
 52413,
 52414,
 52418,
 52419,
 52420,
 52421,
 52422,
 52423,
 52424,
 52425,
 52426,
 52429,
 52431,
 52432,
 52433,
 52437,
 52439,
 52452,
 52454,
 52456,
 52457,
 52458,
 52466,
 52474,
 52476,
 52479,
 52486,
 52492,
 52495,
 52497,
 52505,
 52510,
 52511,
 52513,
 52517,
 52520,
 52534,
 52535,
 52540,
 52549,
 52552,
 52554,
 52567,
 52573,
 52578,
 52587,
 52589,
 52602,
 52603,
 52604,
 52605,
 52606,
 52607,
 52608,
 52612,
 52613,
 52614,
 52622,
 52640,
 52643,
 52644,
 52645,
 52646,
 52647,
 52648,
 52655,
 52656,
 52657,
 52664,
 52665,
 52666,
 52672,
 52689,
 52690,
 52702,
 52703,
 52704,
 52705,
 52706,
 52707,
 52708,
 52718,
 52719,
 52721,
 52722,


In [131]:
teemissn[col04].notnull().sum(), len(set(teemissn[col04].unique()))

(1058, 467)

In [132]:
teemissn.loc[teemissn['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
695,52510,0
1025,52510,12
708,52578,0
285,52578,3
521,52612,19
...,...,...
296,59491,2
879,59492,3
32,59492,0
1051,59492,2


In [133]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : X
teemissn.loc[teemissn[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO


#### 5\. UNI_FC_TEEMISSN_ENG_RES

In [134]:
name05 = name_list[4]
name05

'UNI_FC_TEEMISSN_ENG_RES'

In [135]:
# 6.4s
teemissn_eng_res = construct_machine_certification_data_load(name05)
teemissn_eng_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2761 entries, 0 to 2760
Columns: 101 entries, ADM_NO to NH3_ST_PERMSS
dtypes: float64(77), int64(5), object(19)
memory usage: 2.1+ MB


In [136]:
# CERTI_MODI_SNO 인증일련번호 : 0 ~ 35(빈 번호 존재)
# TEST_RST_CODE 시험결과구분코드(차량별시험결과=00,종합결과=99) : 0, 99
# TEST_MOD 시험모드(CVS-75,Highway,US06,SC03,Cold CO...) : 1, 2, 3, 99

In [137]:
# ADM_NO 관리번호
col05 = 'ADM_NO'
print(cols_dict[name05][col05])
set(teemissn_eng_res[col05].unique())

관리번호


{52346,
 52347,
 52348,
 52349,
 52350,
 52351,
 52352,
 52353,
 52354,
 52355,
 52356,
 52359,
 52360,
 52361,
 52362,
 52366,
 52367,
 52370,
 52372,
 52376,
 52379,
 52381,
 52382,
 52385,
 52386,
 52388,
 52390,
 52391,
 52393,
 52398,
 52399,
 52401,
 52405,
 52406,
 52407,
 52408,
 52409,
 52410,
 52411,
 52412,
 52413,
 52414,
 52418,
 52419,
 52420,
 52421,
 52422,
 52423,
 52424,
 52425,
 52426,
 52429,
 52432,
 52433,
 52437,
 52439,
 52452,
 52454,
 52456,
 52457,
 52458,
 52466,
 52474,
 52476,
 52479,
 52486,
 52492,
 52495,
 52497,
 52505,
 52510,
 52511,
 52513,
 52517,
 52520,
 52534,
 52535,
 52540,
 52549,
 52552,
 52554,
 52567,
 52573,
 52578,
 52587,
 52589,
 52602,
 52603,
 52604,
 52605,
 52606,
 52607,
 52608,
 52612,
 52613,
 52614,
 52622,
 52640,
 52643,
 52644,
 52645,
 52646,
 52647,
 52648,
 52655,
 52656,
 52657,
 52664,
 52665,
 52666,
 52672,
 52689,
 52690,
 52702,
 52703,
 52704,
 52705,
 52706,
 52707,
 52708,
 52718,
 52719,
 52721,
 52722,
 52723,


In [138]:
teemissn_eng_res[col05].notnull().sum(), len(set(teemissn_eng_res[col05].unique()))

(2761, 487)

#### 6\. UNI_FC_TESTINFO

In [139]:
name06 = name_list[5]
name06

'UNI_FC_TESTINFO'

In [140]:
# 1.3s
testinfo = construct_machine_certification_data_load(name06)
testinfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 67 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ADM_NO                         1044 non-null   int64  
 1   EF_SNO                         0 non-null      float64
 2   CERTI_MODI_SNO                 1044 non-null   int64  
 3   INFO1                          177 non-null    object 
 4   INFO2                          1044 non-null   object 
 5   INFO3                          0 non-null      float64
 6   TC_TYPE                        1044 non-null   int64  
 7   INFO4                          0 non-null      float64
 8   ELEC_INFO1                     0 non-null      float64
 9   ELEC_TECH                      0 non-null      float64
 10  ELEC_INFO1_TXT                 0 non-null      float64
 11  ELEC_WARR_ACCUMU               0 non-null      float64
 12  ELEC_WARR_CHARGER              0 non-null      f

In [141]:
# CERTI_MODI_SNO 인증일련번호 : 0 ~ 70(빈 번호 존재)
# TC_TYPE 적용기준(기본/변경) : 1

In [142]:
# ADM_NO 관리번호
col06 = 'ADM_NO'
print(cols_dict[name06][col06])
set(testinfo[col06].unique())

관리번호


{52346,
 52347,
 52348,
 52349,
 52350,
 52351,
 52352,
 52353,
 52354,
 52355,
 52356,
 52359,
 52360,
 52361,
 52362,
 52366,
 52367,
 52370,
 52372,
 52376,
 52379,
 52381,
 52382,
 52385,
 52386,
 52388,
 52390,
 52391,
 52393,
 52398,
 52399,
 52401,
 52405,
 52406,
 52407,
 52408,
 52409,
 52410,
 52411,
 52412,
 52413,
 52414,
 52418,
 52419,
 52420,
 52421,
 52422,
 52423,
 52424,
 52425,
 52426,
 52429,
 52432,
 52433,
 52437,
 52439,
 52452,
 52454,
 52456,
 52457,
 52458,
 52466,
 52474,
 52476,
 52479,
 52486,
 52492,
 52495,
 52497,
 52505,
 52510,
 52511,
 52513,
 52517,
 52520,
 52534,
 52535,
 52540,
 52549,
 52552,
 52554,
 52567,
 52573,
 52578,
 52587,
 52589,
 52602,
 52606,
 52607,
 52612,
 52613,
 52614,
 52622,
 52640,
 52643,
 52644,
 52645,
 52646,
 52647,
 52648,
 52655,
 52656,
 52657,
 52664,
 52665,
 52666,
 52672,
 52689,
 52690,
 52702,
 52704,
 52707,
 52708,
 52718,
 52719,
 52721,
 52722,
 52723,
 52727,
 52739,
 52761,
 52763,
 52764,
 52765,
 52768,


In [143]:
testinfo[col06].notnull().sum(), len(set(testinfo[col06].unique()))

(1044, 453)

In [144]:
testinfo.loc[testinfo['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
1028,52510,12
330,52510,0
579,52578,3
343,52578,0
92,52612,19
...,...,...
1038,59491,1
777,59492,0
961,59492,3
1040,59492,2


In [145]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : X
testinfo.loc[testinfo[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO


#### 7\. UNI_FC_ENG_LIST

In [146]:
name07 = name_list[6]
name07

'UNI_FC_ENG_LIST'

In [147]:
# 3.7s
eng_list = construct_machine_certification_data_load(name07)
eng_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9521 entries, 0 to 9520
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ENG_SNO          9521 non-null   int64  
 1   ADM_NO           9521 non-null   int64  
 2   CERTI_MODI_SNO   9521 non-null   int64  
 3   EF_SNO           9521 non-null   int64  
 4   ENG_TYPE         5 non-null      object 
 5   ENG_ID_NO        5 non-null      object 
 6   MAX_PWR_KW       5 non-null      float64
 7   MAX_PWR_RPM      5 non-null      float64
 8   ENG_DISPL        5 non-null      float64
 9   VEH_IN_COM       9281 non-null   object 
 10  VEH_PRE_NM       9279 non-null   object 
 11  VEH_NM           9287 non-null   object 
 12  VEH_MAKER        9291 non-null   object 
 13  VEH_TYPE         9290 non-null   object 
 14  ETC              58 non-null     object 
 15  REG_DATETIME     9521 non-null   object 
 16  REG_ID           2986 non-null   object 
 17  MOD_DATETIME  

In [148]:
# ENG_SNO 원동기장착일련번호 : 0 ~ 21
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 77
# EF_SNO 동일차종일련번호 : 0 ~ 256
# ENG_ID_NO 대표원동기명(원동기코드) : nan, 'CFYI520/CFYI520U', 'F5GFL414J*B', 'F5HFL464A*F',
# VEH_TYPE_YN 인증서 표출 여부 : nan, 'Y'

In [149]:
# ADM_NO 관리번호
col07 = 'EF_SNO'
print(cols_dict[name07][col07])
set(eng_list[col07].unique())

동일차종일련번호


{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [150]:
eng_list[col07].notnull().sum(), len(set(eng_list[col07].unique()))

(9521, 257)

In [151]:
eng_list.loc[eng_list['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
345,52346,3
337,52346,0
338,52346,0
339,52346,0
340,52346,0
...,...,...
8891,59500,0
8969,59571,0
8970,59571,0
8437,59573,0


In [152]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
eng_list.loc[eng_list[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
337,52346,0
338,52346,0
339,52346,0
340,52346,0
341,52346,1
...,...,...
8397,59500,0
8969,59571,0
8970,59571,0
8436,59573,0


In [153]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
eng_list.loc[eng_list[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO', 'EF_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO,EF_SNO
337,52346,0,0
338,52346,0,1
339,52346,0,2
340,52346,0,3
341,52346,1,7
...,...,...,...
8397,59500,0,2
8969,59571,0,0
8970,59571,0,1
8436,59573,0,0


In [154]:
# 3컬럼('ADM_NO', 'EF_SNO', 'EF_SNO') 중복 존재 : O
eng_list.loc[eng_list[['ADM_NO', 'CERTI_MODI_SNO', 'EF_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO', 'EF_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO,EF_SNO
8480,52510,11,23
8479,52510,11,23
8481,52510,11,23
8482,52510,11,23
8483,52510,11,23
...,...,...,...
8554,59048,1,15
9452,59491,3,4
8615,59491,3,4
8891,59500,0,2


#### 8\. UNI_FC_ENDUENG_EMI

In [155]:
name08 = name_list[7]
name08

'UNI_FC_ENDUENG_EMI'

In [156]:
# 0.2s
endueng_emi = construct_machine_certification_data_load(name08)
endueng_emi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ADM_NO           411 non-null    int64  
 1   CERTI_MODI_SNO   411 non-null    int64  
 2   TEST_METHOD_CHK  411 non-null    object 
 3   TEST_MOD         411 non-null    int64  
 4   TEST_RST_SNO     411 non-null    int64  
 5   NOX_GRAD         167 non-null    float64
 6   NMHC_GRAD        153 non-null    float64
 7   CO_GRAD          163 non-null    float64
 8   PM_GRAD          166 non-null    float64
 9   NOX_YITCT        166 non-null    float64
 10  NMHC_YITCT       153 non-null    float64
 11  CO_YITCT         165 non-null    float64
 12  PM_YITCT         165 non-null    float64
 13  REG_ID           411 non-null    object 
 14  REG_DATETIME     375 non-null    object 
 15  MOD_ID           57 non-null     object 
 16  MOD_DATETIME     21 non-null     object 
 17  PN_GRAD         

In [157]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 22(빈 번호 존재)
# TEST_METHOD_CHK 시험방법체크값 : 'NNNN', 'NNNY', 'NNYN', 'NNYY', 'NYNN', 'NYYN', 'YNNN', 'YNYN', 'YYYY'
# TEST_MOD 배출가스시험모드 : 1, 3, 4
# TEST_RST_SNO 시험결과일련번호 : 1

In [158]:
# ADM_NO 관리번호
col08 = 'ADM_NO'
print(cols_dict[name08][col08])
set(endueng_emi[col08].unique())

관리번호


{52664,
 52665,
 52727,
 52768,
 52786,
 52825,
 52852,
 52927,
 52936,
 52998,
 53000,
 53146,
 53254,
 53277,
 53299,
 53460,
 53464,
 55118,
 55119,
 55142,
 55143,
 55325,
 55329,
 55401,
 55432,
 55673,
 55685,
 55725,
 55926,
 56057,
 56083,
 56101,
 56120,
 56163,
 56191,
 56211,
 56237,
 56250,
 56362,
 56367,
 56475,
 56500,
 56539,
 56572,
 56577,
 56623,
 56800,
 56928,
 57043,
 57059,
 57218,
 57252,
 57254,
 57288,
 57294,
 57303,
 57330,
 57383,
 57415,
 57445,
 57447,
 57455,
 57530,
 57531,
 57533,
 57555,
 57599,
 57600,
 57668,
 57670,
 57743,
 57751,
 57761,
 57762,
 57764,
 57787,
 57789,
 57794,
 57826,
 57836,
 57843,
 57859,
 57876,
 57903,
 57923,
 57929,
 57930,
 57937,
 57956,
 57966,
 57985,
 57988,
 57994,
 58002,
 58006,
 58040,
 58051,
 58058,
 58091,
 58104,
 58110,
 58117,
 58119,
 58126,
 58129,
 58136,
 58137,
 58142,
 58144,
 58145,
 58148,
 58175,
 58212,
 58214,
 58223,
 58237,
 58378,
 58419,
 58420,
 58443,
 58505,
 58523,
 58524,
 58568,
 58576,


In [159]:
endueng_emi[col08].notnull().sum(), len(set(endueng_emi[col08].unique()))

(411, 156)

In [160]:
endueng_emi.loc[endueng_emi['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
222,52664,22
223,52664,22
275,52665,16
276,52665,16
16,52727,12
...,...,...
403,59491,0
407,59492,0
408,59492,0
332,59500,0


In [161]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
endueng_emi.loc[endueng_emi[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
222,52664,22
223,52664,22
275,52665,16
276,52665,16
16,52727,12
...,...,...
403,59491,0
407,59492,0
408,59492,0
332,59500,0


#### 9\. UNI_FC_ENDUENGTEST

In [162]:
name09 = name_list[8]
name09

'UNI_FC_ENDUENGTEST'

In [163]:
# 0.1s
enduengtest = construct_machine_certification_data_load(name09)
enduengtest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ADM_NO          430 non-null    int64 
 1   CERTI_MODI_SNO  430 non-null    int64 
 2   TEST_ENG_SNO    430 non-null    int64 
 3   ENG_TYPE        334 non-null    object
 4   ENG_ID_NO       315 non-null    object
 5   ENG_NO          319 non-null    object
 6   TEST_NO         277 non-null    object
 7   TEST_DATE       318 non-null    object
 8   REG_ID          407 non-null    object
 9   REG_DATETIME    430 non-null    object
 10  MOD_ID          56 non-null     object
 11  MOD_DATETIME    51 non-null     object
dtypes: int64(3), object(9)
memory usage: 40.4+ KB


In [164]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 22(빈 번호 존재)
# TEST_ENG_SNO 시험원동기일련번호 : 0 ~ 19

In [165]:
# ADM_NO 관리번호
col09 = 'ADM_NO'
print(cols_dict[name09][col09])
set(enduengtest[col09].unique())

관리번호


{52664,
 52665,
 52727,
 52768,
 52786,
 52825,
 52852,
 52927,
 52936,
 52998,
 53000,
 53146,
 53254,
 53277,
 53299,
 53460,
 53464,
 55118,
 55119,
 55142,
 55143,
 55325,
 55329,
 55401,
 55432,
 55673,
 55685,
 55725,
 55926,
 56057,
 56083,
 56101,
 56120,
 56163,
 56191,
 56211,
 56237,
 56250,
 56362,
 56367,
 56475,
 56500,
 56539,
 56572,
 56577,
 56623,
 56800,
 56928,
 57043,
 57059,
 57218,
 57252,
 57254,
 57288,
 57294,
 57303,
 57330,
 57383,
 57415,
 57445,
 57447,
 57455,
 57530,
 57531,
 57533,
 57555,
 57599,
 57600,
 57668,
 57670,
 57743,
 57751,
 57761,
 57762,
 57764,
 57787,
 57789,
 57794,
 57826,
 57836,
 57843,
 57859,
 57876,
 57903,
 57923,
 57929,
 57930,
 57937,
 57956,
 57966,
 57985,
 57988,
 57994,
 58002,
 58006,
 58040,
 58051,
 58058,
 58091,
 58104,
 58110,
 58117,
 58119,
 58126,
 58129,
 58136,
 58137,
 58142,
 58144,
 58145,
 58148,
 58175,
 58212,
 58214,
 58223,
 58237,
 58378,
 58419,
 58420,
 58443,
 58505,
 58523,
 58524,
 58568,
 58576,


In [166]:
enduengtest[col09].notnull().sum(), len(set(enduengtest[col09].unique()))

(430, 155)

In [167]:
enduengtest.loc[enduengtest['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
183,52664,22
339,52664,20
7,52825,8
330,52825,7
79,53146,12
...,...,...
371,59018,0
415,59018,0
416,59018,0
417,59018,0


In [168]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
enduengtest.loc[enduengtest[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
249,55142,0
247,55142,0
246,55142,0
245,55142,0
244,55142,0
...,...,...
409,59018,0
415,59018,0
416,59018,0
371,59018,0


#### 10\. UNI_FC_ENDUENGTEST_RES

In [169]:
name10 = name_list[9]
name10

'UNI_FC_ENDUENGTEST_RES'

In [170]:
# 4.2s
enduengtest_res = construct_machine_certification_data_load(name10)
enduengtest_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7581 entries, 0 to 7580
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADM_NO            7581 non-null   int64  
 1   CERTI_MODI_SNO    7581 non-null   int64  
 2   P_ENG_YN          7581 non-null   int64  
 3   SEL_ADM_NO        4236 non-null   float64
 4   TEST_METHOD_CHK   7581 non-null   object 
 5   TEST_MOD          7581 non-null   int64  
 6   TEST_RST_SNO      7581 non-null   int64  
 7   TEST_RST_SNO_SEQ  7581 non-null   int64  
 8   FUEL              0 non-null      float64
 9   DDC_USE_YN        6724 non-null   float64
 10  TEST_CNT          0 non-null      float64
 11  TEST_TIME         7100 non-null   float64
 12  TEST_PLACE        7054 non-null   object 
 13  TEST_DATE         7025 non-null   object 
 14  CO                6034 non-null   float64
 15  NOX               5960 non-null   float64
 16  PM                5901 non-null   float64


In [171]:
# CERTI_MODI_SNO 인증 일련번호 : 0 ~ 22(빈 번호 존재)
# P_ENG_YN 내구대표차원동기여부 : 0, 1, 2
# TEST_METHOD_CHK 시험방법체크값 : 'NNNN', 'NNNY', 'NNYN', 'NNYY', 'NYNN', 'NYYN', 'YNNN', 'YNYN', 'YYYY'
# TEST_MOD 시험모드(CVS-75,Highway,US06,SC03,Cold CO...) : 1, 2
# TEST_RST_SNO 시험결과일련번호 : 1 ~ 268
# TEST_RST_SNO_SEQ 시험결과일련번호의항목순번 : 1 ~ 4
# DDC_USE_YN 지정열화계수 사용유무(사용:1,미사용:0) : nan, 1.0, 2.0

In [172]:
# ADM_NO 관리번호
col10 = 'ADM_NO'
print(cols_dict[name10][col10])
set(enduengtest_res[col10].unique())

관리번호


{52664,
 52665,
 52727,
 52768,
 52786,
 52825,
 52852,
 52927,
 52936,
 52998,
 53000,
 53146,
 53254,
 53277,
 53299,
 53460,
 53464,
 55118,
 55119,
 55142,
 55143,
 55325,
 55329,
 55401,
 55432,
 55673,
 55685,
 55725,
 55926,
 56057,
 56083,
 56101,
 56120,
 56163,
 56191,
 56211,
 56237,
 56250,
 56362,
 56367,
 56475,
 56500,
 56539,
 56572,
 56577,
 56623,
 56800,
 56928,
 57043,
 57059,
 57218,
 57252,
 57254,
 57288,
 57294,
 57303,
 57330,
 57383,
 57415,
 57445,
 57447,
 57455,
 57530,
 57531,
 57533,
 57555,
 57599,
 57600,
 57668,
 57670,
 57743,
 57751,
 57761,
 57762,
 57764,
 57787,
 57789,
 57794,
 57826,
 57836,
 57843,
 57859,
 57876,
 57903,
 57923,
 57929,
 57930,
 57937,
 57956,
 57966,
 57985,
 57988,
 57994,
 58002,
 58006,
 58040,
 58051,
 58058,
 58091,
 58104,
 58110,
 58117,
 58119,
 58126,
 58129,
 58136,
 58137,
 58142,
 58144,
 58145,
 58148,
 58175,
 58212,
 58214,
 58223,
 58237,
 58378,
 58419,
 58420,
 58443,
 58505,
 58523,
 58524,
 58568,
 58576,


In [173]:
enduengtest_res[col10].notnull().sum(), len(set(enduengtest_res[col10].unique()))

(7581, 155)

In [174]:
enduengtest_res.loc[enduengtest_res['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
1521,52664,22
1523,52664,22
1522,52664,22
1520,52664,22
1519,52664,22
...,...,...
6719,59500,0
6461,59573,0
6462,59573,0
6463,59573,0


In [175]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
enduengtest_res.loc[enduengtest_res[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
1521,52664,22
1523,52664,22
1522,52664,22
1520,52664,22
1519,52664,22
...,...,...
6719,59500,0
6461,59573,0
6462,59573,0
6463,59573,0


#### 11\. UNI_FC_KIENGTEST

In [176]:
name11 = name_list[10]
name11

'UNI_FC_KIENGTEST'

In [177]:
# 0.1s
kiengtest = construct_machine_certification_data_load(name11)
kiengtest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307 entries, 0 to 306
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ADM_NO          307 non-null    int64 
 1   CERTI_MODI_SNO  307 non-null    int64 
 2   TEST_ENG_SNO    307 non-null    int64 
 3   ENG_TYPE        209 non-null    object
 4   ENG_ID_NO       196 non-null    object
 5   ENG_NO          209 non-null    object
 6   TEST_NO         179 non-null    object
 7   TEST_DATE       203 non-null    object
 8   REG_ID          307 non-null    object
 9   REG_DATETIME    307 non-null    object
 10  MOD_ID          29 non-null     object
 11  MOD_DATETIME    29 non-null     object
dtypes: int64(3), object(9)
memory usage: 28.9+ KB


In [178]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 35(빈 번호 존재)
# TEST_ENG_SNO 시험원동기일련번호 : 0 ~ 5

In [179]:
# ADM_NO 관리번호
col11 = 'ADM_NO'
print(cols_dict[name11][col11])
set(kiengtest[col11].unique())

관리번호


{52664,
 52665,
 52727,
 52768,
 52786,
 52825,
 52852,
 52925,
 52927,
 52998,
 53277,
 53299,
 53460,
 53464,
 53813,
 54543,
 54730,
 54913,
 55118,
 55119,
 55142,
 55143,
 55325,
 55432,
 55471,
 55673,
 55685,
 55725,
 55926,
 56057,
 56083,
 56101,
 56120,
 56163,
 56191,
 56211,
 56237,
 56250,
 56362,
 56367,
 56475,
 56539,
 56572,
 56577,
 56623,
 56800,
 56928,
 57059,
 57252,
 57254,
 57288,
 57294,
 57303,
 57330,
 57383,
 57415,
 57445,
 57447,
 57455,
 57491,
 57531,
 57535,
 57573,
 57579,
 57599,
 57600,
 57668,
 57670,
 57743,
 57751,
 57761,
 57762,
 57764,
 57787,
 57794,
 57808,
 57826,
 57837,
 57843,
 57869,
 57870,
 57876,
 57879,
 57929,
 57930,
 57937,
 57956,
 57966,
 57985,
 57988,
 57994,
 58002,
 58006,
 58040,
 58051,
 58058,
 58104,
 58110,
 58117,
 58119,
 58126,
 58129,
 58136,
 58137,
 58142,
 58144,
 58145,
 58148,
 58175,
 58212,
 58214,
 58223,
 58237,
 58309,
 58311,
 58356,
 58357,
 58365,
 58378,
 58420,
 58443,
 58523,
 58568,
 58576,
 58602,


In [180]:
kiengtest[col11].notnull().sum(), len(set(kiengtest[col11].unique()))

(307, 152)

In [181]:
kiengtest.loc[kiengtest['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
137,52664,22
237,52664,20
88,53299,9
238,53299,8
125,53464,6
...,...,...
231,59490,1
304,59490,0
83,59573,0
82,59573,0


In [182]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
kiengtest.loc[kiengtest[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
151,55725,0
150,55725,0
149,55725,0
143,55725,0
3,56101,0
...,...,...
298,59033,0
300,59033,0
83,59573,0
82,59573,0


#### 12\. UNI_FC_KIENGTEST_RES

In [183]:
name12 = name_list[11]
name12

'UNI_FC_KIENGTEST_RES'

In [184]:
# 1.3s
kiengtest_res = construct_machine_certification_data_load(name12)
kiengtest_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2740 entries, 0 to 2739
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADM_NO            2740 non-null   int64  
 1   CERTI_MODI_SNO    2740 non-null   int64  
 2   P_ENG_YN          2704 non-null   float64
 3   SEL_ADM_NO        30 non-null     float64
 4   TEST_METHOD_CHK   2740 non-null   object 
 5   TEST_MOD          2740 non-null   int64  
 6   TEST_RST_SNO      2740 non-null   int64  
 7   TEST_RST_SNO_SEQ  2740 non-null   int64  
 8   FUEL              0 non-null      float64
 9   CO                953 non-null    float64
 10  NMHC              924 non-null    float64
 11  NOX               943 non-null    float64
 12  PM                943 non-null    float64
 13  REG_ID            2740 non-null   object 
 14  REG_DATETIME      2740 non-null   object 
 15  MOD_ID            162 non-null    object 
 16  MOD_DATETIME      162 non-null    object 


In [185]:
# CERTI_MODI_SNO 인증 일련번호 : 0 ~ 35(빈 번호 존재)
# P_ENG_YN KI대표원동기 여부 : nan, 1.0, 2.0
# TEST_METHOD_CHK 시험방법체크값 : 'NNNN', 'NNYN', 'NYNN'
# TEST_MOD 시험모드(CVS-75,Highway,US06,SC03,Cold CO...) : 1, 2
# TEST_RST_SNO 시험결과일련번호 : 0 ~ 9
# TEST_RST_SNO_SEQ 시험결과일련번호의항목순번 : 0 ~ 2

In [186]:
# ADM_NO 관리번호
col12 = 'ADM_NO'
print(cols_dict[name12][col12])
set(kiengtest_res[col12].unique())

관리번호


{52664,
 52665,
 52727,
 52768,
 52786,
 52825,
 52852,
 52925,
 52927,
 52998,
 53277,
 53299,
 53460,
 53464,
 53813,
 54543,
 54730,
 54913,
 55118,
 55119,
 55142,
 55143,
 55325,
 55432,
 55471,
 55673,
 55685,
 55725,
 55926,
 56057,
 56083,
 56101,
 56120,
 56163,
 56191,
 56211,
 56237,
 56250,
 56362,
 56367,
 56475,
 56539,
 56572,
 56577,
 56623,
 56800,
 56928,
 57059,
 57252,
 57254,
 57288,
 57294,
 57303,
 57330,
 57383,
 57415,
 57445,
 57447,
 57455,
 57491,
 57531,
 57535,
 57573,
 57579,
 57599,
 57600,
 57668,
 57670,
 57743,
 57751,
 57761,
 57762,
 57764,
 57787,
 57794,
 57808,
 57826,
 57837,
 57843,
 57869,
 57870,
 57876,
 57879,
 57929,
 57930,
 57937,
 57956,
 57966,
 57985,
 57988,
 57994,
 58002,
 58006,
 58040,
 58051,
 58058,
 58104,
 58110,
 58117,
 58119,
 58126,
 58129,
 58136,
 58137,
 58142,
 58144,
 58145,
 58148,
 58175,
 58212,
 58214,
 58223,
 58237,
 58309,
 58311,
 58356,
 58357,
 58365,
 58378,
 58420,
 58443,
 58523,
 58568,
 58576,
 58602,


In [187]:
kiengtest_res[col12].notnull().sum(), len(set(kiengtest_res[col12].unique()))

(2740, 152)

In [188]:
kiengtest_res.loc[kiengtest_res['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
1267,52664,22
1262,52664,22
1263,52664,22
1264,52664,22
1265,52664,22
...,...,...
2526,59573,0
2527,59573,0
2518,59573,0
2525,59573,0


In [189]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
kiengtest_res.loc[kiengtest_res[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
1267,52664,22
1262,52664,22
1263,52664,22
1264,52664,22
1265,52664,22
...,...,...
2526,59573,0
2527,59573,0
2518,59573,0
2525,59573,0


### 건설기계 인증대장

In [6]:
name = '건설기계 인증대장(Tier3~Stage5)'
file_name = f'{name}.xlsx'
cert_ld34 = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name='TIER-3~TIER-4')
cert_ld34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5974 entries, 0 to 5973
Data columns (total 64 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   배출기준             5971 non-null   object
 1   상호명              5971 non-null   object
 2   인증번호             5971 non-null   object
 3   최초인증일자           5971 non-null   object
 4   재교부일자
(변경일자)     5971 non-null   object
 5   접수번호             5971 non-null   object
 6   접수일자             5971 non-null   object
 7   종류               5971 non-null   object
 8   구분               5971 non-null   object
 9   상세내역             4051 non-null   object
 10  엔진형식
(엔진모델명)     5971 non-null   object
 11  엔진코드             5970 non-null   object
 12  엔진배기량            5862 non-null   object
 13  최대출력             5856 non-null   object
 14  Unnamed: 14      5856 non-null   object
 15  엔진제작사            5971 non-null   object
 16  엔진제작국            5971 non-null   object
 17  건설기계 
수입사
(대표자)  5367 non-null   

In [7]:
name = '건설기계 인증대장(Tier3~Stage5)'
file_name = f'{name}.xlsx'
cert_ld5 = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name='STAGE5')
cert_ld5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2058 entries, 0 to 2057
Data columns (total 74 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   배출기준             2054 non-null   object 
 1   상호명              2055 non-null   object 
 2   인증번호             2055 non-null   object 
 3   최초인증일자           2054 non-null   object 
 4   재교부일자
(변경일자)     2054 non-null   object 
 5   접수번호             2054 non-null   object 
 6   접수일자             2054 non-null   object 
 7   종류               2054 non-null   object 
 8   구분               2054 non-null   object 
 9   상세내역             783 non-null    object 
 10  엔진형식
(엔진모델명)     2054 non-null   object 
 11  엔진코드             2053 non-null   object 
 12  엔진배기량            2054 non-null   float64
 13  최대출력             2050 non-null   object 
 14  Unnamed: 14      2050 non-null   object 
 15  엔진
제작사           2054 non-null   object 
 16  엔진제작국            2054 non-null   object 
 17  건설기계 
수입사
(대표자

#### 정보(TIER-3~TIER-4)

##### 컬럼 수정

In [8]:
cert_ld34.columns

Index(['배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자\n(변경일자)', '접수번호', '접수일자', '종류',
       '구분', '상세내역', '엔진형식\n(엔진모델명)', '엔진코드', '엔진배기량', '최대출력', 'Unnamed: 14',
       '엔진제작사', '엔진제작국', '건설기계 \n수입사\n(대표자)', '건설기계\n제작사', '건설기계\n명칭',
       '건설/농업기계 형식', '비고', '정정내역', '기준치', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', '시험결과', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
       'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54',
       'Unnamed: 55', '자체시험여부', '적용기술', 'Unnamed: 58', 'Unnamed: 59',
       'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'DF 변경보고'],
      dtype='object')

In [9]:
cert_ld34.columns = [
    '배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자(변경일자)', '접수번호', '접수일자', '종류', '구분', '상세내역', '엔진형식(엔진모델명)', '엔진코드', '엔진배기량', '최대출력_KW', '최대출력_RPM', '엔진제작사', '엔진제작국', '건설기계수입사(대표자)', '건설기계제작사', '건설기계명칭', '건설/농업기계_형식', '비고', '정정내역', 
    '기준치_CO', '기준치_NMHC+Nox', '기준치_NMHC', '기준치_Nox', '기준치_PM', '기준치_CO2', 
    '시험결과_ISO_8178_C1_CO', '시험결과_ISO_8178_C1_HC+Nox', '시험결과_ISO_8178_C1_NMHC', '시험결과_ISO_8178_C1_Nox', '시험결과_ISO_8178_C1_PM', '시험결과_ISO_8178_C1_CO2', '시험결과_ISO_8178_C1_검사일시', '시험결과_ISO_8178_C1_최대출력_kw', '시험결과_ISO_8178_C1_rpm', 
    '시험결과_NRSC_CO', '시험결과_NRSC_HC+Nox', '시험결과_NRSC_NMHC', '시험결과_NRSC_Nox', '시험결과_NRSC_PM', '시험결과_NRSC_CO2', '시험결과_NRSC_검사일시', '시험결과_NRSC_최대출력_kw', '시험결과_NRSC_최대출력_rpm', 
    '시험결과_NRTC_CO', '시험결과_NRTC_HC+Nox', '시험결과_NRTC_NMHC', '시험결과_NRTC_Nox', '시험결과_NRTC_PM', '시험결과_NRTC_CO2', '시험결과_NRTC_검사일시', '시험결과_NRTC_최대출력_kw', '시험결과_NRTC_최대출력_rpm',
    '자체시험여부', '적용기술_EGR', '적용기술_SCR', '적용기술_DPF', '적용기술_ECU', '적용기술_DOC', '적용기술_기타', 'DF_변경보고'
]
len(cert_ld34.columns)

64

##### 필요한 데이터만 추출(행 제한)

In [10]:
cert_ld34 = cert_ld34.iloc[3:, :]
cert_ld34.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,...,시험결과_NRTC_최대출력_kw,시험결과_NRTC_최대출력_rpm,자체시험여부,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,DF_변경보고
3,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,2009-01-14 00:00:00,-,6940,2008-12-23 00:00:00,건설기계,대표(최초),NaN,...,-,-,"TIER-2기준으로시험,3도만족\n교통입회",X,X,X,O,NaN,NaN,해당없음
4,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2012-02-08 00:00:00,15,2012-02-06 00:00:00,건설기계,대표(정정),대표엔진의\n건설기계 형식 정정,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
5,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2011-08-03 00:00:00,173,2011-07-26 00:00:00,건설기계,대표(보고),대표엔진의\n수입사 및 건설기계추가,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
6,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2011-09-22 00:00:00,204,2011-09-07 00:00:00,건설기계,대표(보고),대표엔진의\n수입사 및 건설기계추가,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
7,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2012-01-25 00:00:00,3,2012-01-10 00:00:00,건설기계,대표(보고),대표엔진의\n수입사 및 건설기계추가,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음


In [11]:
cert_ld34.shape

(5971, 64)

##### 데이터 확인

In [12]:
set(cert_ld34['접수번호'].unique())

{1,
 3,
 5,
 7,
 8,
 '2014_740',
 10,
 9,
 12,
 '2020_1888',
 15,
 18,
 19,
 20,
 21,
 22,
 '2021_1308',
 23,
 24,
 '215\n215~1',
 27,
 '2015_1341',
 29,
 '2016_217',
 28,
 31,
 33,
 34,
 35,
 36,
 '2021_1060',
 '2020_2176',
 41,
 42,
 '2019_836',
 44,
 '2017_504',
 45,
 '2019_984',
 47,
 '2017_1167',
 49,
 51,
 52,
 57,
 58,
 59,
 '2020_2451',
 '2019_491',
 64,
 '2018_810',
 67,
 68,
 69,
 70,
 72,
 73,
 74,
 '2020_320',
 76,
 80,
 81,
 '2017_1020',
 83,
 84,
 85,
 86,
 87,
 '2019_864',
 89,
 88,
 91,
 92,
 '2020_939',
 '95\n95~1',
 94,
 96,
 97,
 95,
 99,
 100,
 '2018_1167',
 102,
 103,
 101,
 105,
 106,
 104,
 '2016_14',
 107,
 110,
 109,
 112,
 113,
 114,
 '2015_1511',
 116,
 117,
 '2020_1271',
 119,
 120,
 121,
 '2018_258',
 123,
 124,
 125,
 '2019_1300',
 127,
 128,
 129,
 '2019_1773',
 '2014_556',
 132,
 '2021_2008',
 126,
 135,
 136,
 '2021_287',
 138,
 139,
 140,
 141,
 142,
 '2020_2231',
 143,
 146,
 '2014_941\n2014_941~1',
 149,
 151,
 '2018_1369',
 154,
 153,
 156,
 157,
 1

##### 데이터 수정(TIER-3~TIER-4)

In [13]:
# 줄 바꿈이 있는 열
# 상세내역, 최대출력_KW, 최대출력_RPM, 정정내역, 자체시험여부

# 줄 바꿈을 간격으로 바꿀 열
# 상세내역

# 줄 바꿈을 없앨 열
# 최대출력_KW, 최대출력_RPM, 정정내역

# 줄 바꿈을 쉼표로 바꿀 열
# 자체시험여부

In [14]:
# 줄 바꿈을 간격으로 바꿀 열
# 상세내역
cert_ld34['상세내역'] = cert_ld34['상세내역'].str.replace('\n', ' ')
cert_ld34[['상세내역']].head()

,상세내역
3,NaN
4,대표엔진의 건설기계 형식 정정
5,대표엔진의 수입사 및 건설기계추가
6,대표엔진의 수입사 및 건설기계추가
7,대표엔진의 수입사 및 건설기계추가


In [15]:
# 줄 바꿈을 없앨 열
# 최대출력_KW, 최대출력_RPM, 정정내역
cert_ld34['최대출력_KW'] = cert_ld34['최대출력_KW'].str.replace('\n', '')
cert_ld34['최대출력_RPM'] = cert_ld34['최대출력_RPM'].str.replace('\n', '')
cert_ld34['정정내역'] = cert_ld34['정정내역'].str.replace('\n', '')
cert_ld34[['최대출력_KW', '최대출력_RPM', '정정내역']].head()

,최대출력_KW,최대출력_RPM,정정내역
3,250(325ps),"2,100(1,900)","1. 동일엔진 추가, 출력병기 (2010.02.02, 재교부)"
4,250(325ps),"2,100(1,900)","1.건설기계 형식 정정(2012.02.,재교부)"
5,250(325ps),"2,100(1,900)",NaN
6,250(325ps),"2,100(1,900)",NaN
7,250(325ps),"2,100(1,900)",NaN


In [16]:
# 줄 바꿈을 쉼표로 바꿀 열
# 자체시험여부
cert_ld34['자체시험여부'] = cert_ld34['자체시험여부'].str.replace('\n', ',')
cert_ld34[['자체시험여부']].head()

,자체시험여부
3,"TIER-2기준으로시험,3도만족,교통입회"
4,NaN
5,NaN
6,NaN
7,NaN


#### 정보(STAGE5)

##### 컬럼 수정

In [17]:
cert_ld5.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자\n(변경일자),접수번호,접수일자,종류,구분,상세내역,...,Unnamed: 64,자체시험여부,DF 대표,적용기술,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,임의설정자료제출여부
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,EGR,SCR,DPF,ECU,DOC,기타,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,rpm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,...,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
cert_ld5.columns

Index(['배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자\n(변경일자)', '접수번호', '접수일자', '종류',
       '구분', '상세내역', '엔진형식\n(엔진모델명)', '엔진코드', '엔진배기량', '최대출력', 'Unnamed: 14',
       '엔진\n제작사', '엔진제작국', '건설기계 \n수입사\n(대표자)', '건설기계\n제작사', '건설기계\n명칭',
       '건설/농업기계 형식', '비고', '기준치', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', '시험결과', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
       'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54',
       'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58',
       'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62',
       'Unnamed: 63', 'Unnamed: 64', '자체시험여부', 'DF 대표', '적용기술', 'Unnamed: 68',
 

In [19]:
cert_ld5.columns = [
    '배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자(변경일자)', '접수번호', '접수일자', '종류', '구분', '상세내역', '엔진형식(엔진모델명)', '엔진코드', '엔진배기량', 
    '최대출력_KW', '최대출력_RPM',
    '엔진제작사', '엔진제작국', '건설기계수입사(대표자)', '건설기계제작사', '건설기계명칭', '건설/농업기계_형식', '비고', 
    '기준치_CO', '기준치_THC+Nox', '기준치_THC', '기준치_Nox', '기준치_PM', '기준치_PN', '기준치_NH3', 
    '시험결과_ISO_8178_C1_CO', '시험결과_ISO_8178_C1_THC+Nox', '시험결과_ISO_8178_C1_THC', '시험결과_ISO_8178_C1_Nox', '시험결과_ISO_8178_C1_PM', '시험결과_ISO_8178_C1_PN', '시험결과_ISO_8178_C1_NH3', '시험결과_ISO_8178_C1_CO2', '시험결과_ISO_8178_C1_연비', '시험결과_ISO_8178_C1_검사일시',
    '시험결과_ISO_8178_C1_최대출력_kw', '시험결과_ISO_8178_C1_최대출력_rpm', 
    '시험결과_NRSC_CO', '시험결과_NRSC_THC+Nox', '시험결과_NRSC_THC', '시험결과_NRSC_Nox', '시험결과_NRSC_PM', '시험결과_NRSC_PN', '시험결과_NRSC_NH3', '시험결과_NRSC_CO2', '시험결과_NRSC_연비', '시험결과_NRSC_검사일시', '시험결과_NRSC_최대출력_kw', '시험결과_NRSC_최대출력_rpm', 
    '시험결과_NRTC_CO', '시험결과_NRTC_THC+Nox', '시험결과_NRTC_THC', '시험결과_NRTC_Nox', '시험결과_NRTC_PM', '시험결과_NRTC_PN', '시험결과_NRTC_NH3', '시험결과_NRTC_CO2', '시험결과_NRTC_연비', '시험결과_NRTC_검사일시', '시험결과_NRTC_최대출력_kw', '시험결과_NRTC_최대출력_rpm', 
    '자체시험여부', 'DF_대표', '적용기술_EGR', '적용기술_SCR', '적용기술_DPF', '적용기술_ECU', '적용기술_DOC', '적용기술_기타', 
    '임의설정자료제출여부'
]

In [20]:
cert_ld5.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,...,시험결과_NRTC_최대출력_rpm,자체시험여부,DF_대표,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,임의설정자료제출여부
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,EGR,SCR,DPF,ECU,DOC,기타,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,rpm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,...,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 필요한 데이터만 추출(행 제한)

In [21]:
cert_ld5 = cert_ld5.iloc[3:, :]
cert_ld5.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,...,시험결과_NRTC_최대출력_rpm,자체시험여부,DF_대표,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,임의설정자료제출여부
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,...,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
cert_ld5.shape

(2055, 74)

In [23]:
cert_ld5.iloc[:5, :20]

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,엔진형식(엔진모델명),엔진코드,엔진배기량,최대출력_KW,최대출력_RPM,엔진제작사,엔진제작국,건설기계수입사(대표자),건설기계제작사,건설기계명칭
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,F4HGE613I,F4HGE613I*V,6728.00,191,2000,FPT Industrial S.p.A,이태리,해당없음,해당없음,해당없음
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617D*V,6728.00,221,2100,FPT Industrial S.p.A,이태리,LS엠트론\n(구자은),NH,트랙터
5,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617D*V,6728.00,221,2100,FPT Industrial S.p.A,이태리,삼정건설기계\n(유재흥),CASE,트랙터
6,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617E*V,6728.00,199,2100,FPT Industrial S.p.A,이태리,LS엠트론\n(구자은),NH,트랙터
7,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617E*V,6728.00,199,2100,FPT Industrial S.p.A,이태리,삼정건설기계\n(유재흥),CASE,트랙터


In [24]:
cert_ld5.iloc[:5, 20:35]

,건설/농업기계_형식,비고,기준치_CO,기준치_THC+Nox,기준치_THC,기준치_Nox,기준치_PM,기준치_PN,기준치_NH3,시험결과_ISO_8178_C1_CO,시험결과_ISO_8178_C1_THC+Nox,시험결과_ISO_8178_C1_THC,시험결과_ISO_8178_C1_Nox,시험결과_ISO_8178_C1_PM,시험결과_ISO_8178_C1_PN
3,해당없음,NaN,3.50,-,0.19,0.40,0.01,1000000000000,10,-,-,-,-,-,-
4,T7.315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Optum 300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,T7.290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Optum 270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
cert_ld5.iloc[:5, 35:50]

,시험결과_ISO_8178_C1_NH3,시험결과_ISO_8178_C1_CO2,시험결과_ISO_8178_C1_연비,시험결과_ISO_8178_C1_검사일시,시험결과_ISO_8178_C1_최대출력_kw,시험결과_ISO_8178_C1_최대출력_rpm,시험결과_NRSC_CO,시험결과_NRSC_THC+Nox,시험결과_NRSC_THC,시험결과_NRSC_Nox,시험결과_NRSC_PM,시험결과_NRSC_PN,시험결과_NRSC_NH3,시험결과_NRSC_CO2,시험결과_NRSC_연비
3,-,-,-,-,-,-,0.01,-,0.01,0.29,0.00,34600000000,1.52,686.36,212.78
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
cert_ld5.iloc[:5, 50:60]

,시험결과_NRSC_검사일시,시험결과_NRSC_최대출력_kw,시험결과_NRSC_최대출력_rpm,시험결과_NRTC_CO,시험결과_NRTC_THC+Nox,시험결과_NRTC_THC,시험결과_NRTC_Nox,시험결과_NRTC_PM,시험결과_NRTC_PN,시험결과_NRTC_NH3
3,2019-11-14 00:00:00,191,2000,0.02,-,0.01,0.21,0.00,25100000000,0.76
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
cert_ld5.iloc[:5, 60:]

,시험결과_NRTC_CO2,시험결과_NRTC_연비,시험결과_NRTC_검사일시,시험결과_NRTC_최대출력_kw,시험결과_NRTC_최대출력_rpm,자체시험여부,DF_대표,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,임의설정자료제출여부
3,706.07,219.15,2019-11-13 00:00:00,191,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 데이터 확인

In [28]:
set(cert_ld5['접수번호'].unique())

{'2018_1235\n2018_1235~1',
 '2018_1236\n2018_1236~1',
 '2019_1347',
 '2019_1348',
 '2019_223\n2019_223~1',
 '2019_224\n2019_224~1',
 '2019_771',
 '2019_772',
 '2020_1230',
 '2020_1417',
 '2020_1721',
 '2020_1956',
 '2020_2105',
 '2020_2200',
 '2020_2210',
 '2020_2211',
 '2020_2221\n2020_2221~1',
 '2020_2372',
 '2020_2375',
 '2020_2417\n2020_2417~1',
 '2020_2657',
 '2021_1033\n2021_1033~1',
 '2021_1034',
 '2021_1143\n2021_1143~1',
 '2021_1144',
 '2021_117',
 '2021_1194\n2021_1194~1',
 '2021_1257',
 '2021_1266',
 '2021_1277',
 '2021_1336\n2021_1336~1',
 '2021_1510',
 '2021_1511\n2021_1511~1',
 '2021_1512\n2021_1512~1',
 '2021_1609',
 '2021_1735',
 '2021_1787',
 '2021_1788',
 '2021_1795\n2021_1795~1',
 '2021_1796',
 '2021_1797\n2021_1797~1',
 '2021_1822\n2021_1822~1',
 '2021_1858',
 '2021_1866',
 '2021_1867\n2021_1867~1',
 '2021_1903\n2021_1901~1',
 '2021_1918\n2021_1918~1',
 '2021_1921\n2021_1921~1',
 '2021_1950',
 '2021_1970\n2021_1970~1',
 '2021_1984',
 '2021_2004',
 '2021_2005',
 '202

##### 데이터 수정(STAGE5)

In [29]:
# 줄 바꿈이 있는 열
# 건설기계수입사(대표자)

# 줄 바꿈을 없앨 열
# 건설기계수입사(대표자)

In [30]:
# 줄 바꿈을 없앨 열
# 건설기계수입사(대표자)
cert_ld5['건설기계수입사(대표자)'] = cert_ld5['건설기계수입사(대표자)'].str.replace('\n', '')
cert_ld5[['건설기계수입사(대표자)']].head()

,건설기계수입사(대표자)
3,해당없음
4,LS엠트론(구자은)
5,삼정건설기계(유재흥)
6,LS엠트론(구자은)
7,삼정건설기계(유재흥)


##### [출력] 컬럼, 데이터 정보들

In [31]:
info01 = cert_ld34.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
info01['총 행수'] = cert_ld34.shape[0]
info01['번호'] = [x for x in range(1, info01.shape[0] + 1)]
info01 = info01[['번호', '컬럼명', '빈 값 개수', '총 행수']]
info01

,번호,컬럼명,빈 값 개수,총 행수
0,1,배출기준,0,5971
1,2,상호명,0,5971
2,3,인증번호,0,5971
3,4,최초인증일자,0,5971
4,5,재교부일자(변경일자),0,5971
...,...,...,...,...
59,60,적용기술_DPF,5683,5971
60,61,적용기술_ECU,5692,5971
61,62,적용기술_DOC,5771,5971
62,63,적용기술_기타,5882,5971


In [32]:
info02 = cert_ld5.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
info02['총 행수'] = cert_ld5.shape[0]
info02['번호'] = [x for x in range(1, info02.shape[0] + 1)]
info02 = info02[['번호', '컬럼명', '빈 값 개수', '총 행수']]
info02

,번호,컬럼명,빈 값 개수,총 행수
0,1,배출기준,1,2055
1,2,상호명,0,2055
2,3,인증번호,0,2055
3,4,최초인증일자,1,2055
4,5,재교부일자(변경일자),1,2055
...,...,...,...,...
69,70,적용기술_DPF,1925,2055
70,71,적용기술_ECU,1925,2055
71,72,적용기술_DOC,1925,2055
72,73,적용기술_기타,1951,2055


In [33]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.12.13'

In [34]:
# with pd.ExcelWriter(os.path.join(an2_fold, f'건설기계_인증대장_데이터정보({today_date}).xlsx')) as writer:
#     info01.to_excel(writer, sheet_name='TIER-3~TIER-4', index=False)
#     info02.to_excel(writer, sheet_name='STAGE5', index=False)

## 2-1 STD_BD21_1_CTCQ_MNGNO
- 건설기계 관리번호 통계

In [471]:
tcmaster.shape, len(tcmaster.ADM_NO.unique())

((624, 159), 624)

In [472]:
tcmaster.ENG_CAR_GB.unique()

array([nan, 'C', 'A', 'CA'], dtype=object)

In [473]:
pr21_1 = tcmaster.groupby(['ENG_CAR_GB', 'CERTI_STATE'], dropna=False)['ADM_NO'].count().reset_index()
pr21_1

,ENG_CAR_GB,CERTI_STATE,ADM_NO
0,A,42,2
1,A,90,110
2,A,C0,17
3,A,NaN,1
4,C,10,1
5,C,41,1
6,C,42,3
7,C,46,1
8,C,90,237
9,C,C0,1


In [474]:
pr21_1['ENG_CAR_GB'] = pr21_1['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr21_1['CERTI_STATE'] = pr21_1['CERTI_STATE'].replace({'90':'인증완료'})
pr21_1 = pr21_1.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'ADM_NO':'관리번호 개수'})
pr21_1

,건설기계/농업기계 구분,처분내용,관리번호 개수
0,농업기계,42,2
1,농업기계,인증완료,110
2,농업기계,C0,17
3,농업기계,NaN,1
4,0건설기계,10,1
5,0건설기계,41,1
6,0건설기계,42,3
7,0건설기계,46,1
8,0건설기계,인증완료,237
9,0건설기계,C0,1


In [475]:
pr21_2 = pr21_1.sort_values(['건설기계/농업기계 구분', '처분내용', '관리번호 개수'], ascending=[True, False, False]).reset_index(drop=True)
pr21_2['건설기계/농업기계 구분'] = pr21_2['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr21_2

,건설기계/농업기계 구분,처분내용,관리번호 개수
0,건설기계,인증완료,237
1,건설기계,C0,1
2,건설기계,46,1
3,건설기계,42,3
4,건설기계,41,1
5,건설기계,10,1
6,건설기계,NaN,41
7,건설/농업기계,인증완료,69
8,건설/농업기계,C0,1
9,건설/농업기계,42,6


In [476]:
pr21_2['관리번호 개수'].sum()

624

In [477]:
pr21_2.columns

Index(['건설기계/농업기계 구분', '처분내용', '관리번호 개수'], dtype='object')

In [478]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20231207'

In [479]:
pr21_2['테이블생성일자'] = today_date

In [480]:
pr21_2 = pr21_2[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '관리번호 개수', 
    '테이블생성일자', 
    ]]
# chd21_2 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '관리번호 개수':'', 
#     '테이블생성일자':'', 
# }
# pr21_2 = pr21_2.rename(columns=chd21_2)
pr21_2.head()

,건설기계/농업기계 구분,처분내용,관리번호 개수,테이블생성일자
0,건설기계,인증완료,237,20231207
1,건설기계,C0,1,20231207
2,건설기계,46,1,20231207
3,건설기계,42,3,20231207
4,건설기계,41,1,20231207


### [출력] STD_BD21_CTCQ_MNGNO

In [630]:
STD_BD21_CTCQ_MNGNO = pr21_2.copy()
STD_BD21_CTCQ_MNGNO.to_csv(os.path.join(df2_fold, f'STD_BD21_CTCQ_MNGNO_{today_date}.csv'), index=False)
STD_BD21_CTCQ_MNGNO.to_excel(os.path.join(df2_fold, f'STD_BD21_CTCQ_MNGNO_{today_date}.xlsx'), index=False)

## 2-2 STD_BD22_EXHST_GAS_CERT_NO_NOCS
- 배출인증번호 건수 통계

In [644]:
tcmaster['EMIS_CERTI_NO'].notnull().sum()

436

In [482]:
pr22_1 = tcmaster.groupby(['ENG_CAR_GB', 'CERTI_STATE'], dropna=False)['EMIS_CERTI_NO'].count().reset_index()
pr22_1

,ENG_CAR_GB,CERTI_STATE,EMIS_CERTI_NO
0,A,42,0
1,A,90,110
2,A,C0,17
3,A,NaN,0
4,C,10,0
5,C,41,0
6,C,42,0
7,C,46,0
8,C,90,236
9,C,C0,1


In [483]:
pr22_1['ENG_CAR_GB'] = pr22_1['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr22_1['CERTI_STATE'] = pr22_1['CERTI_STATE'].replace({'90':'인증완료'})
pr22_1 = pr22_1.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수'})
pr22_2 = pr22_1.sort_values(['건설기계/농업기계 구분', '처분내용', '배출인증번호 개수'], ascending=[True, False, False]).reset_index(drop=True)
pr22_2['건설기계/농업기계 구분'] = pr22_2['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr22_2

,건설기계/농업기계 구분,처분내용,배출인증번호 개수
0,건설기계,인증완료,236
1,건설기계,C0,1
2,건설기계,46,0
3,건설기계,42,0
4,건설기계,41,0
5,건설기계,10,0
6,건설기계,NaN,0
7,건설/농업기계,인증완료,69
8,건설/농업기계,C0,1
9,건설/농업기계,42,0


In [484]:
pr22_2['배출인증번호 개수'].sum()

436

In [485]:
pr22_2['테이블생성일자'] = today_date

In [486]:
pr22_2 = pr22_2[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd22_2 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '배출인증번호 개수':'', 
#     '테이블생성일자':'', 
# }
# pr22_2 = pr22_2.rename(columns=chd22_2)
pr22_2.head()

,건설기계/농업기계 구분,처분내용,배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,236,20231207
1,건설기계,C0,1,20231207
2,건설기계,46,0,20231207
3,건설기계,42,0,20231207
4,건설기계,41,0,20231207


### [출력] STD_BD22_EXHST_GAS_CERT_NO_NOCS

In [631]:
STD_BD22_EXHST_GAS_CERT_NO_NOCS = pr22_2.copy()
STD_BD22_EXHST_GAS_CERT_NO_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD22_EXHST_GAS_CERT_NO_NOCS_{today_date}.csv'), index=False)
STD_BD22_EXHST_GAS_CERT_NO_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD22_EXHST_GAS_CERT_NO_NOCS_{today_date}.xlsx'), index=False)

## 2-3 STD_BD23_FUEL_CERT_NOCS
- 연료별 인증건수 통계

In [488]:
tcmaster['TEST_FUEL'].value_counts(dropna=False)

TEST_FUEL
NaN    624
Name: count, dtype: int64

In [489]:
pr23_1 = tcmaster.groupby(['ENG_CAR_GB', 'CERTI_STATE', 'TEST_FUEL'], dropna=False)['EMIS_CERTI_NO'].count().reset_index()
pr23_1

,ENG_CAR_GB,CERTI_STATE,TEST_FUEL,EMIS_CERTI_NO
0,A,42,NaN,0
1,A,90,NaN,110
2,A,C0,NaN,17
3,A,NaN,NaN,0
4,C,10,NaN,0
5,C,41,NaN,0
6,C,42,NaN,0
7,C,46,NaN,0
8,C,90,NaN,236
9,C,C0,NaN,1


In [490]:
pr23_1['ENG_CAR_GB'] = pr23_1['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr23_1['CERTI_STATE'] = pr23_1['CERTI_STATE'].replace({'90':'인증완료'})
pr23_1 = pr23_1.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수', 'TEST_FUEL':'시험연료'})
pr23_2 = pr23_1.sort_values(['건설기계/농업기계 구분', '처분내용', '배출인증번호 개수'], ascending=[True, False, False]).reset_index(drop=True)
pr23_2['건설기계/농업기계 구분'] = pr23_2['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr23_2

,건설기계/농업기계 구분,처분내용,시험연료,배출인증번호 개수
0,건설기계,인증완료,NaN,236
1,건설기계,C0,NaN,1
2,건설기계,46,NaN,0
3,건설기계,42,NaN,0
4,건설기계,41,NaN,0
5,건설기계,10,NaN,0
6,건설기계,NaN,NaN,0
7,건설/농업기계,인증완료,NaN,69
8,건설/농업기계,C0,NaN,1
9,건설/농업기계,42,NaN,0


In [491]:
pr23_2['배출인증번호 개수'].sum()

436

In [492]:
pr23_2['테이블생성일자'] = today_date

In [493]:
pr23_2 = pr23_2[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '시험연료', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd23_2 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '시험연료':'', 
#     '배출인증번호 개수':'', 
#     '테이블생성일자':'', 
# }
# pr23_2 = pr23_2.rename(columns=chd23_2)
pr23_2.head()

,건설기계/농업기계 구분,처분내용,시험연료,배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,NaN,236,20231207
1,건설기계,C0,NaN,1,20231207
2,건설기계,46,NaN,0,20231207
3,건설기계,42,NaN,0,20231207
4,건설기계,41,NaN,0,20231207


### [출력] STD_BD23_FUEL_CERT_NOCS

In [632]:
STD_BD23_FUEL_CERT_NOCS = pr23_2.copy()
STD_BD23_FUEL_CERT_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD23_FUEL_CERT_NOCS_{today_date}.csv'), index=False)
STD_BD23_FUEL_CERT_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD23_FUEL_CERT_NOCS_{today_date}.xlsx'), index=False)

## 2-4 STD_BD24_OTPT_CERT_NOCS
- 출력별 인증건수 통계

In [495]:
eng_tefspec['RATED_OUT'].value_counts(dropna=False)

RATED_OUT
129     179
168     141
18.2    118
265     110
151     108
       ... 
67.3      1
62.2      1
69.2      1
71.0      1
332       1
Name: count, Length: 821, dtype: int64

In [496]:
eng_tefspec.shape, len(eng_tefspec['ADM_NO'].unique())

((8646, 108), 472)

In [497]:
eng_tefspec.loc[eng_tefspec['EF_SNO'].duplicated(), ['ADM_NO', 'EF_SNO', 'RATED_OUT', 'ENG_DISPL']].sort_values(['ADM_NO', 'EF_SNO'])

,ADM_NO,EF_SNO,RATED_OUT,ENG_DISPL
5021,52346,0,405,12742.00
5018,52346,1,264,12742.00
5019,52346,2,283,12742.00
5020,52346,3,202,9291.00
5023,52346,7,331,12742.00
...,...,...,...,...
7871,59500,3,55.4,2799.00
8314,59571,0,18.4,1267.00
8315,59571,1,18.4,1267.00
7903,59573,0,268.0,15239.00


In [498]:
eng_tefspec.drop_duplicates(['ADM_NO']).shape

(472, 108)

In [499]:
'ENG_CAR_GB' in tcmaster.columns

True

In [500]:
'ENG_CAR_GB' in eng_tefspec.columns

True

In [501]:
pr24_1 = tcmaster[['ADM_NO', 'CERTI_STATE', 'EMIS_CERTI_NO']].merge(eng_tefspec, on='ADM_NO', how='left')
pr24_1.shape

(8495, 110)

In [502]:
'ENG_CAR_GB' in pr24_1.columns

True

In [649]:
pr24_2 = pr24_1.groupby(['ENG_CAR_GB', 'CERTI_STATE', 'RATED_OUT'], dropna=False).agg({'EMIS_CERTI_NO':lambda x:len(x.unique())}).reset_index()
pr24_2

,ENG_CAR_GB,CERTI_STATE,RATED_OUT,EMIS_CERTI_NO
0,A,90,10.2,1
1,A,90,100,1
2,A,90,100.0,1
3,A,90,102,1
4,A,90,103,5
...,...,...,...,...
1614,NaN,C0,47.8,1
1615,NaN,C0,51.5,1
1616,NaN,C0,52,1
1617,NaN,C0,55,3


In [650]:
pr24_2['ENG_CAR_GB'] = pr24_2['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr24_2['CERTI_STATE'] = pr24_2['CERTI_STATE'].replace({'90':'인증완료'})
pr24_2 = pr24_2.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수', 'RATED_OUT':'정격출력(kW)'})
pr24_3 = pr24_2.sort_values(['처분내용', '건설기계/농업기계 구분', '정격출력(kW)'], ascending=[False, True, True]).reset_index(drop=True)
pr24_3['건설기계/농업기계 구분'] = pr24_3['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr24_3

,건설기계/농업기계 구분,처분내용,정격출력(kW),배출인증번호 개수
0,건설기계,인증완료,10.2,1
1,건설기계,인증완료,10.3,2
2,건설기계,인증완료,100,8
3,건설기계,인증완료,100.0,1
4,건설기계,인증완료,102,3
...,...,...,...,...
1614,건설기계,NaN,73,1
1615,건설기계,NaN,79,1
1616,건설기계,NaN,82,1
1617,건설기계,NaN,86,1


In [651]:
pr24_3['배출인증번호 개수'].sum()

2785

In [652]:
pr24_3['테이블생성일자'] = today_date

In [653]:
pr24_3 = pr24_3[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '정격출력(kW)', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd24_3 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '정격출력(kW)':'', 
#     '배출인증번호 개수':'', 
#     '테이블생성일자':'', 
# }
# pr24_3 = pr24_3.rename(columns=chd24_3)
pr24_3.head()

,건설기계/농업기계 구분,처분내용,정격출력(kW),배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,10.2,1,20231207
1,건설기계,인증완료,10.3,2,20231207
2,건설기계,인증완료,100,8,20231207
3,건설기계,인증완료,100.0,1,20231207
4,건설기계,인증완료,102,3,20231207


### [출력] STD_BD24_OTPT_CERT_NOCS

In [654]:
STD_BD24_OTPT_CERT_NOCS = pr24_3.copy()
STD_BD24_OTPT_CERT_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD24_OTPT_CERT_NOCS_{today_date}.csv'), index=False)
STD_BD24_OTPT_CERT_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD24_OTPT_CERT_NOCS_{today_date}.xlsx'), index=False)

## 2-5 STD_BD25_DSPLVL_CERT_NOCS
- 배기량별 인증건수 통계

In [509]:
pr25_1 = tcmaster[['ADM_NO', 'CERTI_STATE', 'EMIS_CERTI_NO']].merge(eng_tefspec, on='ADM_NO', how='left')
pr25_1.shape

(8495, 110)

In [655]:
pr25_2 = pr25_1.groupby(['ENG_CAR_GB', 'CERTI_STATE', 'ENG_DISPL'], dropna=False).agg({'EMIS_CERTI_NO':lambda x:len(x.unique())}).reset_index()
pr25_2

,ENG_CAR_GB,CERTI_STATE,ENG_DISPL,EMIS_CERTI_NO
0,A,90,719.00,1
1,A,90,761.00,1
2,A,90,898.00,1
3,A,90,993.00,1
4,A,90,1001.00,1
...,...,...,...,...
401,NaN,C0,2157.00,1
402,NaN,C0,2287.00,3
403,NaN,C0,2392.00,3
404,NaN,C0,3409.00,1


In [656]:
pr25_2['ENG_CAR_GB'] = pr25_2['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr25_2['CERTI_STATE'] = pr25_2['CERTI_STATE'].replace({'90':'인증완료'})
pr25_2 = pr25_2.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수', 'ENG_DISPL':'배기량'})
pr25_3 = pr25_2.sort_values(['처분내용', '건설기계/농업기계 구분', '배기량'], ascending=[False, True, True]).reset_index(drop=True)
pr25_3['건설기계/농업기계 구분'] = pr25_3['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr25_3

,건설기계/농업기계 구분,처분내용,배기량,배출인증번호 개수
0,건설기계,인증완료,479.00,1
1,건설기계,인증완료,719.00,2
2,건설기계,인증완료,823.00,1
3,건설기계,인증완료,854.00,2
4,건설기계,인증완료,898.00,2
...,...,...,...,...
401,건설기계,NaN,3387.00,1
402,건설기계,NaN,8808.00,1
403,건설기계,NaN,11823.00,1
404,건설기계,NaN,16123.00,1


In [657]:
pr25_3['배출인증번호 개수'].sum()

783

In [658]:
pr25_3['테이블생성일자'] = today_date

In [659]:
pr25_3 = pr25_3[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '배기량', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd25_3 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '시험연료':'', 
#     '배기량':'', 
#     '테이블생성일자':'', 
# }
# pr25_3 = pr25_3.rename(columns=chd25_3)
pr25_3.head()

,건설기계/농업기계 구분,처분내용,배기량,배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,479.00,1,20231207
1,건설기계,인증완료,719.00,2,20231207
2,건설기계,인증완료,823.00,1,20231207
3,건설기계,인증완료,854.00,2,20231207
4,건설기계,인증완료,898.00,2,20231207


### [출력] STD_BD25_DSPLVL_CERT_NOCS

In [660]:
STD_BD25_DSPLVL_CERT_NOCS = pr25_3.copy()
STD_BD25_DSPLVL_CERT_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD25_DSPLVL_CERT_NOCS_{today_date}.csv'), index=False)
STD_BD25_DSPLVL_CERT_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD25_DSPLVL_CERT_NOCS_{today_date}.xlsx'), index=False)

## 2-6 STD_BD26_NTN_CERT_NOCS
- 국내외별 인증건수 통계

In [668]:
pr26_1 = tcmaster.groupby(['ENG_CAR_GB', 'CERTI_STATE', 'CERTI_DIV'], dropna=False)['EMIS_CERTI_NO'].count().reset_index()
pr26_1

,ENG_CAR_GB,CERTI_STATE,CERTI_DIV,EMIS_CERTI_NO
0,A,42,1,0
1,A,90,1,41
2,A,90,2,69
3,A,C0,1,17
4,A,NaN,2,0
5,C,10,1,0
6,C,41,2,0
7,C,42,2,0
8,C,46,1,0
9,C,90,1,41


In [669]:
pr26_1['ENG_CAR_GB'] = pr26_1['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr26_1['CERTI_STATE'] = pr26_1['CERTI_STATE'].replace({'90':'인증완료'})
pr26_1['CERTI_DIV'] = pr26_1['CERTI_DIV'].replace({1:'국내', 2:'수입'})
pr26_1 = pr26_1.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수', 'CERTI_DIV':'인증구분'})
pr26_2 = pr26_1.sort_values(['처분내용', '건설기계/농업기계 구분', '배출인증번호 개수'], ascending=[False, True, False]).reset_index(drop=True)
pr26_2['건설기계/농업기계 구분'] = pr26_2['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr26_2

,건설기계/농업기계 구분,처분내용,인증구분,배출인증번호 개수
0,건설기계,인증완료,수입,195
1,건설기계,인증완료,국내,41
2,건설/농업기계,인증완료,수입,56
3,건설/농업기계,인증완료,국내,13
4,농업기계,인증완료,수입,69
5,농업기계,인증완료,국내,41
6,NaN,인증완료,국내,1
7,건설기계,C0,국내,1
8,건설/농업기계,C0,국내,1
9,농업기계,C0,국내,17


In [670]:
pr26_2['배출인증번호 개수'].sum()

436

In [671]:
pr26_2['테이블생성일자'] = today_date

In [672]:
pr26_2 = pr26_2[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '인증구분', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd26_2 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '인증구분':'', 
#     '배출인증번호 개수':'', 
#     '테이블생성일자':'', 
# }
# pr26_2 = pr26_2.rename(columns=chd26_2)
pr26_2.head()

,건설기계/농업기계 구분,처분내용,인증구분,배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,수입,195,20231207
1,건설기계,인증완료,국내,41,20231207
2,건설/농업기계,인증완료,수입,56,20231207
3,건설/농업기계,인증완료,국내,13,20231207
4,농업기계,인증완료,수입,69,20231207


### [출력] STD_BD26_NTN_CERT_NOCS

In [673]:
STD_BD26_NTN_CERT_NOCS = pr26_2.copy()
STD_BD26_NTN_CERT_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD26_NTN_CERT_NOCS_{today_date}.csv'), index=False)
STD_BD26_NTN_CERT_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD26_NTN_CERT_NOCS_{today_date}.xlsx'), index=False)

## 2-7 STD_BD27_FBCTN_NTN_CERT_NOCS
- 제작국별 인증건수 통계

In [674]:
pr27_1 = tcmaster.groupby(['ENG_CAR_GB', 'CERTI_STATE', 'NATION'], dropna=False)['EMIS_CERTI_NO'].count().reset_index()
pr27_1

,ENG_CAR_GB,CERTI_STATE,NATION,EMIS_CERTI_NO
0,A,42,KR,0
1,A,90,FN,6
2,A,90,FR,8
3,A,90,GM,15
4,A,90,IT,12
5,A,90,JP,26
6,A,90,KR,41
7,A,90,US,2
8,A,C0,KR,17
9,A,NaN,FN,0


In [663]:
pr27_1['ENG_CAR_GB'] = pr27_1['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr27_1['CERTI_STATE'] = pr27_1['CERTI_STATE'].replace({'90':'인증완료'})
pr27_1 = pr27_1.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수', 'NATION':'제작국가'})
pr27_2 = pr27_1.sort_values(['처분내용', '건설기계/농업기계 구분', '배출인증번호 개수'], ascending=[False, True, False]).reset_index(drop=True)
pr27_2['건설기계/농업기계 구분'] = pr27_2['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr27_2

,건설기계/농업기계 구분,처분내용,제작국가,배출인증번호 개수
0,건설기계,인증완료,JP,75
1,건설기계,인증완료,KR,41
2,건설기계,인증완료,GM,28
3,건설기계,인증완료,US,28
4,건설기계,인증완료,UK,24
5,건설기계,인증완료,SW,20
6,건설기계,인증완료,CN,8
7,건설기계,인증완료,IT,8
8,건설기계,인증완료,SU,4
9,건설/농업기계,인증완료,JP,37


In [664]:
pr27_2['배출인증번호 개수'].sum()

436

In [665]:
pr27_2['테이블생성일자'] = today_date

In [666]:
pr27_2 = pr27_2[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '제작국가', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd27_2 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '제작국가':'', 
#     '배출인증번호 개수':'', 
#     '테이블생성일자':'', 
# }
# pr27_2 = pr27_2.rename(columns=chd27_2)
pr27_2.head()

,건설기계/농업기계 구분,처분내용,제작국가,배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,JP,75,20231207
1,건설기계,인증완료,KR,41,20231207
2,건설기계,인증완료,GM,28,20231207
3,건설기계,인증완료,US,28,20231207
4,건설기계,인증완료,UK,24,20231207


### [출력] STD_BD27_FBCTN_NTN_CERT_NOCS

In [667]:
STD_BD27_FBCTN_NTN_CERT_NOCS = pr27_2.copy()
STD_BD27_FBCTN_NTN_CERT_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD27_FBCTN_NTN_CERT_NOCS_{today_date}.csv'), index=False)
STD_BD27_FBCTN_NTN_CERT_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD27_FBCTN_NTN_CERT_NOCS_{today_date}.xlsx'), index=False)

## 2-8 STD_BD28_CERT_YR_CERT_NOCS
- 인증연도별 인증 건수 통계

In [528]:
pr28_1 = tcmaster.copy()

In [529]:
pr28_1['REG_DATETIME'].dtype

dtype('O')

In [530]:
pr28_1['REG_DATETIME'].head()

0    19/01/24 08:47:12.000000000
1    19/01/31 16:26:07.000000000
2    19/04/19 14:16:55.000000000
3    19/04/19 14:22:44.000000000
4    19/04/19 14:46:46.000000000
Name: REG_DATETIME, dtype: object

In [531]:
pr28_1['REG_DATETIME'] = pr28_1['REG_DATETIME'].str.split('.').str[0]
pr28_1['REG_DATETIME'].head()

0    19/01/24 08:47:12
1    19/01/31 16:26:07
2    19/04/19 14:16:55
3    19/04/19 14:22:44
4    19/04/19 14:46:46
Name: REG_DATETIME, dtype: object

In [532]:
pr28_1['REG_DATETIME'].isnull().sum()

0

In [533]:
pr28_1['REG_DATETIME'] = pd.to_datetime(pr28_1['REG_DATETIME'], yearfirst=True)
pr28_1['REG_DATETIME'].head()

0   2019-01-24 08:47:12
1   2019-01-31 16:26:07
2   2019-04-19 14:16:55
3   2019-04-19 14:22:44
4   2019-04-19 14:46:46
Name: REG_DATETIME, dtype: datetime64[ns]

In [534]:
pr28_1['REG_DATETIME_year'] = pr28_1['REG_DATETIME'].dt.year
pr28_1[['REG_DATETIME', 'REG_DATETIME_year']].head()

,REG_DATETIME,REG_DATETIME_year
0,2019-01-24 08:47:12,2019
1,2019-01-31 16:26:07,2019
2,2019-04-19 14:16:55,2019
3,2019-04-19 14:22:44,2019
4,2019-04-19 14:46:46,2019


In [675]:
pr28_2 = pr28_1.groupby(['ENG_CAR_GB', 'CERTI_STATE', 'REG_DATETIME_year'], dropna=False)['EMIS_CERTI_NO'].count().reset_index()
pr28_2

,ENG_CAR_GB,CERTI_STATE,REG_DATETIME_year,EMIS_CERTI_NO
0,A,42,2021,0
1,A,90,2014,52
2,A,90,2015,23
3,A,90,2016,12
4,A,90,2017,2
5,A,90,2018,5
6,A,90,2019,3
7,A,90,2020,1
8,A,90,2021,1
9,A,90,2022,9


In [676]:
pr28_2['ENG_CAR_GB'] = pr28_2['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr28_2['CERTI_STATE'] = pr28_2['CERTI_STATE'].replace({'90':'인증완료'})
pr28_2 = pr28_2.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수', 'REG_DATETIME_year':'인증연도'})
pr28_3 = pr28_2.sort_values(['처분내용', '건설기계/농업기계 구분', '인증연도', '배출인증번호 개수'], ascending=[False, True, True, False]).reset_index(drop=True)
pr28_3['건설기계/농업기계 구분'] = pr28_3['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr28_3

,건설기계/농업기계 구분,처분내용,인증연도,배출인증번호 개수
0,건설기계,인증완료,2014,83
1,건설기계,인증완료,2015,49
2,건설기계,인증완료,2016,24
3,건설기계,인증완료,2017,9
4,건설기계,인증완료,2018,9
5,건설기계,인증완료,2019,17
6,건설기계,인증완료,2020,7
7,건설기계,인증완료,2021,13
8,건설기계,인증완료,2022,24
9,건설기계,인증완료,2023,1


In [677]:
pr28_3['배출인증번호 개수'].sum()

436

In [678]:
pr28_3['테이블생성일자'] = today_date

In [679]:
pr28_3 = pr28_3[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '인증연도', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd27_3 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '인증연도':'', 
#     '배출인증번호 개수':'', 
#     '테이블생성일자':'', 
# }
# pr28_3 = pr28_3.rename(columns=chd27_3)
pr28_3.head()

,건설기계/농업기계 구분,처분내용,인증연도,배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,2014,83,20231207
1,건설기계,인증완료,2015,49,20231207
2,건설기계,인증완료,2016,24,20231207
3,건설기계,인증완료,2017,9,20231207
4,건설기계,인증완료,2018,9,20231207


### [출력] STD_BD28_CERT_YR_CERT_NOCS

In [680]:
STD_BD28_CERT_YR_CERT_NOCS = pr28_3.copy()
STD_BD28_CERT_YR_CERT_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD28_CERT_YR_CERT_NOCS_{today_date}.csv'), index=False)
STD_BD28_CERT_YR_CERT_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD28_CERT_YR_CERT_NOCS_{today_date}.xlsx'), index=False)

## 2-9 STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU
- 제작사별 배출가스 측정치 통계

In [541]:
teemissn_eng_res.shape, len(teemissn_eng_res['ADM_NO'].unique())

((2761, 101), 487)

In [542]:
tcmaster.shape

(624, 159)

In [681]:
pr29_1 = tcmaster[['ADM_NO', 'ENG_CAR_GB', 'CERTI_STATE', 'EMIS_CERTI_NO']].merge(teemissn_eng_res, on='ADM_NO', how='left')
pr29_1.shape

(2710, 104)

In [682]:
pr29_2 = pr29_1.merge(eng_tefspec[['ADM_NO', 'ENG_MAKER']], on='ADM_NO', how='left')
pr29_2.shape

(55785, 105)

In [683]:
pr29_3 = pr29_2.groupby(['ENG_CAR_GB', 'CERTI_STATE', 'ENG_MAKER', 'EMIS_CERTI_NO'], dropna=False).agg({'CO_FINAL_RST':'mean', 'NMHC_FINAL_RST':'mean', 'NOX_FINAL_RST':'mean', 'PM_FINAL_RST':'mean', 'PN_FINAL_RST':'mean', 'NH3_FINAL_RST':'mean', 'NMHCNOX_FINAL_RST':'mean'}).reset_index()
pr29_3

,ENG_CAR_GB,CERTI_STATE,ENG_MAKER,EMIS_CERTI_NO,CO_FINAL_RST,NMHC_FINAL_RST,NOX_FINAL_RST,PM_FINAL_RST,PN_FINAL_RST,NH3_FINAL_RST,NMHCNOX_FINAL_RST
0,A,42,국제종합기계,NaN,0.05,NaN,NaN,0.01,6.56,NaN,4.09
1,A,90,(주)대동,22EN*DD*02,0.15,0.04,3.49,0.00,153000000000.00,NaN,3.53
2,A,90,AGCO POWER,22EN*AP*01,0.01,0.01,0.33,0.01,710000000000.00,NaN,NaN
3,A,90,AGCO POWER,23EN*AP*01,0.01,0.02,0.24,0.00,718800000000.00,NaN,NaN
4,A,90,Agco Sisu Power,14EN*AP*01,0.07,NaN,NaN,0.01,NaN,NaN,1.54
...,...,...,...,...,...,...,...,...,...,...,...
515,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,NaN,90,제작사1,14EN*SS*01,1.50,NaN,NaN,0.10,NaN,NaN,1.10
517,NaN,NaN,CATERPILLAR,09EN*CA*02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,NaN,NaN,Caterpillar,09EN*CA*02,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [684]:
pr29_3['ENG_CAR_GB'] = pr29_3['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr29_3['CERTI_STATE'] = pr29_3['CERTI_STATE'].replace({'90':'인증완료'})
pr29_3 = pr29_3.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호', 'ENG_MAKER':'제작사', 'CO_FINAL_RST':'CO_FINAL_RST_평균', 'NMHC_FINAL_RST':'NMHC_FINAL_RST_평균', 'NOX_FINAL_RST':'NOX_FINAL_RST_평균', 'PM_FINAL_RST':'PM_FINAL_RST_평균', 'PN_FINAL_RST':'PN_FINAL_RST_평균', 'NH3_FINAL_RST':'NH3_FINAL_RST_평균', 'NMHCNOX_FINAL_RST':'NMHCNOX_FINAL_RST_평균'})
pr29_4 = pr29_3.sort_values(['처분내용', '건설기계/농업기계 구분', '제작사'], ascending=[False, True, True]).reset_index(drop=True)
pr29_4['건설기계/농업기계 구분'] = pr29_4['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr29_4

,건설기계/농업기계 구분,처분내용,제작사,배출인증번호,CO_FINAL_RST_평균,NMHC_FINAL_RST_평균,NOX_FINAL_RST_평균,PM_FINAL_RST_평균,PN_FINAL_RST_평균,NH3_FINAL_RST_평균,NMHCNOX_FINAL_RST_평균
0,건설기계,인증완료,(주)대동,21EN*DD*02,0.01,0.03,3.39,0.01,34400000000.00,NaN,3.42
1,건설기계,인증완료,(주)코마츠제작소,09EN*KO*01,0.90,NaN,NaN,0.10,NaN,NaN,3.51
2,건설기계,인증완료,B4.5,19EN*CU*05,0.07,0.04,0.12,0.01,142689000000000.00,0.00,NaN
3,건설기계,인증완료,CATERPILLAR,09EN*CA*01,2.38,NaN,NaN,0.12,NaN,NaN,3.47
4,건설기계,인증완료,CATERPILLAR,09EN*CA*04,2.51,NaN,NaN,0.11,NaN,NaN,3.77
...,...,...,...,...,...,...,...,...,...,...,...
515,건설/농업기계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,농업기계,NaN,NaN,NaN,0.07,0.00,0.28,0.01,NaN,NaN,NaN
517,NaN,NaN,CATERPILLAR,09EN*CA*02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,NaN,NaN,Caterpillar,09EN*CA*02,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [685]:
pr29_4.iloc[:, 4:] = pr29_4.iloc[:, 4:].round(2)
pr29_4.head()

,건설기계/농업기계 구분,처분내용,제작사,배출인증번호,CO_FINAL_RST_평균,NMHC_FINAL_RST_평균,NOX_FINAL_RST_평균,PM_FINAL_RST_평균,PN_FINAL_RST_평균,NH3_FINAL_RST_평균,NMHCNOX_FINAL_RST_평균
0,건설기계,인증완료,(주)대동,21EN*DD*02,0.01,0.03,3.39,0.01,34400000000.00,NaN,3.42
1,건설기계,인증완료,(주)코마츠제작소,09EN*KO*01,0.90,NaN,NaN,0.10,NaN,NaN,3.51
2,건설기계,인증완료,B4.5,19EN*CU*05,0.07,0.04,0.12,0.01,142689000000000.00,0.00,NaN
3,건설기계,인증완료,CATERPILLAR,09EN*CA*01,2.38,NaN,NaN,0.12,NaN,NaN,3.47
4,건설기계,인증완료,CATERPILLAR,09EN*CA*04,2.51,NaN,NaN,0.11,NaN,NaN,3.77


In [686]:
pr29_4['테이블생성일자'] = today_date

In [687]:
pr29_4 = pr29_4[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '제작사', 
    '배출인증번호', 
    'CO_FINAL_RST_평균', 
    'NMHC_FINAL_RST_평균', 
    'NOX_FINAL_RST_평균', 
    'PM_FINAL_RST_평균', 
    'PN_FINAL_RST_평균', 
    'NH3_FINAL_RST_평균', 
    'NMHCNOX_FINAL_RST_평균', 
    '테이블생성일자', 
    ]]
# chd29_4 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '인증연도':'', 
#     '배출인증번호':'', 
#     'CO_FINAL_RST_평균':'', 
#     'NMHC_FINAL_RST_평균':'', 
#     'NOX_FINAL_RST_평균':'', 
#     'PM_FINAL_RST_평균':'', 
#     'PN_FINAL_RST_평균':'', 
#     'NH3_FINAL_RST_평균':'', 
#     'NMHCNOX_FINAL_RST_평균':'', 
#     '테이블생성일자':'', 
# }
# pr29_4 = pr29_4.rename(columns=chd29_4)
pr29_4.head()

,건설기계/농업기계 구분,처분내용,제작사,배출인증번호,CO_FINAL_RST_평균,NMHC_FINAL_RST_평균,NOX_FINAL_RST_평균,PM_FINAL_RST_평균,PN_FINAL_RST_평균,NH3_FINAL_RST_평균,NMHCNOX_FINAL_RST_평균,테이블생성일자
0,건설기계,인증완료,(주)대동,21EN*DD*02,0.01,0.03,3.39,0.01,34400000000.00,NaN,3.42,20231207
1,건설기계,인증완료,(주)코마츠제작소,09EN*KO*01,0.90,NaN,NaN,0.10,NaN,NaN,3.51,20231207
2,건설기계,인증완료,B4.5,19EN*CU*05,0.07,0.04,0.12,0.01,142689000000000.00,0.00,NaN,20231207
3,건설기계,인증완료,CATERPILLAR,09EN*CA*01,2.38,NaN,NaN,0.12,NaN,NaN,3.47,20231207
4,건설기계,인증완료,CATERPILLAR,09EN*CA*04,2.51,NaN,NaN,0.11,NaN,NaN,3.77,20231207


### [출력] STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU

In [688]:
STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU = pr29_4.copy()
STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU.to_csv(os.path.join(df2_fold, f'STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU_{today_date}.csv'), index=False)
STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU.to_excel(os.path.join(df2_fold, f'STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU_{today_date}.xlsx'), index=False)

## [수정중] 2-10 STD_BD210_MNFCTR_CERT_MEVLU
- 제작사별 인증 측정치 통계

In [ ]:
teemissn_eng_res.shape, len(teemissn_eng_res['ADM_NO'].unique())

((2761, 101), 487)

In [ ]:
tcmaster.shape

(624, 159)

In [ ]:
pr29_1 = tcmaster[['ADM_NO', 'ENG_CAR_GB', 'CERTI_STATE', 'EMIS_CERTI_NO']].merge(teemissn_eng_res, on='ADM_NO', how='left')
pr29_1.shape

(2710, 104)

In [ ]:
pr29_2 = pr29_1.merge(eng_tefspec[['ADM_NO', 'ENG_MAKER']], on='ADM_NO', how='left')
pr29_2.shape

(55785, 105)

In [ ]:
pr29_3 = pr29_2.groupby(['ENG_CAR_GB', 'CERTI_STATE', 'ENG_MAKER', 'EMIS_CERTI_NO'], dropna=False).agg({'CO_FINAL_RST':'mean', 'NMHC_FINAL_RST':'mean', 'NOX_FINAL_RST':'mean', 'PM_FINAL_RST':'mean', 'PN_FINAL_RST':'mean', 'NH3_FINAL_RST':'mean', 'NMHCNOX_FINAL_RST':'mean'}).reset_index()
pr29_3

,ENG_CAR_GB,CERTI_STATE,ENG_MAKER,EMIS_CERTI_NO,CO_FINAL_RST,NMHC_FINAL_RST,NOX_FINAL_RST,PM_FINAL_RST,PN_FINAL_RST,NH3_FINAL_RST,NMHCNOX_FINAL_RST
0,A,42,국제종합기계,NaN,0.05,NaN,NaN,0.01,6.56,NaN,4.09
1,A,90,(주)대동,22EN*DD*02,0.15,0.04,3.49,0.00,153000000000.00,NaN,3.53
2,A,90,AGCO POWER,22EN*AP*01,0.01,0.01,0.33,0.01,710000000000.00,NaN,NaN
3,A,90,AGCO POWER,23EN*AP*01,0.01,0.02,0.24,0.00,718800000000.00,NaN,NaN
4,A,90,Agco Sisu Power,14EN*AP*01,0.07,NaN,NaN,0.01,NaN,NaN,1.54
...,...,...,...,...,...,...,...,...,...,...,...
515,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,NaN,90,제작사1,14EN*SS*01,1.50,NaN,NaN,0.10,NaN,NaN,1.10
517,NaN,NaN,CATERPILLAR,09EN*CA*02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,NaN,NaN,Caterpillar,09EN*CA*02,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pr29_3['ENG_CAR_GB'] = pr29_3['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr29_3['CERTI_STATE'] = pr29_3['CERTI_STATE'].replace({'90':'인증완료'})
pr29_3 = pr29_3.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호', 'ENG_MAKER':'제작사', 'CO_FINAL_RST':'CO_FINAL_RST_평균', 'NMHC_FINAL_RST':'NMHC_FINAL_RST_평균', 'NOX_FINAL_RST':'NOX_FINAL_RST_평균', 'PM_FINAL_RST':'PM_FINAL_RST_평균', 'PN_FINAL_RST':'PN_FINAL_RST_평균', 'NH3_FINAL_RST':'NH3_FINAL_RST_평균', 'NMHCNOX_FINAL_RST':'NMHCNOX_FINAL_RST_평균'})
pr29_4 = pr29_3.sort_values(['처분내용', '건설기계/농업기계 구분', '제작사'], ascending=[False, True, True]).reset_index(drop=True)
pr29_4['건설기계/농업기계 구분'] = pr29_4['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr29_4

,건설기계/농업기계 구분,처분내용,제작사,배출인증번호,CO_FINAL_RST_평균,NMHC_FINAL_RST_평균,NOX_FINAL_RST_평균,PM_FINAL_RST_평균,PN_FINAL_RST_평균,NH3_FINAL_RST_평균,NMHCNOX_FINAL_RST_평균
0,건설기계,인증완료,(주)대동,21EN*DD*02,0.01,0.03,3.39,0.01,34400000000.00,NaN,3.42
1,건설기계,인증완료,(주)코마츠제작소,09EN*KO*01,0.90,NaN,NaN,0.10,NaN,NaN,3.51
2,건설기계,인증완료,B4.5,19EN*CU*05,0.07,0.04,0.12,0.01,142689000000000.00,0.00,NaN
3,건설기계,인증완료,CATERPILLAR,09EN*CA*01,2.38,NaN,NaN,0.12,NaN,NaN,3.47
4,건설기계,인증완료,CATERPILLAR,09EN*CA*04,2.51,NaN,NaN,0.11,NaN,NaN,3.77
...,...,...,...,...,...,...,...,...,...,...,...
515,건설/농업기계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,농업기계,NaN,NaN,NaN,0.07,0.00,0.28,0.01,NaN,NaN,NaN
517,NaN,NaN,CATERPILLAR,09EN*CA*02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,NaN,NaN,Caterpillar,09EN*CA*02,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pr29_4.iloc[:, 4:] = pr29_4.iloc[:, 4:].round(2)
pr29_4.head()

,건설기계/농업기계 구분,처분내용,제작사,배출인증번호,CO_FINAL_RST_평균,NMHC_FINAL_RST_평균,NOX_FINAL_RST_평균,PM_FINAL_RST_평균,PN_FINAL_RST_평균,NH3_FINAL_RST_평균,NMHCNOX_FINAL_RST_평균
0,건설기계,인증완료,(주)대동,21EN*DD*02,0.01,0.03,3.39,0.01,34400000000.00,NaN,3.42
1,건설기계,인증완료,(주)코마츠제작소,09EN*KO*01,0.90,NaN,NaN,0.10,NaN,NaN,3.51
2,건설기계,인증완료,B4.5,19EN*CU*05,0.07,0.04,0.12,0.01,142689000000000.00,0.00,NaN
3,건설기계,인증완료,CATERPILLAR,09EN*CA*01,2.38,NaN,NaN,0.12,NaN,NaN,3.47
4,건설기계,인증완료,CATERPILLAR,09EN*CA*04,2.51,NaN,NaN,0.11,NaN,NaN,3.77


In [ ]:
pr29_4['테이블생성일자'] = today_date

In [ ]:
pr29_4 = pr29_4[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '제작사', 
    '배출인증번호', 
    'CO_FINAL_RST_평균', 
    'NMHC_FINAL_RST_평균', 
    'NOX_FINAL_RST_평균', 
    'PM_FINAL_RST_평균', 
    'PN_FINAL_RST_평균', 
    'NH3_FINAL_RST_평균', 
    'NMHCNOX_FINAL_RST_평균', 
    '테이블생성일자', 
    ]]
# chd29_4 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '인증연도':'', 
#     '배출인증번호':'', 
#     'CO_FINAL_RST_평균':'', 
#     'NMHC_FINAL_RST_평균':'', 
#     'NOX_FINAL_RST_평균':'', 
#     'PM_FINAL_RST_평균':'', 
#     'PN_FINAL_RST_평균':'', 
#     'NH3_FINAL_RST_평균':'', 
#     'NMHCNOX_FINAL_RST_평균':'', 
#     '테이블생성일자':'', 
# }
# pr29_4 = pr29_4.rename(columns=chd29_4)
pr29_4.head()

,건설기계/농업기계 구분,처분내용,제작사,배출인증번호,CO_FINAL_RST_평균,NMHC_FINAL_RST_평균,NOX_FINAL_RST_평균,PM_FINAL_RST_평균,PN_FINAL_RST_평균,NH3_FINAL_RST_평균,NMHCNOX_FINAL_RST_평균,테이블생성일자
0,건설기계,인증완료,(주)대동,21EN*DD*02,0.01,0.03,3.39,0.01,34400000000.00,NaN,3.42,20231207
1,건설기계,인증완료,(주)코마츠제작소,09EN*KO*01,0.90,NaN,NaN,0.10,NaN,NaN,3.51,20231207
2,건설기계,인증완료,B4.5,19EN*CU*05,0.07,0.04,0.12,0.01,142689000000000.00,0.00,NaN,20231207
3,건설기계,인증완료,CATERPILLAR,09EN*CA*01,2.38,NaN,NaN,0.12,NaN,NaN,3.47,20231207
4,건설기계,인증완료,CATERPILLAR,09EN*CA*04,2.51,NaN,NaN,0.11,NaN,NaN,3.77,20231207


### [출력] STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU

In [ ]:
STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU = pr29_4.copy()
STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU.to_csv(os.path.join(df2_fold, f'STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU_{today_date}.csv'), index=False)
STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU.to_excel(os.path.join(df2_fold, f'STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU_{today_date}.xlsx'), index=False)

## 2-10 STD_BD210_CERT_REGSTR_CERT_NO
- 인증대장 인증번호 통계

In [551]:
cert_ld34.shape, len(cert_ld34['인증번호'].unique())

((5971, 64), 368)

In [552]:
cert_ld5.shape, len(cert_ld5['인증번호'].unique())

((2055, 74), 131)

In [553]:
cert_ld34['인증번호'].notnull().sum()

5971

In [554]:
cert_ld5['인증번호'].notnull().sum()

2055

In [555]:
pr210_1 = cert_ld34.groupby('종류', dropna=False)['인증번호'].count().reset_index()
pr210_1 = pr210_1.rename(columns={'인증번호':'인증번호개수'})
pr210_1['시트명'] = 'TIER-3~TIER-4'
pr210_1.head()

,종류,인증번호개수,시트명
0,건설기계,4519,TIER-3~TIER-4
1,농업기계,1452,TIER-3~TIER-4


In [556]:
pr210_1['인증번호개수'].sum()

5971

In [557]:
pr210_2 = cert_ld5.groupby('종류', dropna=False)['인증번호'].count().reset_index()
pr210_2 = pr210_2.rename(columns={'인증번호':'인증번호개수'})
pr210_2['시트명'] = 'STAGE5'
pr210_2.head()

,종류,인증번호개수,시트명
0,건설기계,1430,STAGE5
1,농업기계,624,STAGE5
2,NaN,1,STAGE5


In [558]:
pr210_2['인증번호개수'].sum()

2055

In [559]:
pr210_3 = pd.concat([pr210_1, pr210_2], ignore_index=True)
pr210_3

,종류,인증번호개수,시트명
0,건설기계,4519,TIER-3~TIER-4
1,농업기계,1452,TIER-3~TIER-4
2,건설기계,1430,STAGE5
3,농업기계,624,STAGE5
4,NaN,1,STAGE5


In [560]:
pr210_3['인증번호개수'].sum()

8026

In [561]:
pr210_3['테이블생성일자'] = today_date

In [695]:
pr210_3 = pr210_3[[
    '시트명', 
    '종류', 
    '인증번호개수', 
    '테이블생성일자', 
    ]]
# chd210_3 = {
#     '종류':'', 
#     '시트명':'', 
#     '인증번호개수':'', 
#     '테이블생성일자':'', 
# }
# pr210_3 = pr210_3.rename(columns=chd210_3)
pr210_3.head()

,시트명,종류,인증번호개수,테이블생성일자
0,TIER-3~TIER-4,건설기계,4519,20231207
1,TIER-3~TIER-4,농업기계,1452,20231207
2,STAGE5,건설기계,1430,20231207
3,STAGE5,농업기계,624,20231207
4,STAGE5,NaN,1,20231207


### [출력] STD_BD210_CERT_REGSTR_CERT_NO

In [696]:
STD_BD210_CERT_REGSTR_CERT_NO = pr210_3.copy()
STD_BD210_CERT_REGSTR_CERT_NO.to_csv(os.path.join(df2_fold, f'STD_BD210_CERT_REGSTR_CERT_NO_{today_date}.csv'), index=False)
STD_BD210_CERT_REGSTR_CERT_NO.to_excel(os.path.join(df2_fold, f'STD_BD210_CERT_REGSTR_CERT_NO_{today_date}.xlsx'), index=False)

## 2-11 STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO
- 엔진배기량별 인증대장 인증번호 통계

In [564]:
cert_ld34.shape, len(cert_ld34['인증번호'].unique())

((5971, 64), 368)

In [565]:
cert_ld34['인증번호'].notnull().sum()

5971

In [566]:
cert_ld5.shape, len(cert_ld5['인증번호'].unique())

((2055, 74), 131)

In [567]:
cert_ld5['인증번호'].notnull().sum()

2055

In [568]:
pr211_1 = cert_ld34.groupby(['종류', '엔진배기량'], dropna=False)['인증번호'].count().reset_index()
pr211_1 = pr211_1.rename(columns={'인증번호':'인증번호개수'})
pr211_1['시트명'] = 'TIER-3~TIER-4'
pr211_1.head()

,종류,엔진배기량,인증번호개수,시트명
0,건설기계,479,10,TIER-3~TIER-4
1,건설기계,719,9,TIER-3~TIER-4
2,건설기계,854,14,TIER-3~TIER-4
3,건설기계,898,6,TIER-3~TIER-4
4,건설기계,993,7,TIER-3~TIER-4


In [569]:
pr211_1['인증번호개수'].sum()

5971

In [570]:
pr211_2 = cert_ld5.groupby(['종류', '엔진배기량'], dropna=False)['인증번호'].count().reset_index()
pr211_2 = pr211_2.rename(columns={'인증번호':'인증번호개수'})
pr211_2['시트명'] = 'STAGE5'
pr211_2.head()

,종류,엔진배기량,인증번호개수,시트명
0,건설기계,479.00,3,STAGE5
1,건설기계,719.00,22,STAGE5
2,건설기계,778.00,1,STAGE5
3,건설기계,854.00,12,STAGE5
4,건설기계,898.00,12,STAGE5


In [571]:
pr211_2['인증번호개수'].sum()

2055

In [572]:
pr211_3 = pd.concat([pr211_1, pr211_2], ignore_index=True)
pr211_3

,종류,엔진배기량,인증번호개수,시트명
0,건설기계,479,10,TIER-3~TIER-4
1,건설기계,719,9,TIER-3~TIER-4
2,건설기계,854,14,TIER-3~TIER-4
3,건설기계,898,6,TIER-3~TIER-4
4,건설기계,993,7,TIER-3~TIER-4
...,...,...,...,...
346,농업기계,6596.00,2,STAGE5
347,농업기계,6728.00,86,STAGE5
348,농업기계,6788.00,37,STAGE5
349,농업기계,7365.00,12,STAGE5


In [573]:
pr211_3['인증번호개수'].sum()

8026

In [574]:
pr211_3['테이블생성일자'] = today_date

In [693]:
pr211_3 = pr211_3[[
    '시트명', 
    '종류', 
    '엔진배기량', 
    '인증번호개수', 
    '테이블생성일자', 
    ]]
# chd211_3 = {
#     '종류':'',
#     '엔진배기량':'',  
#     '시트명':'', 
#     '인증번호개수':'', 
#     '테이블생성일자':'', 
# }
# pr211_3 = pr211_3.rename(columns=chd211_3)
pr211_3.head()

,시트명,종류,엔진배기량,인증번호개수,테이블생성일자
0,TIER-3~TIER-4,건설기계,479,10,20231207
1,TIER-3~TIER-4,건설기계,719,9,20231207
2,TIER-3~TIER-4,건설기계,854,14,20231207
3,TIER-3~TIER-4,건설기계,898,6,20231207
4,TIER-3~TIER-4,건설기계,993,7,20231207


### [출력] STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO

In [694]:
STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO = pr211_3.copy()
STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO.to_csv(os.path.join(df2_fold, f'STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO_{today_date}.csv'), index=False)
STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO.to_excel(os.path.join(df2_fold, f'STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO_{today_date}.xlsx'), index=False)

## 2-12 STD_BD212_OTPT_CERT_REGSTR_CERT_NO
- 출력별 인증대장 인증번호 통계

In [577]:
cert_ld34.shape, len(cert_ld34['인증번호'].unique())

((5971, 64), 368)

In [578]:
cert_ld34['인증번호'].notnull().sum()

5971

In [579]:
cert_ld5.shape, len(cert_ld5['인증번호'].unique())

((2055, 74), 131)

In [580]:
cert_ld5['인증번호'].notnull().sum()

2055

In [581]:
pr212_1 = cert_ld34.groupby(['종류', '최대출력_KW'], dropna=False)['인증번호'].count().reset_index()
pr212_1 = pr212_1.rename(columns={'인증번호':'인증번호개수'})
pr212_1['시트명'] = 'TIER-3~TIER-4'
pr212_1.head()

,종류,최대출력_KW,인증번호개수,시트명
0,건설기계,100(N:134ps),1,TIER-3~TIER-4
1,건설기계,105(142ps),1,TIER-3~TIER-4
2,건설기계,106.6(N:137ps),1,TIER-3~TIER-4
3,건설기계,106.6(N:145ps),1,TIER-3~TIER-4
4,건설기계,108(135ps),1,TIER-3~TIER-4


In [582]:
pr212_1['인증번호개수'].sum()

5971

In [583]:
pr212_2 = cert_ld5.groupby(['종류', '최대출력_KW'], dropna=False)['인증번호'].count().reset_index()
pr212_2 = pr212_2.rename(columns={'인증번호':'인증번호개수'})
pr212_2['시트명'] = 'STAGE5'
pr212_2.head()

,종류,최대출력_KW,인증번호개수,시트명
0,건설기계,7.10,3,STAGE5
1,건설기계,7.50,2,STAGE5
2,건설기계,7.60,4,STAGE5
3,건설기계,8.80,2,STAGE5
4,건설기계,9.30,3,STAGE5


In [584]:
pr212_2['인증번호개수'].sum()

2055

In [585]:
pr212_3 = pd.concat([pr212_1, pr212_2], ignore_index=True)
pr212_3

,종류,최대출력_KW,인증번호개수,시트명
0,건설기계,100(N:134ps),1,TIER-3~TIER-4
1,건설기계,105(142ps),1,TIER-3~TIER-4
2,건설기계,106.6(N:137ps),1,TIER-3~TIER-4
3,건설기계,106.6(N:145ps),1,TIER-3~TIER-4
4,건설기계,108(135ps),1,TIER-3~TIER-4
...,...,...,...,...
653,농업기계,215,2,STAGE5
654,농업기계,221,8,STAGE5
655,농업기계,250,2,STAGE5
656,농업기계,NaN,1,STAGE5


In [586]:
pr212_3['인증번호개수'].sum()

8026

In [587]:
pr212_3['테이블생성일자'] = today_date

In [697]:
pr212_3 = pr212_3[[
    '시트명', 
    '종류', 
    '최대출력_KW', 
    '인증번호개수', 
    '테이블생성일자', 
    ]]
# chd212_3 = {
#     '시트명':'', 
#     '종류':'',
#     '최대출력_KW':'',  
#     '인증번호개수':'', 
#     '테이블생성일자':'', 
# }
# pr212_3 = pr212_3.rename(columns=chd212_3)
pr212_3.head()

,시트명,종류,최대출력_KW,인증번호개수,테이블생성일자
0,TIER-3~TIER-4,건설기계,100(N:134ps),1,20231207
1,TIER-3~TIER-4,건설기계,105(142ps),1,20231207
2,TIER-3~TIER-4,건설기계,106.6(N:137ps),1,20231207
3,TIER-3~TIER-4,건설기계,106.6(N:145ps),1,20231207
4,TIER-3~TIER-4,건설기계,108(135ps),1,20231207


### [출력] STD_BD212_OTPT_CERT_REGSTR_CERT_NO

In [698]:
STD_BD212_OTPT_CERT_REGSTR_CERT_NO = pr212_3.copy()
STD_BD212_OTPT_CERT_REGSTR_CERT_NO.to_csv(os.path.join(df2_fold, f'STD_BD212_OTPT_CERT_REGSTR_CERT_NO_{today_date}.csv'), index=False)
STD_BD212_OTPT_CERT_REGSTR_CERT_NO.to_excel(os.path.join(df2_fold, f'STD_BD212_OTPT_CERT_REGSTR_CERT_NO_{today_date}.xlsx'), index=False)

## 2-13 STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO
- 제작사별 인증대장 인증번호 통계

In [590]:
cert_ld34.shape, len(cert_ld34['인증번호'].unique())

((5971, 64), 368)

In [591]:
cert_ld34['인증번호'].notnull().sum()

5971

In [592]:
cert_ld5.shape, len(cert_ld5['인증번호'].unique())

((2055, 74), 131)

In [593]:
cert_ld5['인증번호'].notnull().sum()

2055

In [594]:
pr213_1 = cert_ld34.groupby(['종류', '엔진제작사'], dropna=False)['인증번호'].count().reset_index()
pr213_1 = pr213_1.rename(columns={'인증번호':'인증번호개수'})
pr213_1['시트명'] = 'TIER-3~TIER-4'
pr213_1.head()

,종류,엔진제작사,인증번호개수,시트명
0,건설기계,CATERPILLAR,337,TIER-3~TIER-4
1,건설기계,Daimler AG,52,TIER-3~TIER-4
2,건설기계,Deutz AG,148,TIER-3~TIER-4
3,건설기계,FPT Industrial S.p.A,56,TIER-3~TIER-4
4,건설기계,IHI 에그리텍,10,TIER-3~TIER-4


In [595]:
pr213_1['인증번호개수'].sum()

5971

In [596]:
pr213_2 = cert_ld5.groupby(['종류', '엔진제작사'], dropna=False)['인증번호'].count().reset_index()
pr213_2 = pr213_2.rename(columns={'인증번호':'인증번호개수'})
pr213_2['시트명'] = 'STAGE5'
pr213_2.head()

,종류,엔진제작사,인증번호개수,시트명
0,건설기계,(주)대동,23,STAGE5
1,건설기계,FPT Industrial S.p.A,50,STAGE5
2,건설기계,구보다,274,STAGE5
3,건설기계,도이츠,53,STAGE5
4,건설기계,립헬,16,STAGE5


In [597]:
pr213_2['인증번호개수'].sum()

2055

In [598]:
pr213_3 = pd.concat([pr213_1, pr213_2], ignore_index=True)
pr213_3

,종류,엔진제작사,인증번호개수,시트명
0,건설기계,CATERPILLAR,337,TIER-3~TIER-4
1,건설기계,Daimler AG,52,TIER-3~TIER-4
2,건설기계,Deutz AG,148,TIER-3~TIER-4
3,건설기계,FPT Industrial S.p.A,56,TIER-3~TIER-4
4,건설기계,IHI 에그리텍,10,TIER-3~TIER-4
...,...,...,...,...
81,농업기계,엘에스엠트론,22,STAGE5
82,농업기계,죤디어,80,STAGE5
83,농업기계,콜러/롬바르디니,11,STAGE5
84,농업기계,티와이엠,63,STAGE5


In [599]:
pr213_3['인증번호개수'].sum()

8026

In [600]:
pr213_3['테이블생성일자'] = today_date

In [699]:
pr213_3 = pr213_3[[
    '시트명', 
    '종류', 
    '엔진제작사', 
    '인증번호개수', 
    '테이블생성일자', 
    ]]
# chd213_3 = {
#     '시트명':'', 
#     '종류':'',
#     '엔진제작사':'',  
#     '인증번호개수':'', 
#     '테이블생성일자':'', 
# }
# pr213_3 = pr213_3.rename(columns=chd213_3)
pr213_3.head()

,시트명,종류,엔진제작사,인증번호개수,테이블생성일자
0,TIER-3~TIER-4,건설기계,CATERPILLAR,337,20231207
1,TIER-3~TIER-4,건설기계,Daimler AG,52,20231207
2,TIER-3~TIER-4,건설기계,Deutz AG,148,20231207
3,TIER-3~TIER-4,건설기계,FPT Industrial S.p.A,56,20231207
4,TIER-3~TIER-4,건설기계,IHI 에그리텍,10,20231207


### [출력] STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO

In [700]:
STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO = pr213_3.copy()
STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO.to_csv(os.path.join(df2_fold, f'STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO_{today_date}.csv'), index=False)
STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO.to_excel(os.path.join(df2_fold, f'STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO_{today_date}.xlsx'), index=False)

## 2-14 STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO
- 제작국별 인증대장 인증번호 통계

In [603]:
cert_ld34.shape, len(cert_ld34['인증번호'].unique())

((5971, 64), 368)

In [604]:
cert_ld34['인증번호'].notnull().sum()

5971

In [605]:
cert_ld5.shape, len(cert_ld5['인증번호'].unique())

((2055, 74), 131)

In [606]:
cert_ld5['인증번호'].notnull().sum()

2055

In [607]:
pr214_1 = cert_ld34.groupby(['종류', '엔진제작국'], dropna=False)['인증번호'].count().reset_index()
pr214_1 = pr214_1.rename(columns={'인증번호':'인증번호개수'})
pr214_1['시트명'] = 'TIER-3~TIER-4'
pr214_1.head()

,종류,엔진제작국,인증번호개수,시트명
0,건설기계,독일,285,TIER-3~TIER-4
1,건설기계,미국,1017,TIER-3~TIER-4
2,건설기계,스웨덴,388,TIER-3~TIER-4
3,건설기계,영국,1173,TIER-3~TIER-4
4,건설기계,이탈리아,16,TIER-3~TIER-4


In [608]:
pr214_1['인증번호개수'].sum()

5971

In [609]:
pr214_2 = cert_ld5.groupby(['종류', '엔진제작국'], dropna=False)['인증번호'].count().reset_index()
pr214_2 = pr214_2.rename(columns={'인증번호':'인증번호개수'})
pr214_2['시트명'] = 'STAGE5'
pr214_2.head()

,종류,엔진제작국,인증번호개수,시트명
0,건설기계,독일,53,STAGE5
1,건설기계,미국,179,STAGE5
2,건설기계,스웨덴,81,STAGE5
3,건설기계,스위스,16,STAGE5
4,건설기계,영국,320,STAGE5


In [610]:
pr214_2['인증번호개수'].sum()

2055

In [611]:
pr214_3 = pd.concat([pr214_1, pr214_2], ignore_index=True)
pr214_3

,종류,엔진제작국,인증번호개수,시트명
0,건설기계,독일,285,TIER-3~TIER-4
1,건설기계,미국,1017,TIER-3~TIER-4
2,건설기계,스웨덴,388,TIER-3~TIER-4
3,건설기계,영국,1173,TIER-3~TIER-4
4,건설기계,이탈리아,16,TIER-3~TIER-4
5,건설기계,이태리,84,TIER-3~TIER-4
6,건설기계,일본,833,TIER-3~TIER-4
7,건설기계,중국,89,TIER-3~TIER-4
8,건설기계,프랑스,9,TIER-3~TIER-4
9,건설기계,한국,625,TIER-3~TIER-4


In [612]:
pr214_3['인증번호개수'].sum()

8026

In [613]:
pr214_3['테이블생성일자'] = today_date

In [701]:
pr214_3 = pr214_3[[
    '시트명', 
    '종류', 
    '엔진제작국', 
    '인증번호개수', 
    '테이블생성일자', 
    ]]
# chd214_3 = {
#     '시트명':'', 
#     '종류':'',
#     '엔진제작국':'',  
#     '인증번호개수':'', 
#     '테이블생성일자':'', 
# }
# pr214_3 = pr214_3.rename(columns=chd214_3)
pr214_3.head()

,시트명,종류,엔진제작국,인증번호개수,테이블생성일자
0,TIER-3~TIER-4,건설기계,독일,285,20231207
1,TIER-3~TIER-4,건설기계,미국,1017,20231207
2,TIER-3~TIER-4,건설기계,스웨덴,388,20231207
3,TIER-3~TIER-4,건설기계,영국,1173,20231207
4,TIER-3~TIER-4,건설기계,이탈리아,16,20231207


### [출력] STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO

In [702]:
STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO = pr214_3.copy()
STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO.to_csv(os.path.join(df2_fold, f'STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO_{today_date}.csv'), index=False)
STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO.to_excel(os.path.join(df2_fold, f'STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO_{today_date}.xlsx'), index=False)

# 과제4

## 로드

### 등록정보

In [ ]:
# test kbj
# test je
# 5m 31.4s
car = wd.export_to_pandas("SELECT VIN, VHCL_ERSR_YN, FRST_REG_YMD, MANP_MNG_NO, BSPL_STDG_CD, EXHST_GAS_GRD_CD, VHCTY_CD FROM STD_CEG_CAR_MIG;")
car_ch_col = {
    'VIN':'차대번호', 
    'VHCL_ERSR_YN':'차량말소여부',
    'FRST_REG_YMD':'최초등록일자',
    'MANP_MNG_NO':'제원관리번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'VHCTY_CD':'차종', 

    # 'EXHST_GAS_CERT_NO':'배출가스인증번호',
    # 'YRIDNW':'차량연식', 
    # 'PURPS_CD2':'용도', 
    # 'VHCL_FBCTN_YMD':'제작일자', 
    # 'VHCL_MNG_NO':'차량관리번호', 
    # 'VHRNO':'자동차등록번호',
    # 'EXTGAS_INSP_VLD_YMD':'검사유효일',
    # 'VHCL_OWNR_CL_CD':'소유자구분',  
}
carr = car.rename(columns=car_ch_col)

print(car.shape)
print(carr.columns)
print('data load : STD_CEG_CAR_MIG')

### 제원정보

In [ ]:
# 4.1s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, MNFCTR_NM FROM STD_CEG_CAR_SRC_MIG;")
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'MNFCTR_NM':'제작사명', 
    
    # 'VHCTY_TY_CD2':'차종유형', 
    # 'VHCNM':'차명', 
    # 'VHCL_FRM':'자동차형식', 
    # 'EGIN_TY':'엔진형식', 
    # 'VHCTY_CL_CD':'차종분류',
    # 'TOTL_WGHT':'총중량',
    # 'CRYNG_WGHT':'적재중량',
    # 'DSPLVL':'배기량', 
    # 'EGIN_OTPT':'엔진출력',
}
srcr = src.rename(columns=src_ch_col)

print(srcr.shape)
print(srcr.columns)
print('data load : STD_CEG_CAR_SRC_MIG')

### 정기검사

In [ ]:
# 3m 34.9s
# jgt = wb.export_to_pandas("SELECT VIN, FDRM_INSP_YMD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_DRVNG_DSTNC FROM STD_TB_JGT_HIS WHERE ROWNUM <= 10000;") # 테스트용
jgt = wd.export_to_pandas("SELECT VIN, FDRM_INSP_YMD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_DRVNG_DSTNC FROM STD_TB_JGT_HIS;")
jgt_ch_col = {
    'VIN':'차대번호', 
    'FDRM_INSP_YMD':'검사일자',
    'FDRM_INSP_JGMT':'검사판정',
    'FDRM_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'FDRM_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'FDRM_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'FDRM_DRVNG_DSTNC':'주행거리',

    # 'FDRM_INSP_INSP_MTHD_CD':'검사방법', 
    # 'FDRM_INSP_KND_CD':'검사종류', 
    # 'FDRM_NLOD_SMO_MSTVL4':'무부하매연측정치4', 
    # 'FDRM_NLOD_SMO_MSTVL5':'무부하매연측정치5', 
    # 'FDRM_NLOD_SMO_MSTVL6':'무부하매연측정치6', 
    # 'FDRM_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    # 'FDRM_NLOD_SMO_JT_YN2':'무부하매연판정2', 
    # 'FDRM_NLOD_SMO_JT_YN3':'무부하매연판정3', 
    # 'FDRM_NLOD_SMO_JT_YN4':'무부하매연판정4', 
    # 'FDRM_NLOD_SMO_JT_YN5':'무부하매연판정5', 
    # 'FDRM_NLOD_SMO_JT_YN6':'무부하매연판정6', 
    # 'FDRM_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
    # 'FDRM_NLOD_SMO_PRMVL2':'무부하매연허용치2', 
    # 'FDRM_NLOD_SMO_PRMVL3':'무부하매연허용치3', 
    # 'FDRM_NLOD_SMO_PRMVL4':'무부하매연허용치4', 
    # 'FDRM_NLOD_SMO_PRMVL5':'무부하매연허용치5', 
    # 'FDRM_NLOD_SMO_PRMVL6':'무부하매연허용치6', 
}
jgtr = jgt.rename(columns=jgt_ch_col)

print(jgtr.shape)
print(jgtr.columns)
print('data load : STD_CEG_CAR_SRC_MIG')

### 정밀검사

In [ ]:
# 10m 30.8s
# eet = wb.export_to_pandas("SELECT VIN, PRCINSP_YMD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PRCINSP_DRVNG_DSTNC FROM STD_TB_EET_HIS_ME_ME WHERE ROWNUM <= 10000;") # 테스트용
eet = wd.export_to_pandas("SELECT VIN, PRCINSP_YMD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PRCINSP_DRVNG_DSTNC FROM STD_TB_EET_HIS_ME;")
eet_ch_col = {
    'VIN':'차대번호', 
    'PRCINSP_YMD':'검사일자',
    'PRCINSP_JGMT':'검사판정', 
    'PREC_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'PREC_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'PREC_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'PRCINSP_DRVNG_DSTNC':'주행거리', 

    # 'PRCINSP_MSRMT_MTHD_CD':'검사방법', 
    # 'PRCINSP_KND_CD':'검사종류', 
    # 'PREC_NLOD_SMO_MSTVL4':'무부하매연측정치4', 
    # 'PREC_NLOD_SMO_MSTVL5':'무부하매연측정치5', 
    # 'PREC_NLOD_SMO_MSTVL6':'무부하매연측정치6', 
    # 'PREC_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    # 'PREC_NLOD_SMO_JT_YN2':'무부하매연판정2', 
    # 'PREC_NLOD_SMO_JT_YN3':'무부하매연판정3', 
    # 'PREC_NLOD_SMO_JT_YN4':'무부하매연판정4', 
    # 'PREC_NLOD_SMO_JT_YN5':'무부하매연판정5', 
    # 'PREC_NLOD_SMO_JT_YN6':'무부하매연판정6', 
    # 'PREC_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
    # 'PREC_NLOD_SMO_PRMVL2':'무부하매연허용치2', 
    # 'PREC_NLOD_SMO_PRMVL3':'무부하매연허용치3', 
    # 'PREC_NLOD_SMO_PRMVL4':'무부하매연허용치4', 
    # 'PREC_NLOD_SMO_PRMVL5':'무부하매연허용치5', 
    # 'PREC_NLOD_SMO_PRMVL6':'무부하매연허용치6', 
}
eetr = eet.rename(columns=eet_ch_col)

print(eetr.shape)
print(eetr.columns)
print('data load : STD_TB_EET_HIS_ME')

### 법정동코드

In [ ]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구', 
    # 'STDG_CTPV_CD':'시도코드',
    # 'STDG_SGG_CD':'시군구코드', 
}
coder = code.rename(columns=code_ch_col)

print(coder.shape)
print(coder.columns)
print('data load : STD_BJCD_INFO')

### 1\. 전문정비(STD_TB_ERR_RES)

In [133]:
er_dict = {
    'DRVNG_DSTNC':'주행거리', 
    'VHRNO':'자동차등록번호', 
    'VIN':'차대번호', 
    'LOD_COX_MEVLU':'부하일산화탄소측정치', 
    'LOD_HC_MEVLU':'부하탄화수소측정치', 
    'LOD_NOX_MEVLU':'부하질소산화물측정치', 
    'LOD_SMO_MEVLU':'부하매연측정치', 
    'LOD_TQ_MEVLU':'부하토크측정치', 
    'OVHUL_YMD':'정비일자', 
    'SPCY_OVHUL_BIZPLC_NM':'전문정비사업소명', 
    'OVHUL_RCPT_NO':'정비접수번호', 
    'PRCINSP_RINSP_RCPT_NO':'정밀재검사접수번호', 
    'NOLOD_COX_MEVLU':'무부하일산화탄소측정치', 
    'NOLOD_HC_MEVLU':'무부하탄화수소측정치', 
    'NOLOD_AIR_EXCRT_MEVLU':'무부하공기과잉률측정치', 
    'NOLOD_SMO_MEVLU':'무부하매연측정치', 
}

In [134]:
# 0.3s
name = 'STD_TB_ERR_RES(20231011)'
file_name = f'{name}.csv'
er = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False)
er.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54617 entries, 0 to 54616
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DRVNG_DSTNC            54617 non-null  float64
 1   VHRNO                  54617 non-null  object 
 2   VIN                    51051 non-null  object 
 3   LOD_COX_MEVLU          53884 non-null  object 
 4   LOD_HC_MEVLU           53886 non-null  object 
 5   LOD_NOX_MEVLU          53883 non-null  object 
 6   LOD_SMO_MEVLU          53941 non-null  object 
 7   LOD_TQ_MEVLU           53819 non-null  object 
 8   OVHUL_YMD              54617 non-null  int64  
 9   SPCY_OVHUL_BIZPLC_NM   54617 non-null  object 
 10  OVHUL_RCPT_NO          54617 non-null  int64  
 11  PRCINSP_RINSP_RCPT_NO  54617 non-null  object 
 12  NOLOD_COX_MEVLU        53870 non-null  object 
 13  NOLOD_HC_MEVLU         53871 non-null  object 
 14  NOLOD_AIR_EXCRT_MEVLU  28484 non-null  object 
 15  NO

In [135]:
er = er.rename(columns=er_dict)
er.columns

Index(['주행거리', '자동차등록번호', '차대번호', '부하일산화탄소측정치', '부하탄화수소측정치', '부하질소산화물측정치',
       '부하매연측정치', '부하토크측정치', '정비일자', '전문정비사업소명', '정비접수번호', '정밀재검사접수번호',
       '무부하일산화탄소측정치', '무부하탄화수소측정치', '무부하공기과잉률측정치', '무부하매연측정치'],
      dtype='object')

#### 데이터 확인

In [136]:
set(er['정비접수번호'].unique())

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}

##### [출력] 컬럼, 데이터 정보들

In [137]:
er_cols = pd.DataFrame({'컬럼명':er_dict.keys(), '컬럼내용':er_dict.values()})
er_cols

,컬럼명,컬럼내용
0,DRVNG_DSTNC,주행거리
1,VHRNO,자동차등록번호
2,VIN,차대번호
3,LOD_COX_MEVLU,부하일산화탄소측정치
4,LOD_HC_MEVLU,부하탄화수소측정치
5,LOD_NOX_MEVLU,부하질소산화물측정치
6,LOD_SMO_MEVLU,부하매연측정치
7,LOD_TQ_MEVLU,부하토크측정치
8,OVHUL_YMD,정비일자
9,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명


In [138]:
temp = er.isnull().sum().reset_index()
temp = temp.rename(columns={'index':'컬럼내용', 0:'빈 값 개수'})
temp['총 행수'] = er.shape[0]
temp['번호'] = [x for x in range(1, temp.shape[0] + 1)]
ttemp = temp.merge(er_cols, on='컬럼내용', how='left')
ttemp = ttemp[['번호', '컬럼명', '컬럼내용', '빈 값 개수', '총 행수']]
ttemp

,번호,컬럼명,컬럼내용,빈 값 개수,총 행수
0,1,DRVNG_DSTNC,주행거리,0,54617
1,2,VHRNO,자동차등록번호,0,54617
2,3,VIN,차대번호,3566,54617
3,4,LOD_COX_MEVLU,부하일산화탄소측정치,733,54617
4,5,LOD_HC_MEVLU,부하탄화수소측정치,731,54617
5,6,LOD_NOX_MEVLU,부하질소산화물측정치,734,54617
6,7,LOD_SMO_MEVLU,부하매연측정치,676,54617
7,8,LOD_TQ_MEVLU,부하토크측정치,798,54617
8,9,OVHUL_YMD,정비일자,0,54617
9,10,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명,0,54617


In [139]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.11.14'

In [140]:
# ttemp.to_excel(os.path.join(an4_fold, f'전문정비_데이터정보({today_date}).xlsx'), index=False)

#### 데이터 수정

In [141]:
er['정비일자'].isnull().sum()

0

In [142]:
er['정비일자'].describe()

count      54617.00
mean    20083896.95
std        16435.80
min     20060727.00
25%     20070801.00
50%     20080507.00
75%     20091109.00
max     20121010.00
Name: 정비일자, dtype: float64

In [143]:
er['정비일자'] = pd.to_datetime(er['정비일자'], format='%Y%m%d', errors='coerce')
er['정비일자'].head()

0   2008-01-10
1   2008-01-23
2   2008-01-08
3   2008-01-18
4   2008-01-07
Name: 정비일자, dtype: datetime64[ns]

In [144]:
er['정비일자'].describe()

count                   54617
unique                   1839
top       2007-10-02 00:00:00
freq                      123
first     2006-07-27 00:00:00
last      2012-10-10 00:00:00
Name: 정비일자, dtype: object

### 2\. 전문정비상세(STD_TB_ERR_RES_DET)

In [193]:
erd_dict = {
    'VHRNO':'자동차등록번호', 
    'VIN':'차대번호', 
    'OVHUL_CN':'정비내용', 
    'OVHUL_YMD':'정비일자', 
    'SPCY_OVHUL_BIZPLC_NM':'전문정비사업소명', 
    'OVHUL_CMPNT_NM':'정비부품명', 
    'OVHUL_RCPT_NO':'정비접수번호', 
    'PRCINSP_RINSP_RCPT_NO':'정밀재검사접수번호', 
}

In [196]:
# 0.3s
name = 'STD_TB_ERR_RES_DET_20231011'
file_name = f'{name}.csv'
erd = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False)
erd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106819 entries, 0 to 106818
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VHRNO                  106818 non-null  object 
 1   VIN                    94670 non-null   object 
 2   OVHUL_CN               106817 non-null  object 
 3   OVHUL_YMD              106818 non-null  float64
 4   SPCY_OVHUL_BIZPLC_NM   106818 non-null  object 
 5   OVHUL_CMPNT_NM         106817 non-null  object 
 6   OVHUL_RCPT_NO          106818 non-null  float64
 7   PRCINSP_RINSP_RCPT_NO  0 non-null       float64
dtypes: float64(3), object(5)
memory usage: 6.5+ MB


In [197]:
erd = erd.rename(columns=erd_dict)
erd.columns

Index(['자동차등록번호', '차대번호', '정비내용', '정비일자', '전문정비사업소명', '정비부품명', '정비접수번호',
       '정밀재검사접수번호'],
      dtype='object')

In [198]:
erd['정비내용'].unique()

array(['연료분사펌프(조속기조정\x7f봉인)', '엔진오일', '공회전속도', '연료휠터', '노즐(디젤인젝터포함)',
       '공연비(혼합비)', '에어크리너', '연료압력조절기', '엔진정비', '냉각계통', 'EGR제어용 서머밸브',
       '과급기', '점화시기(분사시기) 조정', 'EGR밸브', '디젤산화촉매(DOC)',
       '정화조절밸브(Purge Control Valve)', '배전기(Distributor)', '냉각수온센서',
       '산소센서', '기화기(믹서포함)', '정화용촉매(Catalytic Converter)', '공기유량센서',
       '전자제어장치(ECU)', '블로바이가스 호스', '캐니스터 및 필터', '점화플러그', '점화배선', '배기관',
       '드로틀바디', '인젝터(휘발유)', 'PCV밸브', '소음기', '드로틀포지션센서', '매연여과장치(DPF)',
       nan], dtype=object)

In [199]:
erd['정비부품명'].unique()

array(['연료장치', '엔진계통', '흡기계통', '배출가스 재순환장치', '점화계통', '배출가스 전환장치',
       '연료증발가스 방지장치', '각종센서', '블로바이가스 환원장치', '배기계통', nan], dtype=object)

#### 데이터 확인

In [148]:
erd.shape

(106819, 8)

In [149]:
set(erd['정비접수번호'].unique())

{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, nan}

##### [출력] 컬럼, 데이터 정보들

In [150]:
erd_cols = pd.DataFrame({'컬럼명':erd_dict.keys(), '컬럼내용':erd_dict.values()})
erd_cols

,컬럼명,컬럼내용
0,VHRNO,자동차등록번호
1,VIN,차대번호
2,OVHUL_CN,정비내용
3,OVHUL_YMD,정비일자
4,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명
5,OVHUL_CMPNT_NM,정비부품명
6,OVHUL_RCPT_NO,정비접수번호
7,PRCINSP_RINSP_RCPT_NO,정밀재검사접수번호


In [151]:
temp = erd.isnull().sum().reset_index()
temp = temp.rename(columns={'index':'컬럼내용', 0:'빈 값 개수'})
temp['총 행수'] = erd.shape[0]
temp['번호'] = [x for x in range(1, temp.shape[0] + 1)]
ttemp = temp.merge(erd_cols, on='컬럼내용', how='left')
ttemp = ttemp[['번호', '컬럼명', '컬럼내용', '빈 값 개수', '총 행수']]
ttemp

,번호,컬럼명,컬럼내용,빈 값 개수,총 행수
0,1,VHRNO,자동차등록번호,1,106819
1,2,VIN,차대번호,12149,106819
2,3,OVHUL_CN,정비내용,2,106819
3,4,OVHUL_YMD,정비일자,1,106819
4,5,SPCY_OVHUL_BIZPLC_NM,전문정비사업소명,1,106819
5,6,OVHUL_CMPNT_NM,정비부품명,2,106819
6,7,OVHUL_RCPT_NO,정비접수번호,1,106819
7,8,PRCINSP_RINSP_RCPT_NO,정밀재검사접수번호,106819,106819


In [152]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.11.14'

In [153]:
# ttemp.to_excel(os.path.join(an4_fold, f'전문정비상세_데이터정보({today_date}).xlsx'), index=False)

#### 데이터 수정

In [154]:
erd['정비일자'].isnull().sum()

1

In [155]:
erd['정비일자'].describe()

count     106818.00
mean    20083911.85
std        15852.89
min     20060727.00
25%     20070829.00
50%     20080529.00
75%     20090921.00
max     20120928.00
Name: 정비일자, dtype: float64

In [156]:
erd['정비일자'] = pd.to_datetime(erd['정비일자'], format='%Y%m%d', errors='coerce')
erd['정비일자'].head()

0   2007-02-08
1   2006-08-11
2   2006-08-11
3   2006-08-11
4   2006-12-11
Name: 정비일자, dtype: datetime64[ns]

In [157]:
erd['정비일자'].describe()

count                  106818
unique                   1832
top       2007-08-13 00:00:00
freq                      232
first     2006-07-27 00:00:00
last      2012-09-28 00:00:00
Name: 정비일자, dtype: object

### 전문정비 추가 데이터

#### 데이터 확인

In [35]:
os.listdir(raw4_fold)

['tb_rsd_ge_list_202311071035.csv',
 'tb_rsd_ge_list_ex_202311071035.csv',
 'tb_rsd_ge_sub_202311071035.csv',
 'tb_rse_det_ex_202312061021.csv',
 'tb_rse_out_result_202311071036.csv',
 'tb_rse_reg_202311071036.csv',
 'tb_rse_std_202311071036.csv',
 'tb_rsm_org_202311071037.csv',
 'tb_rso_reg_202311071037.csv',
 'tb_us_org_202311071037.csv',
 '전문정비내역수정_20231208.csv']

In [159]:
len(os.listdir(raw4_fold))

9

In [160]:
print('tb_rsd_ge_list'.upper())
print('tb_rsd_ge_list_ex'.upper())
print('tb_rsd_ge_sub'.upper())
print('tb_rse_out_result'.upper())
print('tb_rse_reg'.upper())
print('tb_rse_std'.upper())
print('tb_rsm_org'.upper())
print('tb_rso_reg'.upper())
print('tb_us_org'.upper())

TB_RSD_GE_LIST
TB_RSD_GE_LIST_EX
TB_RSD_GE_SUB
TB_RSE_OUT_RESULT
TB_RSE_REG
TB_RSE_STD
TB_RSM_ORG
TB_RSO_REG
TB_US_ORG


In [161]:
raw4_fold

'D:/data/processing/big3/raw/BD4'

In [162]:
# 0.3s
name = 'tb_rsd_ge_list_202311071035'
file_name = f'{name}.csv'
rsd_ge_list = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rsd_ge_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71065 entries, 0 to 71064
Data columns (total 55 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ge_idx             71065 non-null  int64  
 1   rsd_month          71065 non-null  int64  
 2   car_ser_no         71065 non-null  object 
 3   snt                61457 non-null  float64
 4   ge_cnt             71065 non-null  int64  
 5   search1            11799 non-null  float64
 6   search2            2351 non-null   float64
 7   search3            52534 non-null  float64
 8   search4            61061 non-null  float64
 9   car_reg_no         71065 non-null  object 
 10  car_nm             71065 non-null  object 
 11  usage              71065 non-null  object 
 12  car_year           71065 non-null  int64  
 13  car_type_nm        71065 non-null  object 
 14  car_weight         71065 non-null  int64  
 15  tot_weight         71065 non-null  int64  
 16  engine_capacity    710

In [163]:
stat1 = rsd_ge_list.isnull().sum().reset_index()
stat1 = stat1.rename(columns={'index':'컬럼명', 0:'결측치 개수'})
stat1['총 행수'] = rsd_ge_list.shape[0]
stat1['번호'] = [x for x in range(1, stat1.shape[0] + 1)]
stat1 = stat1[['번호', '컬럼명', '결측치 개수', '총 행수']]
stat1

,번호,컬럼명,결측치 개수,총 행수
0,1,ge_idx,0,71065
1,2,rsd_month,0,71065
2,3,car_ser_no,0,71065
3,4,snt,9608,71065
4,5,ge_cnt,0,71065
5,6,search1,59266,71065
6,7,search2,68714,71065
7,8,search3,18531,71065
8,9,search4,10004,71065
9,10,car_reg_no,0,71065


In [164]:
for file_name in os.listdir(raw4_fold):
    print(file_name)

tb_rsd_ge_list_202311071035.csv
tb_rsd_ge_list_ex_202311071035.csv
tb_rsd_ge_sub_202311071035.csv
tb_rse_out_result_202311071036.csv
tb_rse_reg_202311071036.csv
tb_rse_std_202311071036.csv
tb_rsm_org_202311071037.csv
tb_rso_reg_202311071037.csv
tb_us_org_202311071037.csv


In [165]:
# 12.8s
for file_name in os.listdir(raw4_fold):
    name = file_name.split('.')[0]
    df = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
    print(f'{name} : {df.shape}')

tb_rsd_ge_list_202311071035 : (71065, 55)
tb_rsd_ge_list_ex_202311071035 : (119898, 70)


tb_rsd_ge_sub_202311071035 : (469327, 69)
tb_rse_out_result_202311071036 : (3640, 53)
tb_rse_reg_202311071036 : (4140, 26)
tb_rse_std_202311071036 : (3410, 43)
tb_rsm_org_202311071037 : (652, 42)
tb_rso_reg_202311071037 : (4192, 37)
tb_us_org_202311071037 : (15711, 21)


#### [출력] 컬럼, 데이터 정보들

In [166]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.11.14'

In [167]:
# # 12.8s
# with pd.ExcelWriter(os.path.join(an4_fold, f'건설기계_추가데이터_현황({today_date}).xlsx')) as writer:
#     for file_name in os.listdir(raw4_fold):
#         name = file_name.split('.')[0]
#         df = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
#         temp = df.isnull().sum().reset_index()
#         temp = temp.rename(columns={'index':'컬럼명', 0:'결측치 개수'})
#         temp['총 행수'] = df.shape[0]
#         temp['번호'] = [x for x in range(1, temp.shape[0] + 1)]
#         temp = temp[['번호', '컬럼명', '결측치 개수', '총 행수']]
#         temp.to_excel(writer, sheet_name=name, index=False)

### 3\. TB_RSD_GE_LIST

In [ ]:
os.listdir(raw4_fold)

['tb_rsd_ge_list_202311071035.csv',
 'tb_rsd_ge_list_ex_202311071035.csv',
 'tb_rsd_ge_sub_202311071035.csv',
 'tb_rse_out_result_202311071036.csv',
 'tb_rse_reg_202311071036.csv',
 'tb_rse_std_202311071036.csv',
 'tb_rsm_org_202311071037.csv',
 'tb_rso_reg_202311071037.csv',
 'tb_us_org_202311071037.csv']

In [196]:
# 0.3s
name = 'tb_rsd_ge_list_202311071035'
file_name = f'{name}.csv'
rsd_ge_list = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rsd_ge_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71065 entries, 0 to 71064
Data columns (total 55 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ge_idx             71065 non-null  int64  
 1   rsd_month          71065 non-null  int64  
 2   car_ser_no         71065 non-null  object 
 3   snt                61457 non-null  float64
 4   ge_cnt             71065 non-null  int64  
 5   search1            11799 non-null  float64
 6   search2            2351 non-null   float64
 7   search3            52534 non-null  float64
 8   search4            61061 non-null  float64
 9   car_reg_no         71065 non-null  object 
 10  car_nm             71065 non-null  object 
 11  usage              71065 non-null  object 
 12  car_year           71065 non-null  int64  
 13  car_type_nm        71065 non-null  object 
 14  car_weight         71065 non-null  int64  
 15  tot_weight         71065 non-null  int64  
 16  engine_capacity    710

In [172]:
rsd_ge_list.columns

Index(['ge_idx', 'rsd_month', 'car_ser_no', 'snt', 'ge_cnt', 'search1',
       'search2', 'search3', 'search4', 'car_reg_no', 'car_nm', 'usage',
       'car_year', 'car_type_nm', 'car_weight', 'tot_weight',
       'engine_capacity', 'engine_power', 'car_fuel_nm', 'first_reg_date',
       'smo_chk_exp_date', 'chk_ge_fr', 'chk_ge_to', 'rso_reg_status',
       'make_reg_date', 'car_fuel_cd', 'car_type_cd', 'reg_region', 'sido',
       'sigungu', 'usage_cd', 'usage_ex', 'ge_reg_type', 'last_chk_exp_date',
       'car_type_nm2', 'car_maker_nm', 'engine_type', 'mileage', 'road_cont',
       'road_type', 'road_money', 'road_reason', 'conf_yn', 'conf_flag',
       'selection_date', 'selection_user', 'confirm_date', 'confirm_user',
       'imps_reason', 'sido_cd', 'sigungu_cd', 'sido_nm', 'sigungu_nm',
       'cncl_yn', 'note'],
      dtype='object')

In [192]:
set(rsd_ge_list['mileage'].unique())

{0.0,
 1.0,
 229377.0,
 229380.0,
 229383.0,
 8.0,
 32777.0,
 10.0,
 98315.0,
 65552.0,
 196629.0,
 196630.0,
 24.0,
 589862.0,
 131122.0,
 262206.0,
 426048.0,
 98370.0,
 393282.0,
 524356.0,
 65607.0,
 163915.0,
 393295.0,
 163924.0,
 131164.0,
 196704.0,
 360550.0,
 98410.0,
 32882.0,
 131197.0,
 262270.0,
 327807.0,
 262269.0,
 458883.0,
 393349.0,
 163980.0,
 229517.0,
 65681.0,
 163987.0,
 360600.0,
 32922.0,
 327837.0,
 229537.0,
 327855.0,
 295088.0,
 229551.0,
 196788.0,
 262324.0,
 65716.0,
 98485.0,
 164024.0,
 98493.0,
 262344.0,
 131273.0,
 262352.0,
 98514.0,
 196819.0,
 262357.0,
 721110.0,
 164058.0,
 98526.0,
 229598.0,
 295141.0,
 33000.0,
 262376.0,
 98539.0,
 393455.0,
 196849.0,
 98554.0,
 327937.0,
 196869.0,
 164105.0,
 131340.0,
 164108.0,
 393492.0,
 295196.0,
 196896.0,
 98595.0,
 164134.0,
 426280.0,
 229674.0,
 655663.0,
 131385.0,
 98619.0,
 360764.0,
 164157.0,
 131390.0,
 131396.0,
 98628.0,
 229702.0,
 164170.0,
 131403.0,
 33100.0,
 557387.0,
 65871.0,


In [193]:
rsd_ge_list['mileage'].describe()

count               9608.00
mean        115644599111.14
std       11335508143510.87
min                    0.00
25%               130623.50
50%               196997.50
75%               278993.25
max     1111111111111111.00
Name: mileage, dtype: float64

### 4\. [✔] TB_RSD_GE_LIST_EX

In [ ]:
os.listdir(raw4_fold)

['tb_rsd_ge_list_202311071035.csv',
 'tb_rsd_ge_list_ex_202311071035.csv',
 'tb_rsd_ge_sub_202311071035.csv',
 'tb_rse_out_result_202311071036.csv',
 'tb_rse_reg_202311071036.csv',
 'tb_rse_std_202311071036.csv',
 'tb_rsm_org_202311071037.csv',
 'tb_rso_reg_202311071037.csv',
 'tb_us_org_202311071037.csv']

In [46]:
# 0.3s
name = 'tb_rsd_ge_list_ex_202311071035'
file_name = f'{name}.csv'
rsd_ge_list_ex = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rsd_ge_list_ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119898 entries, 0 to 119897
Data columns (total 70 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ge_idx                 119898 non-null  int64  
 1   rsd_date               119898 non-null  int64  
 2   car_reg_no             119898 non-null  object 
 3   car_ser_no             119866 non-null  object 
 4   car_nm                 119896 non-null  object 
 5   car_type_cd            62630 non-null   float64
 6   car_year               119896 non-null  object 
 7   usage_cd               62630 non-null   float64
 8   car_fuel_cd            119894 non-null  object 
 9   car_weight             119501 non-null  float64
 10  tot_weight             119495 non-null  float64
 11  engine_capacity        119501 non-null  float64
 12  engine_power           119502 non-null  object 
 13  engine_type            119494 non-null  object 
 14  mileage                119898 non-nu

In [47]:
rsd_ge_list_ex.columns

Index(['ge_idx', 'rsd_date', 'car_reg_no', 'car_ser_no', 'car_nm',
       'car_type_cd', 'car_year', 'usage_cd', 'car_fuel_cd', 'car_weight',
       'tot_weight', 'engine_capacity', 'engine_power', 'engine_type',
       'mileage', 'make_reg_date', 'first_reg_date', 'smo_chk_exp_date',
       'sido', 'sigungu', 'org_id', 'cer_exp_year', 'cer_exp_km', 'mistake_yn',
       'repair_name', 'repair_comment', 'car_age', 'det_yn', 'rec_date',
       'rec_nm', 'las_chg_nm', 'las_chg_date', 'cer_repair_yn', 'reg_region',
       'doc_name', 'reg_no', 'print_yn', 'print_date', 'print_rec_nm',
       'sido_cd', 'sigungu_cd', 'sido_nm', 'sigungu_nm', 'smo_chk_yn',
       'smo_chk_date', 'smo_chk_org_nm', 'smo_chk_method', 'smo_chk_judge_yn',
       'unload_smo_val1', 'unload_smo_lim1', 'unload_smo_judge1',
       'unload_smo_val2', 'unload_smo_lim2', 'unload_smo_judge2',
       'unload_smo_val3', 'unload_smo_lim3', 'unload_smo_judge3',
       'unload_smo_val4', 'unload_smo_lim4', 'unload_smo_judge4'

In [234]:
ge_chc = {
    'ge_idx':'일련번호', 
    'rsd_date':'점검일자', 
    'car_reg_no':'차량번호', 
    'car_ser_no':'차대번호', 
    'car_nm':'차명', 
    'car_type_cd':'차종', 
    'car_year':'연식', 
    'usage_cd':'용도', 
    'car_fuel_cd':'연료', 
    'car_weight':'중량', 
    'tot_weight':'총중량', 
    'engine_capacity':'엔진배기량', 
    'engine_power':'엔진출력', 
    'engine_type':'엔진형식', 
    'mileage':'주행거리', 
    'make_reg_date':'제작일자', 
    'first_reg_date':'최초등록일', 
    'smo_chk_exp_date':'정밀검사만료일자', 
    'sido':'시도', 
    'sigungu':'시군구', 
    'org_id':'전문정비사업자코드', 
    'cer_exp_year':'보증기간_년', 
    'cer_exp_km':'보증기간_Km', 
    'mistake_yn':'보증기간내_고의_또는_과실여부',  
    'repair_name':'정비사성명', 
    'repair_comment':'정비사의견', 
    'car_age':'차령', 
    'det_yn':'정비여부', 
    'rec_date':'생성일자', 
    'rec_nm':'생성자명', 
    'las_chg_nm':'최종수정자명', 
    'las_chg_date':'최종수정일자', 
    'cer_repair_yn':'보증수리여부', 
    'reg_region':'소재지', 
    'doc_name':'소유자명', 
    'reg_no':'접수번호', 
    'print_yn':'출력여부', 
    'print_date':'출력일자', 
    'print_rec_nm':'출력자명', 
    'sido_cd':'시도코드', 
    'sigungu_cd':'시군구코드', 
    'sido_nm':'시도명', 
    'sigungu_nm':'시군구명', 
    'smo_chk_yn':'정밀검사유무', 
    'smo_chk_date':'정밀검사일자', 
    'smo_chk_org_nm':'정밀검사소명', 
    'smo_chk_method':'정밀검사측정방법', 
    'smo_chk_judge_yn':'정밀검사판정', 
    'unload_smo_val1':'무부하매연측정치1', 
    'unload_smo_lim1':'무부하매연허용치1', 
    'unload_smo_judge1':'무부하매연판정1', 
    'unload_smo_val2':'무부하매연측정치2', 
    'unload_smo_lim2':'무부하매연허용치2', 
    'unload_smo_judge2':'무부하매연판정2', 
    'unload_smo_val3':'무부하매연측정치3', 
    'unload_smo_lim3':'무부하매연허용치3', 
    'unload_smo_judge3':'무부하매연판정3', 
    'unload_smo_val4':'무부하매연측정치4', 
    'unload_smo_lim4':'무부하매연허용치4', 
    'unload_smo_judge4':'무부하매연판정4', 
    'unload_smo_val5':'무부하매연측정치5', 
    'unload_smo_lim5':'무부하매연허용치5', 
    'unload_smo_judge5':'무부하매연판정5', 
    'unload_smo_val6':'무부하매연측정치6', 
    'unload_smo_lim6':'무부하매연허용치6', 
    'unload_smo_judge6':'무부하매연판정6', 
    'smo_fun_chk_judge_yn':'관능검사판정', 
    'smo_fun_chk_fail_item':'관능검사부적합내용', 
    'smo_update_date':'정밀검사수정일', 
    'conf_yn':'확정여부', 
}
rsd_ge_list_exr = rsd_ge_list_ex.rename(columns=ge_chc)
rsd_ge_list_exr.columns

Index(['일련번호', '점검일자', '차량번호', '차대번호', '차명', '차종', '연식', '용도', '연료', '중량',
       '총중량', '엔진배기량', '엔진출력', '엔진형식', '주행거리', '제작일자', '최초등록일', '정밀검사만료일자',
       '시도', '시군구', '전문정비사업자코드', '보증기간_년', '보증기간_Km', '보증기간내_고의_또는_과실여부',
       '정비사성명', '정비사의견', '차령', '정비여부', '생성일자', '생성자명', '최종수정자명', '최종수정일자',
       '보증수리여부', '소재지', '소유자명', '접수번호', '출력여부', '출력일자', '출력자명', '시도코드',
       '시군구코드', '시도명', '시군구명', '정밀검사유무', '정밀검사일자', '정밀검사소명', '정밀검사측정방법',
       '정밀검사판정', '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2',
       '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3',
       '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5',
       '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6', '관능검사판정', '관능검사부적합내용',
       '정밀검사수정일', '확정여부'],
      dtype='object')

In [49]:
set(rsd_ge_list_ex['reg_no'].unique())

{'ZD05_20161212_00001',
 'ZL05_20150818_00002',
 'ZA69_20210728_00001',
 'ZL05_20150801_00001',
 'ZN16_20210607_00001',
 'ZA05_20150205_00001',
 'ZI02_20160716_00002',
 'ZF01_20141213_00002',
 'ZI15_20160404_00001',
 'ZN11_20180417_00002',
 'ZL05_20210222_00002',
 '20061030_00004',
 'ZA36_20190617_00001',
 'ZF07_20170202_00001',
 'ZA85_20200207_00001',
 'ZN11_20210403_00001',
 'ZN06_20230717_00001',
 'ZC06_20180315_00001',
 'ZN12_20190329_00001',
 'ZH43_20160803_00002',
 'ZE12_20220126_00001',
 'ZN11_20220516_00002',
 'ZH21_20150402_00001',
 'ZC01_20150825_00001',
 'ZA26_20150203_00001',
 'ZC06_20200215_00001',
 'ZJ03_20141226_00002',
 'ZM10_20201125_00002',
 'ZQ01_20210226_00001',
 'ZA11_20160222_00001',
 'ZA77_20171127_00001',
 'ZL05_20170403_00001',
 'ZA85_20181025_00001',
 'ZA77_20170311_00001',
 '20100930_00001',
 'ZO01_20201222_00001',
 'ZC08_20150723_00001',
 'ZN05_20201118_00002',
 'ZO10_20181027_00001',
 'ZA74_20230530_00001',
 'ZN31_20220121_00001',
 'ZH14_20201105_00001',
 '

In [51]:
rsd_ge_list_ex['car_year'].describe()

count     119896
unique        99
top         2001
freq       14160
Name: car_year, dtype: object

In [52]:
rsd_ge_list_ex.shape, len(rsd_ge_list_ex['ge_idx'].unique())

((119898, 70), 119898)

In [68]:
rsd_ge_list_ex.head()

,ge_idx,rsd_date,car_reg_no,car_ser_no,car_nm,car_type_cd,car_year,usage_cd,car_fuel_cd,car_weight,...,unload_smo_val5,unload_smo_lim5,unload_smo_judge5,unload_smo_val6,unload_smo_lim6,unload_smo_judge6,smo_fun_chk_judge_yn,smo_fun_chk_fail_item,smo_update_date,conf_yn
0,3,20130805,85거9409,KPALE4A811P134760,코란도-밴,331.00,2001,2.00,B,1705.00,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,20170414.00,Y
1,11,20130806,35머4273,KMHMH81VP1U097748,트라제XG,152.00,2001,2.00,B,1845.00,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,20170414.00,Y
2,12,20130806,96도3224,KMFFE17APSU238435,포터초장축더블캡,311.00,1995,2.00,B,1495.00,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,20170414.00,Y
3,15,20130807,85오2664,KMFMA17HPNU008954,현대4.5톤카고트럭-초장축,312.00,1992,2.00,B,4640.00,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,20170414.00,Y
4,16,20130807,33서4306,KMHNN81WP1U012631,테라칸,113.00,2001,2.00,B,2430.00,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,20170414.00,Y


### 5\. TB_RSD_GE_SUB

In [ ]:
os.listdir(raw4_fold)

['tb_rsd_ge_list_202311071035.csv',
 'tb_rsd_ge_list_ex_202311071035.csv',
 'tb_rsd_ge_sub_202311071035.csv',
 'tb_rse_out_result_202311071036.csv',
 'tb_rse_reg_202311071036.csv',
 'tb_rse_std_202311071036.csv',
 'tb_rsm_org_202311071037.csv',
 'tb_rso_reg_202311071037.csv',
 'tb_us_org_202311071037.csv']

In [216]:
# 0.3s
name = 'tb_rsd_ge_sub_202311071035'
file_name = f'{name}.csv'
rsd_ge_sub = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rsd_ge_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469327 entries, 0 to 469326
Data columns (total 69 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   sub_idx            469327 non-null  int64  
 1   fst                459692 non-null  float64
 2   chk_sts            469272 non-null  object 
 3   ge_chk             26834 non-null   object 
 4   rsd_month          469327 non-null  int64  
 5   rsd_date           469327 non-null  int64  
 6   rsd_time           469325 non-null  object 
 7   sitecode           469326 non-null  object 
 8   car_nm             469327 non-null  object 
 9   usage              469327 non-null  object 
 10  car_year           469327 non-null  int64  
 11  car_type_nm        469327 non-null  object 
 12  car_weight         469327 non-null  int64  
 13  tot_weight         469327 non-null  int64  
 14  engine_capacity    469326 non-null  float64
 15  engine_power       469327 non-null  object 
 16  sp

In [217]:
rsd_ge_sub.columns

Index(['sub_idx', 'fst', 'chk_sts', 'ge_chk', 'rsd_month', 'rsd_date',
       'rsd_time', 'sitecode', 'car_nm', 'usage', 'car_year', 'car_type_nm',
       'car_weight', 'tot_weight', 'engine_capacity', 'engine_power', 'speed',
       'accel', 'vsp', 'uvsmoke', 'irsmoke', 'co', 'ppmhc', 'ppmno', 'co_ex',
       'ppmhc_ex', 'ppmno_ex', 'car_ser_no', 'car_reg_no', 'usage_ex',
       'fuel_ex', 'car_type_ex', 'first_reg_date', 'smo_chk_exp_date',
       'smo_chk_mon', 'chk_ge_fr', 'chk_ge_to', 'f__', 'd__', 'e__',
       'smo_chk_method', 'unload_smo_val1', 'unload_smo_val2',
       'unload_smo_val3', 'unload_smo_val4', 'unload_smo_val5',
       'unload_smo_val6', 'unload_smo_lim1', 'unload_smo_lim2',
       'unload_smo_lim3', 'unload_smo_lim4', 'unload_smo_lim5',
       'unload_smo_lim6', 'make_reg_date', 'file_nm', 'rsd_file_nm',
       'smo_chk_cd', 'std_tag', 'std_tag2', 'imagename', 'ge_reg_type',
       'last_chk_exp_date', 'air', 'air_ex_from', 'air_ex_to', 'ppmhchex',
       'sampl

In [237]:
set(rsd_ge_sub['air_type'].unique())

{0.0, 1.0, 2.0, nan}

In [220]:
rsd_ge_sub['fst'].describe()

count   459692.00
mean         1.51
std          1.63
min          1.00
25%          1.00
50%          1.00
75%          2.00
max         70.00
Name: fst, dtype: float64

### 6\. TB_RSE_OUT_RESULT

In [ ]:
os.listdir(raw4_fold)

['tb_rsd_ge_list_202311071035.csv',
 'tb_rsd_ge_list_ex_202311071035.csv',
 'tb_rsd_ge_sub_202311071035.csv',
 'tb_rse_out_result_202311071036.csv',
 'tb_rse_reg_202311071036.csv',
 'tb_rse_std_202311071036.csv',
 'tb_rsm_org_202311071037.csv',
 'tb_rso_reg_202311071037.csv',
 'tb_us_org_202311071037.csv']

In [238]:
# 0.3s
name = 'tb_rse_out_result_202311071036'
file_name = f'{name}.csv'
rse_out_result = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rse_out_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3640 entries, 0 to 3639
Data columns (total 53 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   rse_idx                   3640 non-null   int64  
 1   rse_date                  3640 non-null   int64  
 2   rse_org_cd                3623 non-null   object 
 3   car_reg_no                3640 non-null   object 
 4   car_ser_no                3640 non-null   object 
 5   cer_exp_year              3263 non-null   object 
 6   cer_exp_km                3263 non-null   float64
 7   cer_repair_yn             3632 non-null   object 
 8   mileage                   3530 non-null   object 
 9   mistake_yn                3632 non-null   object 
 10  repair_name               3262 non-null   object 
 11  repair_comment            3245 non-null   object 
 12  before_smo_val            1788 non-null   float64
 13  before_smo_lim            1789 non-null   float64
 14  before_c

In [239]:
rse_out_result.columns

Index(['rse_idx', 'rse_date', 'rse_org_cd', 'car_reg_no', 'car_ser_no',
       'cer_exp_year', 'cer_exp_km', 'cer_repair_yn', 'mileage', 'mistake_yn',
       'repair_name', 'repair_comment', 'before_smo_val', 'before_smo_lim',
       'before_co_val', 'before_co_lim', 'before_hc_val', 'before_hc_lim',
       'before_nox_val', 'before_nox_lim', 'before_air_over_val',
       'before_air_over_lim', 'after_smo_val1', 'after_smo_lim1',
       'after_co_val1', 'after_co_lim1', 'after_hc_val1', 'after_hc_lim1',
       'after_nox_val1', 'after_nox_lim1', 'after_air_over_val1',
       'after_air_over_lim1_from', 'after_air_over_lim1_to', 'after_smo_val2',
       'after_smo_lim2', 'after_co_val2', 'after_co_lim2', 'after_hc_val2',
       'after_hc_lim2', 'after_nox_val2', 'after_nox_lim2',
       'after_air_over_val2', 'after_air_over_lim2_from',
       'after_air_over_lim2_to', 'rec_date', 'rec_nm', 'las_chg_nm',
       'las_chg_date', 'rse_out_final_yn', 'car_age', 'before_air_over_lim2',
     

In [240]:
set(rse_out_result['rse_org_cd'].unique())

{'113D',
 '113a',
 '113d',
 '1146',
 '1147',
 '114b',
 '116c',
 '118d',
 '119c',
 '11a1',
 '11a2',
 '4165',
 '416b',
 '417j',
 '417l',
 '4182',
 '41hc',
 '41k1',
 '8001',
 '8002',
 '8003',
 'A002',
 'A003',
 'A004',
 'A005',
 'A008',
 'A010',
 'A011',
 'A012',
 'A017',
 'A019',
 'A022',
 'A026',
 'A032',
 'A037',
 'A042',
 'A043',
 'A046',
 'A048',
 'A050',
 'A060',
 'A061',
 'A070',
 'A073',
 'A074',
 'A075',
 'A078',
 'A091',
 'A096',
 'A102',
 'ACAC',
 'B006',
 'B015',
 'B022',
 'B027',
 'B102',
 'C001',
 'C004',
 'C009',
 'C012',
 'C016',
 'C019',
 'C021',
 'C025',
 'C041',
 'C049',
 'C054',
 'C057',
 'C058',
 'C063',
 'C081',
 'C084',
 'D019',
 'D020',
 'D021',
 'D033',
 'D049',
 'D058',
 'D081',
 'D099',
 'E005',
 'E031',
 'E034',
 'E045',
 'E058',
 'E070',
 'F001',
 'F003',
 'F005',
 'F006',
 'F007',
 'F008',
 'F018',
 'F021',
 'F022',
 'F026',
 'F029',
 'F032',
 'F034',
 'G002',
 'G003',
 'G025',
 'G053',
 'H002',
 'H009',
 'H011',
 'H015',
 'H020',
 'H023',
 'H027',
 'H030',
 

In [ ]:
rse_out_result['fst'].describe()

count   459692.00
mean         1.51
std          1.63
min          1.00
25%          1.00
50%          1.00
75%          2.00
max         70.00
Name: fst, dtype: float64

### 7\. [✔] TB_RSE_REG

In [ ]:
os.listdir(raw4_fold)

['tb_rsd_ge_list_202311071035.csv',
 'tb_rsd_ge_list_ex_202311071035.csv',
 'tb_rsd_ge_sub_202311071035.csv',
 'tb_rse_out_result_202311071036.csv',
 'tb_rse_reg_202311071036.csv',
 'tb_rse_std_202311071036.csv',
 'tb_rsm_org_202311071037.csv',
 'tb_rso_reg_202311071037.csv',
 'tb_us_org_202311071037.csv']

In [229]:
# 0.3s
name = 'tb_rse_reg_202311071036'
file_name = f'{name}.csv'
rse_reg = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rse_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4140 entries, 0 to 4139
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rse_idx          4140 non-null   int64  
 1   rso_idx          3769 non-null   float64
 2   car_reg_no       3766 non-null   object 
 3   car_ser_no       3766 non-null   object 
 4   org_id           4140 non-null   object 
 5   org_nm           4133 non-null   object 
 6   reg_no           4140 non-null   int64  
 7   reg_no_full      4140 non-null   object 
 8   rse_date         4140 non-null   int64  
 9   rse_org_cd       4109 non-null   object 
 10  rse_org_nm       4130 non-null   object 
 11  rse_reg_no       4140 non-null   int64  
 12  rse_reg_no_full  4140 non-null   object 
 13  rse_pay_gb       4140 non-null   int64  
 14  rse_how_gb       4140 non-null   object 
 15  rse_etc_gb       4126 non-null   float64
 16  rse_status       4140 non-null   int64  
 17  rec_date      

In [230]:
rse_reg.columns

Index(['rse_idx', 'rso_idx', 'car_reg_no', 'car_ser_no', 'org_id', 'org_nm',
       'reg_no', 'reg_no_full', 'rse_date', 'rse_org_cd', 'rse_org_nm',
       'rse_reg_no', 'rse_reg_no_full', 'rse_pay_gb', 'rse_how_gb',
       'rse_etc_gb', 'rse_status', 'rec_date', 'rec_nm', 'las_chg_date',
       'las_chg_nm', 'det_yn', 'print_yn', 'print_date', 'rse_amt',
       'scale_gb'],
      dtype='object')

In [250]:
reg_chc = {
    'rse_idx':'일련번호_MAX', 
    'rso_idx':'개선명령_IDX', 
    'car_reg_no':'차량번호', 
    'car_ser_no':'차대번호', 
    'org_id':'업체코드_접수업체', 
    'org_nm':'업체명_접수업체', 
    'reg_no':'순번_접수업체', 
    'reg_no_full':'접수번호_접수업체', 
    'rse_date':'접수일자', 
    'rse_org_cd':'업체코드_검사업체', 
    'rse_org_nm':'업체명_검사업체', 
    'rse_reg_no':'순번_검사업체', 
    'rse_reg_no_full':'접수번호_검사업체', 
    'rse_pay_gb':'수납구분', 
    'rse_how_gb':'검사방법', 
    'rse_etc_gb':'특이사항', 
    'rse_status':'확인검사상태', 
    'rec_date':'생성일자', 
    'rec_nm':'생성자명', 
    'las_chg_date':'최종수정일자', 
    'las_chg_nm':'최종수정자명', 
    'det_yn':'정비내역등록여부', 
    'print_yn':'출력여부', 
    'print_date':'출력일시', 
    'rse_amt':'수수료', 
    'scale_gb':'규모구분', 
}
rse_regr = rse_reg.rename(columns=reg_chc)
rse_regr.columns

Index(['일련번호_MAX', '개선명령_IDX', '차량번호', '차대번호', '업체코드_접수업체', '업체명_접수업체',
       '순번_접수업체', '접수번호_접수업체', '접수일자', '업체코드_검사업체', '업체명_검사업체', '순번_검사업체',
       '접수번호_검사업체', '수납구분', '검사방법', '특이사항', '확인검사상태', '생성일자', '생성자명', '최종수정일자',
       '최종수정자명', '정비내역등록여부', '출력여부', '출력일시', '수수료', '규모구분'],
      dtype='object')

In [231]:
set(rse_reg['car_ser_no'].unique())

{'KMHSJ81WP6U083712',
 'KNMA4C2CM8P105325',
 'KNAGD22932A157316',
 'KPBFA2AD13P070630',
 'KPACE4AB17P011428',
 'KMHEN41BPXA206657',
 '000000000000',
 'KMHFU41MP3A273156',
 'KMHEN41BPWA049168',
 'KN92J8MJD3UB1R001',
 'KMHFU41AP4A315897',
 'KMHPM81CP6U271948',
 'KPBFH2AF18P264346',
 'KMFZSZ7JBBU723537',
 'KMHEU41MP8A600317',
 'KMJRD37FPWU410992',
 'KMHEC41LBCA402850',
 'KMFWVH7JP4U634237',
 'KMFZCZ7JAAU564024',
 'KN3HNP6D25K057822',
 'KNFSE0L428K344508',
 'KMHEA415BEA068249',
 'KNAGD22934A365289',
 'KMHEU41MBAA774731',
 'KMHFC41MP7A236594',
 'KMHEU41MBCA828428',
 'KNCWJX74ABK610799',
 'KMHJP81VP5U207506',
 'KMHHP61FP1U001260',
 'KMFZCD7JP4U034520',
 'KMFFD37BPYU476175',
 'KMZKL17DPTU153385',
 'KLDUA75SDYK533955',
 'KNAGN418BDA377011',
 'KNAGE22539A381712',
 'KMFZCS7JP8U425897',
 'KMJHD17HP9C002727',
 'KMFZCN7HP4U019863',
 'KNCSE03429K373520',
 'KMJHA17HPEC019230',
 'KN92J8MJD3UB1R042',
 'KNAJC52184A251502',
 'KLY7B11ZD6C131523',
 'KMHSD81VP1U19613',
 'KNFSTZ74ABK575431',
 'KMHEU41MP7A404

In [232]:
rse_reg['rse_reg_no'].describe()

count   4140.00
mean       1.89
std        3.01
min        1.00
25%        1.00
50%        1.00
75%        2.00
max       38.00
Name: rse_reg_no, dtype: float64

In [233]:
rse_reg['rse_org_nm'].unique()

array(['(주)영등포자동차정비검사소', '비케이프라임산업주식회사', '신흥자동차정비검사(주)', '(주)문래카독크',
       '인영자동차공업사(주)', '남부자동차공업(주)', '(주)대전공업사', '(주)영일정비', '성산검사소-신동성출장',
       '(주)중산자동차정비검사소', '(자)유승기업', '서울상운차량공업(주)', '(주)웅신1급자동차공업사',
       '(주)현대일산자동차정비센타', nan, '(주)금정모터스', '형제자동차공업사', '웅지자동차공업(주)',
       '오신산업(주)', '이글공업사(주)', '에스원자동차정비', '(주)원진엠엔에스', '성동검사소-성우출장',
       '현대동구서비스', '대원자동차공업사', '대명자동차공업(주)-서울', '청원자동차정비(주)',
       '성산검사소-목동한신출장', '한일1급정비공장', '(주)명진자동차', '현대남대구써비스', '(자)세기자동차',
       '(주)영남중공업', '(유)고려한일자동차공업사', '정원자동차정비공업사', '고양검사소-에이스출장',
       '미진자동차공업(주)', '(주) 효성자동차공업', '(주)신대양정비', '(주)대창기업', '마포공업사',
       '원격측정기관', '광주자동차공업사(주)', '화양검사정비사업소', '(주)북의정부자동차', '한일커먼레일',
       '(주)안일자동차써비스', '(주)수지자동차', '교문자동차공업사', '(주)합성자동차', '(주)동양공업사',
       '카프자동차정비(주)', '대림씨에스(주)', '동심자동차주식회사', '21세기자동차정비공업사', '(주)에이원모터스',
       '영포자동차정비', '부천검사소-일신출장', '상신정비공업(주)', '금강자동차정비산업(주)-서울',
       '노원검사소-아세아자동차종합정비', '공단써비스쎈타', '만평자동차정비공장', '예지엠에스(주)', '공단써비스센타',
       '(주)범아자동차', '중원자동차공업사(성남시)', 

### 8\. TB_RSE_STD

In [ ]:
os.listdir(raw4_fold)

['tb_rsd_ge_list_202311071035.csv',
 'tb_rsd_ge_list_ex_202311071035.csv',
 'tb_rsd_ge_sub_202311071035.csv',
 'tb_rse_out_result_202311071036.csv',
 'tb_rse_reg_202311071036.csv',
 'tb_rse_std_202311071036.csv',
 'tb_rsm_org_202311071037.csv',
 'tb_rso_reg_202311071037.csv',
 'tb_us_org_202311071037.csv']

In [255]:
# 0.3s
name = 'tb_rse_std_202311071036'
file_name = f'{name}.csv'
rse_std = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rse_std.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3410 entries, 0 to 3409
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rse_idx            3410 non-null   int64  
 1   rse_reg_no         3410 non-null   object 
 2   rse_how_gb         3410 non-null   object 
 3   gb_co_lim          1306 non-null   float64
 4   gb_hc_lim          1306 non-null   float64
 5   gb_nox_lim         1306 non-null   float64
 6   gb_bh_hp_lim       3410 non-null   float64
 7   db_1mod_lim        3410 non-null   int64  
 8   db_2mod_lim        3410 non-null   int64  
 9   db_3mod_lim        3410 non-null   int64  
 10  db_op_lim          3363 non-null   float64
 11  db_revc_lim        3410 non-null   int64  
 12  gm_co_lim          1305 non-null   float64
 13  gm_hc_lim          1305 non-null   float64
 14  gm_air_over_lim1   3410 non-null   float64
 15  gm_air_over_lim2   3410 non-null   float64
 16  dm_smo_lim         3410 

In [256]:
rse_std.columns

Index(['rse_idx', 'rse_reg_no', 'rse_how_gb', 'gb_co_lim', 'gb_hc_lim',
       'gb_nox_lim', 'gb_bh_hp_lim', 'db_1mod_lim', 'db_2mod_lim',
       'db_3mod_lim', 'db_op_lim', 'db_revc_lim', 'gm_co_lim', 'gm_hc_lim',
       'gm_air_over_lim1', 'gm_air_over_lim2', 'dm_smo_lim', 'dm_revc_lim',
       'gt_co_lim', 'gt_hc_lim', 'gt_air_over_lim1', 'gt_air_over_lim2',
       'dk_smo_lim', 'dk_car_weight_lim', 'dk_op_lim', 'dk_car_type_cd',
       'gi_co_b_lim', 'gi_hc_b_lim', 'gi_co_m_lim', 'gi_hc_m_lim',
       'gi_nox_lim', 'gi_bh_hp_lim', 'gi_air_over_lim1', 'gi_air_over_lim2',
       'rse_time', 'rse_yn', 'rse_user_id', 'usage_cd', 'kagumgi', 'car_year',
       'atm_type_gb', 'combustion_yn', 'car_reg_no'],
      dtype='object')

In [265]:
set(rse_std['rse_reg_no'].unique())

{'H0792019040990001',
 'A0022018062190002',
 'D0212014022890001',
 'D0332018040290001',
 'H0152016062890001',
 'D0202014012390001',
 'A0612014070390001',
 'C0192015081890002',
 'F0032018102390001',
 'A0322017033090003',
 'A0022017011690001',
 '41652016070590004',
 'A0482013100190001',
 'A0032017041890004',
 'H1812013111190001',
 'G0022017103090001',
 'A0322017022790002',
 'H0392015010390002',
 'H0392013102290001',
 'A0122017081190001',
 'A0172020042790003',
 '116c2017121390001',
 '41652020102890001',
 'H0792019112090001',
 'A0222015110690001',
 '119c2016072690002',
 'D0332020071090001',
 'C0042017053090001',
 'F0012017092290001',
 'A0962013092690001',
 'A0962014121190002',
 'A0912016120190001',
 '417l2017032290001',
 'H0392014120990001',
 'A1022016111690003',
 'D0192014040590001',
 'A0322017113090001',
 'A0042013102290002',
 'D0492014011090001',
 'H0202013110590001',
 'N0082014100190001',
 'F0212014102990001',
 'H0152016090290001',
 'A0222017011090001',
 'D0212013111990001',
 'F0262015

In [ ]:
rse_std['rse_reg_no'].describe()

count   4140.00
mean       1.89
std        3.01
min        1.00
25%        1.00
50%        1.00
75%        2.00
max       38.00
Name: rse_reg_no, dtype: float64

### 9\. TB_RSM_ORG

In [ ]:
os.listdir(raw4_fold)

['tb_rsd_ge_list_202311071035.csv',
 'tb_rsd_ge_list_ex_202311071035.csv',
 'tb_rsd_ge_sub_202311071035.csv',
 'tb_rse_out_result_202311071036.csv',
 'tb_rse_reg_202311071036.csv',
 'tb_rse_std_202311071036.csv',
 'tb_rsm_org_202311071037.csv',
 'tb_rso_reg_202311071037.csv',
 'tb_us_org_202311071037.csv']

In [266]:
# 0.3s
name = 'tb_rsm_org_202311071037'
file_name = f'{name}.csv'
rsm_org = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rsm_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   org_id             652 non-null    int64  
 1   org_code           567 non-null    object 
 2   sido               221 non-null    object 
 3   sigungu            221 non-null    object 
 4   receipte_no        652 non-null    object 
 5   receipte_date      652 non-null    int64  
 6   send_date          651 non-null    float64
 7   org_nm             652 non-null    object 
 8   org_owner          652 non-null    object 
 9   org_addr1          652 non-null    object 
 10  org_addr2          492 non-null    object 
 11  org_owner_tel_no   652 non-null    object 
 12  org_post_no        652 non-null    object 
 13  org_simple_addr    652 non-null    object 
 14  org_tel_no         652 non-null    object 
 15  ins_item_group_id  0 non-null      float64
 16  rec_date           652 non

In [267]:
rsm_org.columns

Index(['org_id', 'org_code', 'sido', 'sigungu', 'receipte_no', 'receipte_date',
       'send_date', 'org_nm', 'org_owner', 'org_addr1', 'org_addr2',
       'org_owner_tel_no', 'org_post_no', 'org_simple_addr', 'org_tel_no',
       'ins_item_group_id', 'rec_date', 'rec_nm', 'las_chg_date', 'las_chg_nm',
       'co', 'ch', 'no', 'air', 'smoke', 'org_reg_no', 'us_org_id',
       'org_status', 'org_gld', 'org_ele_status', 'rsm_type_cd', 'rsm_join_yn',
       'rsm_join_org_nm', 'owner_reg_no', 'nos', 'tcp_yn', 'sido_cd',
       'sigungu_cd', 'x_coord', 'y_coord', 'sido_nm', 'sigungu_nm'],
      dtype='object')

In [274]:
set(rsm_org['rsm_join_org_nm'].unique())

{nan, '성동그린 모터스'}

In [ ]:
rsm_org['rse_reg_no'].describe()

count   4140.00
mean       1.89
std        3.01
min        1.00
25%        1.00
50%        1.00
75%        2.00
max       38.00
Name: rse_reg_no, dtype: float64

### 10\. TB_RSO_REG

In [ ]:
os.listdir(raw4_fold)

['tb_rsd_ge_list_202311071035.csv',
 'tb_rsd_ge_list_ex_202311071035.csv',
 'tb_rsd_ge_sub_202311071035.csv',
 'tb_rse_out_result_202311071036.csv',
 'tb_rse_reg_202311071036.csv',
 'tb_rse_std_202311071036.csv',
 'tb_rsm_org_202311071037.csv',
 'tb_rso_reg_202311071037.csv',
 'tb_us_org_202311071037.csv']

In [275]:
# 0.3s
name = 'tb_rso_reg_202311071037'
file_name = f'{name}.csv'
rso_reg = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
rso_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4192 entries, 0 to 4191
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rso_idx          4192 non-null   int64  
 1   ge_idx           4191 non-null   float64
 2   sub_idx          4186 non-null   float64
 3   reg_no           4192 non-null   int64  
 4   up_rso_date      4191 non-null   float64
 5   up_exp_date      4192 non-null   int64  
 6   rso_reason       2848 non-null   object 
 7   rso_type         4189 non-null   float64
 8   doc_name         3994 non-null   object 
 9   doc_birth        2657 non-null   object 
 10  doc_post         2854 non-null   object 
 11  doc_addr1        2827 non-null   object 
 12  doc_addr2        2825 non-null   object 
 13  doc_tel_no       2853 non-null   object 
 14  doc_last_site    2923 non-null   object 
 15  doc_last_time    2923 non-null   object 
 16  rso_reg_status   4192 non-null   object 
 17  rse_status    

In [276]:
rso_reg.columns

Index(['rso_idx', 'ge_idx', 'sub_idx', 'reg_no', 'up_rso_date', 'up_exp_date',
       'rso_reason', 'rso_type', 'doc_name', 'doc_birth', 'doc_post',
       'doc_addr1', 'doc_addr2', 'doc_tel_no', 'doc_last_site',
       'doc_last_time', 'rso_reg_status', 'rse_status', 'reg_region',
       'reg_tel_no', 'org_id', 'rsd_month', 'rec_date', 'rec_nm',
       'las_chg_date', 'las_chg_nm', 'reg_no_full', 'rsd_idx',
       'rso_reason_stop', 'stop_from_date', 'stop_to_date', 'rso_stop_type',
       'print_yn', 'print_date', 'print_rec', 'rso_stop_status', 'r_reason'],
      dtype='object')

In [284]:
set(rso_reg['rse_status'].unique())

{'N', 'Y'}

In [278]:
rso_reg['reg_no'].describe()

count   4192.00
mean      51.97
std       62.52
min        1.00
25%        8.00
50%       31.00
75%       73.00
max      343.00
Name: reg_no, dtype: float64

### 11\. TB_US_ORG

In [ ]:
os.listdir(raw4_fold)

['tb_rsd_ge_list_202311071035.csv',
 'tb_rsd_ge_list_ex_202311071035.csv',
 'tb_rsd_ge_sub_202311071035.csv',
 'tb_rse_out_result_202311071036.csv',
 'tb_rse_reg_202311071036.csv',
 'tb_rse_std_202311071036.csv',
 'tb_rsm_org_202311071037.csv',
 'tb_rso_reg_202311071037.csv',
 'tb_us_org_202311071037.csv']

In [285]:
# 0.3s
name = 'tb_us_org_202311071037'
file_name = f'{name}.csv'
us_org = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep='|')
us_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15711 entries, 0 to 15710
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   org_id            15711 non-null  object 
 1   biz_cd            15711 non-null  object 
 2   org_nm            15711 non-null  object 
 3   user_grp_cd       15711 non-null  object 
 4   area_cd           15711 non-null  object 
 5   owner_nm          15711 non-null  object 
 6   owner_post_no     10714 non-null  object 
 7   owner_addr1       10756 non-null  object 
 8   owner_addr2       10243 non-null  object 
 9   org_post_no       14659 non-null  object 
 10  org_addr1         14755 non-null  object 
 11  org_addr2         13466 non-null  object 
 12  org_tel_no        12752 non-null  object 
 13  dedicate_no       1 non-null      float64
 14  dedicate_date     1 non-null      float64
 15  reg_date          15072 non-null  object 
 16  reg_id            14950 non-null  object

In [286]:
us_org.columns

Index(['org_id', 'biz_cd', 'org_nm', 'user_grp_cd', 'area_cd', 'owner_nm',
       'owner_post_no', 'owner_addr1', 'owner_addr2', 'org_post_no',
       'org_addr1', 'org_addr2', 'org_tel_no', 'dedicate_no', 'dedicate_date',
       'reg_date', 'reg_id', 'sido_cd', 'sigungu_cd', 'max_user_cnt',
       'auto_lock_period'],
      dtype='object')

In [293]:
set(us_org['auto_lock_period'].unique())

{90, 365}

In [292]:
us_org['max_user_cnt'].describe()

count   14070.00
mean        5.82
std         2.48
min         0.00
25%         5.00
50%         5.00
75%         5.00
max       150.00
Name: max_user_cnt, dtype: float64

### 12\. [✔] TB_RSE_DET_EX

In [37]:
# 0.3s
name = 'tb_rse_det_ex_202312061021'
file_name = f'{name}.csv'
rse_det_ex = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep=',')
rse_det_ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 675495 entries, 0 to 675494
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   rse_idx          675495 non-null  int64  
 1   det_date         0 non-null       float64
 2   repair_dev_cd    675495 non-null  object 
 3   repair_part_cd   675495 non-null  object 
 4   repair_part1_yn  675495 non-null  object 
 5   repair_part2_yn  675495 non-null  object 
 6   repair_part3_yn  675495 non-null  object 
 7   repair_part4_yn  675495 non-null  object 
 8   rec_date         675495 non-null  int64  
 9   rec_nm           112410 non-null  object 
 10  las_chg_date     37849 non-null   float64
 11  las_chg_nm       0 non-null       float64
dtypes: float64(3), int64(2), object(7)
memory usage: 61.8+ MB


In [235]:
rse_det_ex.columns.to_list()

['rse_idx',
 'det_date',
 'repair_dev_cd',
 'repair_part_cd',
 'repair_part1_yn',
 'repair_part2_yn',
 'repair_part3_yn',
 'repair_part4_yn',
 'rec_date',
 'rec_nm',
 'las_chg_date',
 'las_chg_nm']

In [236]:
det_chc = {
    'rse_idx':'일련번호', 
    'det_date':'상세일자', 
    'repair_dev_cd':'수리디이브이_코드', 
    'repair_part_cd':'수리부분코드', 
    'repair_part1_yn':'수리부분1유무', 
    'repair_part2_yn':'수리부분2유무', 
    'repair_part3_yn':'수리부분3유무', 
    'repair_part4_yn':'수리부분4유무', 
    'rec_date':'생성일자', 
    'rec_nm':'생성자명', 
    'las_chg_date':'최종수정일자', 
    'las_chg_nm':'최종수정자명', 
}
rse_det_exr = rse_det_ex.rename(columns=det_chc)
rse_det_exr.columns

Index(['일련번호', '상세일자', '수리디이브이_코드', '수리부분코드', '수리부분1유무', '수리부분2유무', '수리부분3유무',
       '수리부분4유무', '생성일자', '생성자명', '최종수정일자', '최종수정자명'],
      dtype='object')

### 전문정비내역수정
- 전문정비사업자_2회부적합정비결과등록_(20200101~20230731)_v10

In [74]:
# 0.3s
name = '전문정비내역수정_20231208'
file_name = f'{name}.csv'
err_biz = pd.read_csv(os.path.join(raw4_fold, file_name), low_memory=False, sep=',')
err_biz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26102 entries, 0 to 26101
Columns: 207 entries, No to 배전기(Distributor) 점검/조정
dtypes: float64(22), int64(3), object(182)
memory usage: 41.2+ MB


In [75]:
err_biz.columns

Index(['No', '접수번호', '정비일자', '차대번호', '차량번호', '차량소유자명', '차종', '차명', '차량연료',
       '차량연식(년도)',
       ...
       '점화배선 분해/탈착', '공기유량센서 분해/탈착', '노즐(디젤인젝터포함) 교환', '엔진정비 점검/조정',
       '블로바이가스 호스 점검/조정', 'EGR밸브 점검/조정', '소음기 교환',
       '정화조절밸브(Purge Control Valve) 정비불가능', '소음기 점검/조정',
       '배전기(Distributor) 점검/조정'],
      dtype='object', length=207)

### 전문정비 사업자 현황 from 환경공단

In [216]:
temp4_fold = 'D:/data/processing/big3/raw/BD4/전문정비사업자 현황 from 환경공단'

In [217]:
err_biz_curstt = pd.DataFrame()
for one in os.listdir(temp4_fold):
    if 'xls' in one:
        temp = pd.read_excel(os.path.join(temp4_fold, one))
        err_biz_curstt = pd.concat([err_biz_curstt, temp], ignore_index=False)
err_biz_curstt.shape

(497, 4)

In [218]:
err_biz_curstt.head()

,전문정비사업자 명,주소,전화번호,시도
0,강원대우자동차(주),강원도 강릉시 강변로534번길 11,033-653-3516,강원
1,강원부란자,강원도 춘천시 근화길 11 (근화동),033-252-0864,강원
2,그린 기능장 카 서비스,강원도 원주시 우산로 21 (우산동),033-735-8294,강원
3,기아자동차(주) 원주서비스센터,강원도 원주시 우산로 233,033-743-8585,강원
4,기아자동차(주)강릉서비스센터,강원도 강릉시 강변로604번길 11-4,033-650-8502,강원


#### [출력] 전국 전문정비사업자 현황

In [213]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20231213'

In [214]:
# err_biz_curstt.to_excel(os.path.join(temp4_fold, f'전국_전문정비사업자_현황_{today_date}.xlsx'), index=False)

In [238]:
err_biz_curstt = pd.read_excel(os.path.join(raw4_fold, '전국_전문정비사업자_현황_20231213.xlsx'))
err_biz_curstt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   전문정비사업자 명  497 non-null    object
 1   주소         497 non-null    object
 2   전화번호       497 non-null    object
 3   시도         497 non-null    object
dtypes: object(4)
memory usage: 15.7+ KB


## 전처리

### rsd_ge_list_ex와 rse_det_ex 병합
- rse_det_ex : 'repair_part1_yn', 'repair_part2_yn', 'repair_part3_yn', 'repair_part4_yn' 중 단 하나라도 'Y'가 있는 샘플만 추출
- rsd_ge_list_ex(ge_ldx), rse_det_ex(rse_idx) 기준 병합

In [200]:
rse_det_ex['repair_part1_yn'].unique()

array(['N', 'Y'], dtype=object)

In [201]:
rse_det_ex01 = rse_det_ex[
    (rse_det_ex['repair_part1_yn'] == 'Y') | 
    (rse_det_ex['repair_part2_yn'] == 'Y') | 
    (rse_det_ex['repair_part3_yn'] == 'Y') | 
    (rse_det_ex['repair_part4_yn'] == 'Y')
    ].reset_index(drop=True)
rse_det_ex01.shape

(267003, 12)

In [202]:
rse_det_ex01.shape, len(rse_det_ex01['rse_idx'].unique())

((267003, 12), 117540)

In [203]:
rse_det_ex01.loc[rse_det_ex01['rse_idx'].duplicated(keep=False), ['rse_idx', 'repair_dev_cd', 'repair_part_cd', 'repair_part1_yn', 'repair_part2_yn', 'repair_part3_yn', 'repair_part4_yn', 'rec_date']]

,rse_idx,repair_dev_cd,repair_part_cd,repair_part1_yn,repair_part2_yn,repair_part3_yn,repair_part4_yn,rec_date
1,14,E,E1,Y,N,N,N,20130807
2,14,E,E2,Y,N,N,N,20130807
3,19,B,B5,Y,N,N,N,20130808
4,19,E,E1,Y,N,N,N,20130808
5,19,E,E2,Y,N,N,N,20130808
...,...,...,...,...,...,...,...,...
266997,117445,D,D2,N,Y,N,N,20230220
266998,117206,J,J1,N,N,Y,N,20230203
267000,117445,D,D3,N,Y,N,N,20230220
267001,117219,H,H1,Y,N,N,N,20230203


In [191]:
rse_det_ex.shape

(675495, 12)

In [192]:
rse_det_ex01.shape

(267003, 12)

In [57]:
rse_det_ex01.head()

,rse_idx,det_date,repair_dev_cd,repair_part_cd,repair_part1_yn,repair_part2_yn,repair_part3_yn,repair_part4_yn,rec_date,rec_nm,las_chg_date,las_chg_nm
0,9,NaN,B,B1,Y,N,N,N,20130806,NaN,NaN,NaN
1,14,NaN,E,E1,Y,N,N,N,20130807,NaN,20130807.00,NaN
2,14,NaN,E,E2,Y,N,N,N,20130807,NaN,20130807.00,NaN
3,19,NaN,B,B5,Y,N,N,N,20130808,NaN,NaN,NaN
4,19,NaN,E,E1,Y,N,N,N,20130808,NaN,NaN,NaN


In [61]:
rsd_ge_list_ex.shape, len(rsd_ge_list_ex['ge_idx'].unique())

((119898, 70), 119898)

In [60]:
rse_det_ex01.shape, len(rse_det_ex01['rse_idx'].unique())

((267003, 12), 117540)

In [84]:
gd = rsd_ge_list_ex.merge(rse_det_ex, left_on='ge_idx', right_on='rse_idx', how='left')
gd.shape

(675026, 82)

In [85]:
gd.columns

Index(['ge_idx', 'rsd_date', 'car_reg_no', 'car_ser_no', 'car_nm',
       'car_type_cd', 'car_year', 'usage_cd', 'car_fuel_cd', 'car_weight',
       'tot_weight', 'engine_capacity', 'engine_power', 'engine_type',
       'mileage', 'make_reg_date', 'first_reg_date', 'smo_chk_exp_date',
       'sido', 'sigungu', 'org_id', 'cer_exp_year', 'cer_exp_km', 'mistake_yn',
       'repair_name', 'repair_comment', 'car_age', 'det_yn', 'rec_date_x',
       'rec_nm_x', 'las_chg_nm_x', 'las_chg_date_x', 'cer_repair_yn',
       'reg_region', 'doc_name', 'reg_no', 'print_yn', 'print_date',
       'print_rec_nm', 'sido_cd', 'sigungu_cd', 'sido_nm', 'sigungu_nm',
       'smo_chk_yn', 'smo_chk_date', 'smo_chk_org_nm', 'smo_chk_method',
       'smo_chk_judge_yn', 'unload_smo_val1', 'unload_smo_lim1',
       'unload_smo_judge1', 'unload_smo_val2', 'unload_smo_lim2',
       'unload_smo_judge2', 'unload_smo_val3', 'unload_smo_lim3',
       'unload_smo_judge3', 'unload_smo_val4', 'unload_smo_lim4',
       'unl

In [246]:
len(gd['ge_idx'].unique()), len(gd['rse_idx'].unique())

(119898, 117267)

In [248]:
len(gd['ge_idx'].unique()) - len(gd['rse_idx'].unique())

2631

In [249]:
gd['ge_idx'].isnull().sum()

0

In [247]:
gd['rse_idx'].isnull().sum()

2632

In [224]:
# smo_chk_org_nm : 검사소명
ggrp1 = gd.groupby(['sido_nm']).agg({'smo_chk_org_nm':lambda x:len(x.unique())}).reset_index()
ggrp1

,sido_nm,smo_chk_org_nm
0,강원도,59
1,강원특별자치도,3
2,경기도,607
3,경상남도,198
4,경상북도,250
5,광주광역시,155
6,대구광역시,228
7,대전광역시,158
8,부산광역시,250
9,서울특별시,462


In [225]:
gd.loc[gd['sido_nm'] == '강원특별자치도', 'smo_chk_org_nm']

665128    진성공업사(강원)
665129    진성공업사(강원)
665398    진성공업사(강원)
665462    의정부자동차검사소
665463    의정부자동차검사소
665464    의정부자동차검사소
665465    의정부자동차검사소
670199     춘천자동차검사소
674826     춘천자동차검사소
Name: smo_chk_org_nm, dtype: object

In [226]:
gd['smo_chk_org_nm'].unique()

array(['스피드1급검사정비', '천안자동차검사소', '성서자동차검사정비', ..., '경산서비스기아오토큐(주)',
       '(주)완주자동차정비사업소', '(주)카지트'], dtype=object)

In [86]:
gd['rec_date_y'].dtype

dtype('float64')

In [87]:
sorted_gd = gd[(gd['rec_date_y'] >= 20200101) & (gd['rec_date_y'] <= 20230731)].reset_index(drop=True)
sorted_gd.shape

(75380, 82)

In [88]:
# smo_chk_org_nm : 검사소명
rgrp1 = rsd_ge_list_ex.groupby(['sido_nm']).agg({'smo_chk_org_nm':lambda x:len(x.unique())}).reset_index()
rgrp1

,sido_nm,smo_chk_org_nm
0,강원도,59
1,강원특별자치도,3
2,경기도,607
3,경상남도,198
4,경상북도,250
5,광주광역시,155
6,대구광역시,228
7,대전광역시,158
8,부산광역시,250
9,서울특별시,462


In [89]:
# smo_chk_org_nm : 검사소명
gdgrp1 = sorted_gd.groupby(['sido_nm']).agg({'smo_chk_org_nm':lambda x:len(x.unique())}).reset_index()
gdgrp1 = gdgrp1.sort_values(['sido_nm'])
gdgrp1

,sido_nm,smo_chk_org_nm
0,강원도,36
1,강원특별자치도,2
2,경기도,357
3,경상남도,149
4,경상북도,190
5,광주광역시,93
6,대구광역시,157
7,대전광역시,92
8,부산광역시,147
9,서울특별시,253


In [82]:
ebgrp1= err_biz.groupby(['시도(전문정비사업자)']).agg({'검사소명':lambda x:len(x.unique())}).reset_index()
ebgrp1 = ebgrp1.sort_values(['시도(전문정비사업자)'])
ebgrp1

,시도(전문정비사업자),검사소명
0,강원도,49
1,경기도,162
2,경상남도,97
3,경상북도,121
4,광주광역시,55
5,대구광역시,110
6,대전광역시,50
7,부산광역시,58
8,서울특별시,129
9,세종특별자치시,23


In [215]:
gd['smo_chk_org_nm'].unique()

array(['스피드1급검사정비', '천안자동차검사소', '성서자동차검사정비', ..., '경산서비스기아오토큐(주)',
       '(주)완주자동차정비사업소', '(주)카지트'], dtype=object)

In [220]:
err_biz_curstt.shape, len(err_biz_curstt['전문정비사업자 명'].unique())

((497, 4), 497)

In [219]:
len(set(err_biz_curstt['전문정비사업자 명'].to_list()) - set(gd['smo_chk_org_nm'].unique()))

402

In [243]:
len(gd['smo_chk_org_nm'].unique())

1708

In [222]:
len(set(err_biz_curstt['전문정비사업자 명'].to_list()) & set(gd['smo_chk_org_nm'].unique()))

95

In [244]:
gdgrp1 = gd.drop_duplicates('smo_chk_org_nm').groupby(['sido_nm'], dropna=False).agg({'smo_chk_org_nm':lambda x:len(x.unique())}).reset_index()
gdgrp1.loc[gdgrp1.shape[0], ['sido_nm', 'smo_chk_org_nm']] = ['합계', gdgrp1['smo_chk_org_nm'].sum()]
gdgrp1

,sido_nm,smo_chk_org_nm
0,강원도,11.00
1,경기도,225.00
2,경상남도,86.00
3,경상북도,107.00
4,광주광역시,58.00
5,대구광역시,113.00
6,대전광역시,64.00
7,부산광역시,74.00
8,서울특별시,167.00
9,세종특별자치시,7.00


In [252]:
rse_reg.columns

Index(['rse_idx', 'rso_idx', 'car_reg_no', 'car_ser_no', 'org_id', 'org_nm',
       'reg_no', 'reg_no_full', 'rse_date', 'rse_org_cd', 'rse_org_nm',
       'rse_reg_no', 'rse_reg_no_full', 'rse_pay_gb', 'rse_how_gb',
       'rse_etc_gb', 'rse_status', 'rec_date', 'rec_nm', 'las_chg_date',
       'las_chg_nm', 'det_yn', 'print_yn', 'print_date', 'rse_amt',
       'scale_gb'],
      dtype='object')

In [253]:
# 정밀검사소명, 업체명(검사업체)
len(set(gd['smo_chk_org_nm'].unique()) - set(rse_reg['rse_org_nm'].unique()))

1542

In [255]:
len(gd['org_id'].unique())

487

In [254]:
# 전문정비사업자코드, 업체코드(검사업체)
len(set(gd['org_id'].unique()) - set(rse_reg['rse_org_cd'].unique()))

485

## 분석

# 과제5

## 로드

### 등록정보

In [ ]:
# 5m 31.4s
car = wd.export_to_pandas("SELECT VIN, VHCL_ERSR_YN, FRST_REG_YMD, MANP_MNG_NO, EXHST_GAS_GRD_CD, VHCTY_CD FROM STD_CEG_CAR_MIG;")
car_ch_col = {
    'VIN':'차대번호', 
    'VHCL_ERSR_YN':'차량말소여부',
    'FRST_REG_YMD':'최초등록일자',
    'MANP_MNG_NO':'제원관리번호', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'VHCTY_CD':'차종', 

    # 'BSPL_STDG_CD':'법정동코드', 
    # 'EXHST_GAS_CERT_NO':'배출가스인증번호',
    # 'YRIDNW':'차량연식', 
    # 'PURPS_CD2':'용도', 
    # 'VHCL_FBCTN_YMD':'제작일자', 
    # 'VHCL_MNG_NO':'차량관리번호', 
    # 'VHRNO':'자동차등록번호',
    # 'EXTGAS_INSP_VLD_YMD':'검사유효일',
    # 'VHCL_OWNR_CL_CD':'소유자구분',  
}
carr = car.rename(columns=car_ch_col)

print(carr.columns)
print('data load : STD_CEG_CAR_MIG')

### 제원정보

In [ ]:
# 4.1s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, MNFCTR_NM, VHCTY_TY_CD2 FROM STD_CEG_CAR_SRC_MIG;")
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'MNFCTR_NM':'제작사명', 
    'VHCTY_TY_CD2':'차종유형', 
    
    # 'VHCNM':'차명', 
    # 'VHCL_FRM':'자동차형식', 
    # 'EGIN_TY':'엔진형식', 
    # 'VHCTY_CL_CD':'차종분류',
    # 'TOTL_WGHT':'총중량',
    # 'CRYNG_WGHT':'적재중량',
    # 'DSPLVL':'배기량', 
    # 'EGIN_OTPT':'엔진출력',
}
srcr = src.rename(columns=src_ch_col)

print(srcr.columns)
print('data load : STD_CEG_CAR_SRC_MIG')

### 정기검사

In [ ]:
# 3m 34.9s
# jgt = wb.export_to_pandas("SELECT VIN, FDRM_INSP_YMD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_DRVNG_DSTNC FROM STD_TB_JGT_HIS WHERE ROWNUM <= 10000;") # 테스트용
jgt = wd.export_to_pandas("SELECT VIN, FDRM_INSP_YMD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3 FROM STD_TB_JGT_HIS;")
jgt_ch_col = {
    'VIN':'차대번호', 
    'FDRM_INSP_YMD':'검사일자',
    'FDRM_INSP_JGMT':'검사판정',
    'FDRM_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'FDRM_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'FDRM_NLOD_SMO_MSTVL3':'무부하매연측정치3', 

    # 'FDRM_DRVNG_DSTNC':'주행거리',
    # 'FDRM_INSP_INSP_MTHD_CD':'검사방법', 
    # 'FDRM_INSP_KND_CD':'검사종류', 
    # 'FDRM_NLOD_SMO_MSTVL4':'무부하매연측정치4', 
    # 'FDRM_NLOD_SMO_MSTVL5':'무부하매연측정치5', 
    # 'FDRM_NLOD_SMO_MSTVL6':'무부하매연측정치6', 
    # 'FDRM_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    # 'FDRM_NLOD_SMO_JT_YN2':'무부하매연판정2', 
    # 'FDRM_NLOD_SMO_JT_YN3':'무부하매연판정3', 
    # 'FDRM_NLOD_SMO_JT_YN4':'무부하매연판정4', 
    # 'FDRM_NLOD_SMO_JT_YN5':'무부하매연판정5', 
    # 'FDRM_NLOD_SMO_JT_YN6':'무부하매연판정6', 
    # 'FDRM_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
    # 'FDRM_NLOD_SMO_PRMVL2':'무부하매연허용치2', 
    # 'FDRM_NLOD_SMO_PRMVL3':'무부하매연허용치3', 
    # 'FDRM_NLOD_SMO_PRMVL4':'무부하매연허용치4', 
    # 'FDRM_NLOD_SMO_PRMVL5':'무부하매연허용치5', 
    # 'FDRM_NLOD_SMO_PRMVL6':'무부하매연허용치6', 
}
jgtr = jgt.rename(columns=jgt_ch_col)

print(jgtr.columns)
print('data load : STD_CEG_CAR_SRC_MIG')

### 정밀검사

In [ ]:
# 10m 30.8s
# eet = wb.export_to_pandas("SELECT VIN, PRCINSP_YMD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PRCINSP_DRVNG_DSTNC FROM STD_TB_EET_HIS_ME_ME WHERE ROWNUM <= 10000;") # 테스트용
eet = wd.export_to_pandas("SELECT VIN, PRCINSP_YMD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3 FROM STD_TB_EET_HIS_ME;")
eet_ch_col = {
    'VIN':'차대번호', 
    'PRCINSP_YMD':'검사일자',
    'PRCINSP_JGMT':'검사판정', 
    'PREC_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'PREC_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'PREC_NLOD_SMO_MSTVL3':'무부하매연측정치3', 

    # 'PRCINSP_DRVNG_DSTNC':'주행거리', 
    # 'PRCINSP_MSRMT_MTHD_CD':'검사방법', 
    # 'PRCINSP_KND_CD':'검사종류', 
    # 'PREC_NLOD_SMO_MSTVL4':'무부하매연측정치4', 
    # 'PREC_NLOD_SMO_MSTVL5':'무부하매연측정치5', 
    # 'PREC_NLOD_SMO_MSTVL6':'무부하매연측정치6', 
    # 'PREC_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    # 'PREC_NLOD_SMO_JT_YN2':'무부하매연판정2', 
    # 'PREC_NLOD_SMO_JT_YN3':'무부하매연판정3', 
    # 'PREC_NLOD_SMO_JT_YN4':'무부하매연판정4', 
    # 'PREC_NLOD_SMO_JT_YN5':'무부하매연판정5', 
    # 'PREC_NLOD_SMO_JT_YN6':'무부하매연판정6', 
    # 'PREC_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
    # 'PREC_NLOD_SMO_PRMVL2':'무부하매연허용치2', 
    # 'PREC_NLOD_SMO_PRMVL3':'무부하매연허용치3', 
    # 'PREC_NLOD_SMO_PRMVL4':'무부하매연허용치4', 
    # 'PREC_NLOD_SMO_PRMVL5':'무부하매연허용치5', 
    # 'PREC_NLOD_SMO_PRMVL6':'무부하매연허용치6', 
}
eetr = eet.rename(columns=eet_ch_col)

print(eetr.columns)
print('data load : STD_TB_EET_HIS_ME')

### 구조변경검사

In [18]:
# 3.6s
gb = wd.export_to_pandas("SELECT VIN, PRCINSP_YMD FROM STD_DLM_TB_ERP_GUBUN_BAECHUL;")
gb_ch_col = {
    'VIN':'차대번호', 
    'PRCINSP_YMD':'정밀검사일자',
}
gbr = gb.rename(columns=gb_ch_col)

print(gbr.columns)
print('data load : STD_DLM_TB_ERP_GUBUN_BAECHUL')

Index(['차대번호', '정밀검사일자'], dtype='object')
data load : STD_DLM_TB_ERP_GUBUN_BAECHUL


### RSD 수시점검

In [17]:
# 49.4s
rsd = wd.export_to_pandas("SELECT VIN, MSRMT_YMD, FUEL_CD, COX_MEVLU, HC_MEVLU, NOX_MEVLU FROM STD_TB_RSD_FINAL;")
rsd_ch_col = {
    'VIN':'차대번호', 
    'MSRMT_YMD':'측정일자',
    'FUEL_CD':'연료', 
    'COX_MEVLU':'일산화탄소측정치', 
    'HC_MEVLU':'탄화수소측정치', 
    'NOX_MEVLU':'질소산화물측정치', 
}
rsdr = rsd.rename(columns=rsd_ch_col)

print(rsdr.columns)
print('data load : STD_TB_RSD_FINAL')

Index(['차대번호', '측정일자', '연료코드', '일산화탄소측정치', '탄화수소측정치', '질소산화물측정치'], dtype='object')
data load : STD_TB_RSD_FINAL


### 저감장치 부착이력

In [19]:
# 3.0s
att = wd.export_to_pandas("SELECT VIN, RDCDVC_SE_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분',

    # 'RDCDVC_KND_CD':'저감장치종류',
}
attr = att.rename(columns=att_ch_col)

print(attr.columns)
print('data load : STD_DLM_TB_ERP_ATT_HIS')

Index(['차대번호', '저감장치구분'], dtype='object')
data load : STD_DLM_TB_ERP_ATT_HIS


### 구조변경검사(STD_DLM_TB_ERP_GUBUN_BAECHUL)

In [133]:
# # 11.7s
# name = 'STD_DLM_TB_ERP_GUBUN_BAECHUL(20231011)'
# file_name = f'{name}.csv'
# gb = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False)
# gb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1321945 entries, 0 to 1321944
Data columns (total 39 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   AIR_PSSR                   1152822 non-null  float64
 1   AIR_TMPRT                  1152822 non-null  float64
 2   CO2_VAL                    1110507 non-null  float64
 3   DRVNG_DSTNC                1320563 non-null  float64
 4   FNCINSP_JGMT_YN            1321919 non-null  object 
 5   INSPOF_NM                  1321945 non-null  object 
 6   INSPOF_RPRSV_NM            1318198 non-null  object 
 7   MAXM_OTPT_MSRMT_VAL        445619 non-null   float64
 8   MAXM_OTPT_PRMT_VAL         544747 non-null   float64
 9   O2_VAL                     1110507 non-null  float64
 10  PRCINSP_YMD                1321945 non-null  int64  
 11  PRCINSP_VLD_YMD            127529 non-null   float64
 12  PRCINSP_VLD_WHL            1321945 non-null  object 
 13  PRCINSP_JGMT

In [134]:
# gb.shape

(1321945, 39)

In [135]:
# gb.columns

Index(['AIR_PSSR', 'AIR_TMPRT', 'CO2_VAL', 'DRVNG_DSTNC', 'FNCINSP_JGMT_YN',
       'INSPOF_NM', 'INSPOF_RPRSV_NM', 'MAXM_OTPT_MSRMT_VAL',
       'MAXM_OTPT_PRMT_VAL', 'O2_VAL', 'PRCINSP_YMD', 'PRCINSP_VLD_YMD',
       'PRCINSP_VLD_WHL', 'PRCINSP_JGMT', 'PRCINSP_KND', 'PRCINSP_RCPT_NO',
       'RINSP_WHL', 'VHCL_PURPS_CD', 'VHRNO', 'VIN',
       'STRCT_CHG_INSP_MSRMT_MTHD', 'NOLOD_SMO_MSRMT_VAL1',
       'NOLOD_SMO_PRMT_VAL1', 'NOLOD_SMO_MSRMT_VAL2', 'NOLOD_SMO_PRMT_VAL2',
       'NOLOD_SMO_MSRMT_VAL3', 'NOLOD_SMO_PRMT_VAL3', 'NOLOD_SMO_MSRMT_VAL4',
       'NOLOD_SMO_PRMT_VAL4', 'NOLOD_SMO_MSRMT_VAL5', 'NOLOD_SMO_PRMT_VAL5',
       'NOLOD_SMO_MSRMT_VAL6', 'NOLOD_SMO_PRMT_VAL6', 'NOLOD_SMO_JGMT_YN1',
       'NOLOD_SMO_JGMT_YN2', 'NOLOD_SMO_JGMT_YN3', 'NOLOD_SMO_JGMT_YN4',
       'NOLOD_SMO_JGMT_YN5', 'NOLOD_SMO_JGMT_YN6'],
      dtype='object')

#### 데이터 확인

In [137]:
# set(gb.FNCINSP_JGMT_YN.unique())

{' ', '8', 'N', 'Y', nan}

In [139]:
# set(gb.VHCL_PURPS_CD.unique())

{'A20B', 'A20O', 'A20P', 'A20T'}

In [141]:
# set(gb.STRCT_CHG_INSP_MSRMT_MTHD.unique())

{'무부하검사(TSI)',
 '무부하검사(급가속)',
 '무부하검사(정지가동)',
 '부하검사(ASM-Idling)',
 '부하검사(ASM2525)',
 '부하검사(KD-147)',
 '부하검사(LUG DOWN)'}

In [143]:
# set(gb.NOLOD_SMO_JGMT_YN6.unique())

{' ', '0', 'N', 'Y', nan}

##### [출력] 컬럼, 데이터 정보들

In [144]:
# temp = gb.isnull().sum().reset_index()
# temp = temp.rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
# temp['총 행수'] = gb.shape[0]
# temp['번호'] = [x for x in range(1, temp.shape[0] + 1)]
# temp = temp[['번호', '컬럼명', '빈 값 개수', '총 행수']]
# temp

,번호,컬럼명,빈 값 개수,총 행수
0,1,AIR_PSSR,169123,1321945
1,2,AIR_TMPRT,169123,1321945
2,3,CO2_VAL,211438,1321945
3,4,DRVNG_DSTNC,1382,1321945
4,5,FNCINSP_JGMT_YN,26,1321945
5,6,INSPOF_NM,0,1321945
6,7,INSPOF_RPRSV_NM,3747,1321945
7,8,MAXM_OTPT_MSRMT_VAL,876326,1321945
8,9,MAXM_OTPT_PRMT_VAL,777198,1321945
9,10,O2_VAL,211438,1321945


In [145]:
# today_date = datetime.today().strftime('%Y.%m.%d')
# today_date

'2023.11.02'

In [146]:
# temp.to_excel(os.path.join(an5_fold, f'구조변경_데이터정보({today_date}).xlsx'), index=False)

## 전처리